# ✅ 1. Define the libraries and upload the dataset

In [ ]:
# Step 1: Create a new environment
!python -m venv cleanenv

# Step 2: Activate it
# On Windows:
!cleanenv\Scripts\activate
# On Mac/Linux:
#source cleanenv/bin/activate

# Step 3: Install only what you need
!pip install numpy==1.26.4 scipy==1.13.0 scikit-learn==1.5.0 imbalanced-learn==0.13.0 tensorflow==2.18.0

In [ ]:
# GOOD (pick one)
import torch                         # PyTorch only
# OR
import tensorflow as tf              # TensorFlow only
# OR
import jax                           # JAX only

In [ ]:
# Import TF first so cuDNN is registered once
import tensorflow as tf
import torch

In [ ]:
import numpy, scipy, sklearn, imblearn, tensorflow as tf

print("numpy:", numpy.__version__)
print("scipy:", scipy.__version__)
print("scikit-learn:", sklearn.__version__)
print("imbalanced-learn:", imblearn.__version__)
print("tensorflow:", tf.__version__)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.combine import SMOTEENN, SMOTETomek
from sklearn.utils import resample
from collections import Counter

This script creates **hourly-level dynamic features** for each patient during **Ramadan** using continuous glucose monitoring (CGM) data and lifestyle metrics (activity, sleep, physiology) from wearable devices.
It’s part of a preprocessing pipeline for modeling glucose behavior and hypoglycemia risk.

Here’s a complete explanation 👇

---

## 🧩 **Overall Goal**

To transform raw timestamped CGM and wearable data into **hourly summarized features** that represent glucose dynamics, lifestyle behavior, and physiological activity during Ramadan — ready for statistical or machine-learning analysis.

---

## 🧭 **1️⃣ Load and Parse Data**

* Loads the file:

  ```
  intraday_with_visits.csv
  ```

  which includes per-minute or per-sample CGM and Huawei sensor data.
* Converts all timestamps (`start`, `date`) to datetime format.
* Extracts:

  * `hour` → the nearest hour (e.g., 14:00, 15:00).
  * `hour_of_day` → the hour index (0–23).

👉 *Purpose:* Prepare a unified hourly timeline for every patient.

---

## 📆 **2️⃣ Filter for Ramadan Period**

* Keeps only data between **March 22 – April 19, 2023**.
* Ensures the dataset includes `cgm` readings (continuous glucose values).
* Adds a **binary flag `hypo`** = `True` when CGM ≤ 70 mg/dL (hypoglycemia reading).

👉 *Purpose:* Focus analysis strictly on the fasting month, removing other phases.

---

## ⏱ **3️⃣ Validate Hourly Windows**

* Keeps only hours with **≥4 CGM readings** to ensure data quality.
* This removes incomplete or sparse hours.

👉 *Purpose:* Guarantee each hourly feature represents stable glucose behavior.

---

## 📊 **4️⃣ Compute Hourly CGM Statistics**

For each patient and hour:

* `cgm_min` → minimum glucose value
* `cgm_max` → maximum glucose value
* `cgm_mean` → mean glucose level
* `cgm_std` → standard deviation (glucose variability)

Also adds:

* `hypo_label` → `1` if any CGM reading in that hour was ≤70 mg/dL.

👉 *Purpose:* Capture both variability and hypoglycemia presence within each hour.

---

## 🧮 **5️⃣ Composite Glucose Features**

Creates two derived indicators:

* `cgm_mean_plus_std`  → average + variability
* `cgm_mean_minus_std` → average – variability

👉 *Purpose:* Encode range boundaries for dynamic glucose variation.

---

## 🧠 **6️⃣ PCA on CGM Variables**

* Runs **Principal Component Analysis (PCA)** on `[cgm_min, cgm_max, cgm_mean, cgm_std]`.
* Extracts **3 principal components** (`pca_cgm1`, `pca_cgm2`, `pca_cgm3`).
* Reports explained variance (usually >95%).

👉 *Purpose:* Compress CGM dynamics into orthogonal, interpretable axes — summarizing glucose pattern, amplitude, and variability.

---

## 🏃‍♀️ **7️⃣ PCA on Lifestyle / Activity / Sleep Features**

* Selects available columns:

  ```
  steps, distance, calories, heart_rate, spo2, deep, light, rem, nap, awake
  ```
* Averages these per hour per patient.
* Runs PCA → extracts **3 lifestyle components**:

  * `pc1_activity_energy` → overall activity/energy output
  * `pc2_physiology` → physiological or heart-rate–related factors
  * `pc3_sleep_rest` → rest and sleep quality indices
* Reports explained variance ratio.

👉 *Purpose:* Reduce multiple wearable signals into interpretable latent factors.

---

## 📑 **8️⃣ Finalize and Sort**

* Orders the dataset by patient and hour.
* Keeps only relevant feature columns:

  ```
  cgm_min, cgm_max, cgm_mean, cgm_std,
  cgm_mean_plus_std, cgm_mean_minus_std,
  pca_cgm1–3, pc1_activity_energy, pc2_physiology, pc3_sleep_rest, hypo_label
  ```
* Prints a preview of the final dataset.

---

## 💾 **9️⃣ Save Hourly Feature File**

Exports the final hourly-level dataset to:

```
/kaggle/working/dynamic_hourly_features_ramadan.csv
```

Each row now represents **one patient-hour** with fully engineered glucose and lifestyle features.

---

## ✅ **Summary in One Line**

> This code aggregates intraday CGM and wearable sensor data into **hourly-level Ramadan features**, computing glucose statistics, detecting hypoglycemia, and summarizing glucose and lifestyle variability using **PCA-derived composite components** — producing a clean, feature-rich dataset for modeling hourly glucose dynamics during fasting.


In [ ]:
import pandas as pd 
import numpy as np
from sklearn.decomposition import PCA

# =========================
# CONFIG
# =========================
CSV_PATH = "/kaggle/input/hmcdataset/intraday_with_visits.csv"  # ✅ update path if needed
OUT_HOURLY_CSV = "/kaggle/working/dynamic_hourly_features_ramadan.csv"

RAMADAN_START = pd.to_datetime("2023-03-22")
RAMADAN_END   = pd.to_datetime("2023-04-19")

# =========================
# STEP 0: Load & prepare data
# =========================
df = pd.read_csv(CSV_PATH)

# Parse timestamps
df["start"] = pd.to_datetime(df["start"], errors="coerce")
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df["hour"] = df["start"].dt.floor("h")
df["hour_of_day"] = df["start"].dt.hour

# Numeric conversion
for col in df.columns:
    if col not in ["patientID", "huaweiID", "visit_assigned", "period_main", "start", "date", "hour", "hour_of_day"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# =========================
# STEP 0.1: Ramadan filter
# =========================
df = df[(df["date"] >= RAMADAN_START) & (df["date"] <= RAMADAN_END)].copy()

# Ensure CGM exists
if "cgm" not in df.columns:
    raise ValueError("❌ Dataset must include 'cgm' column.")
df_cgm = df.dropna(subset=["cgm"]).copy()

# Hypo reading flag (<= 70 mg/dL)
df_cgm["hypo"] = df_cgm["cgm"] <= 70

# =========================
# STEP 1: Filter valid hours (≥4 CGM readings)
# =========================
valid_hours = (
    df_cgm.groupby(["patientID", "hour"])
    .filter(lambda g: g["cgm"].notna().sum() >= 4)
)

# =========================
# STEP 2: Compute hourly CGM statistics
# =========================
hourly_features = (
    valid_hours
    .groupby(["patientID", "hour_of_day", "hour"], as_index=False)
    .agg(
        cgm_min=("cgm", "min"),
        cgm_max=("cgm", "max"),
        cgm_mean=("cgm", "mean"),
        cgm_std=("cgm", "std")
    )
)

# Hypoglycemia label per hour
hypo_per_hour = (
    valid_hours.groupby(["patientID", "hour"])["cgm"]
    .apply(lambda x: (x < 70).any())
    .reset_index(name="hypo_label")
)
hourly_features = hourly_features.merge(hypo_per_hour, on=["patientID", "hour"], how="left")

# =========================
# STEP 3: Composite CGM features
# =========================
hourly_features["cgm_mean_plus_std"] = hourly_features["cgm_mean"] + hourly_features["cgm_std"]
hourly_features["cgm_mean_minus_std"] = hourly_features["cgm_mean"] - hourly_features["cgm_std"]

# =========================
# STEP 4: PCA on CGM stats → 3 components
# =========================
pca_input_cgm = hourly_features[["cgm_min", "cgm_max", "cgm_mean", "cgm_std"]].fillna(0)
pca_cgm = PCA(n_components=3, random_state=42)
cgm_components = pca_cgm.fit_transform(pca_input_cgm)

hourly_features["pca_cgm1"] = cgm_components[:, 0]
hourly_features["pca_cgm2"] = cgm_components[:, 1]
hourly_features["pca_cgm3"] = cgm_components[:, 2]

print("CGM PCA explained variance:", pca_cgm.explained_variance_ratio_.round(3))

# =========================
# STEP 5: PCA on lifestyle/activity/sleep features
# =========================
lifestyle_cols = ["steps", "distance", "calories", "heart_rate", "spo2",
                  "deep", "light", "rem", "nap", "awake"]
lifestyle_cols = [c for c in lifestyle_cols if c in df_cgm.columns]

if lifestyle_cols:
    lifestyle_hourly = (
        df_cgm.groupby(["patientID", "hour"], as_index=False)[lifestyle_cols]
        .mean()
        .fillna(0)
    )

    # Merge lifestyle into hourly_features
    hourly_features = hourly_features.merge(
        lifestyle_hourly, on=["patientID", "hour"], how="left"
    ).fillna(0)

    # Run PCA
    pca_life = PCA(n_components=3, random_state=42)
    life_components = pca_life.fit_transform(hourly_features[lifestyle_cols])

    hourly_features["pc1_activity_energy"] = life_components[:, 0]
    hourly_features["pc2_physiology"] = life_components[:, 1]
    hourly_features["pc3_sleep_rest"] = life_components[:, 2]

    print("Lifestyle PCA explained variance:", pca_life.explained_variance_ratio_.round(3))

# =========================
# STEP 6: Finalize dataset
# =========================
hourly_features = hourly_features.sort_values(["patientID", "hour"]).reset_index(drop=True)

DYNAMIC_FEATURES = [
    "cgm_min", "cgm_max", "cgm_mean", "cgm_std",
    "cgm_mean_plus_std", "cgm_mean_minus_std",
    "pca_cgm1", "pca_cgm2", "pca_cgm3",
    "pc1_activity_energy", "pc2_physiology", "pc3_sleep_rest"
]

print(hourly_features[["patientID", "hour"] + DYNAMIC_FEATURES + ["hypo_label"]].head())


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l1
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf


# Leak-safe All static visist and hourly Ramadan features + Balanced LSTM
(hourly builder + sequences + training utilities)Below is a single, leak‑safe end‑to‑end script that:

Robustly detects patient/date/variable/value columns (handles headers like PatientID (Huawei Data)).

Splits by patient first, then fits PCA & scalers on TRAIN only.

Builds sequences with optional per‑patient static features.

Trains LSTM variants with class‑weighted focal loss and optional resampling.

Chooses decision thresholds on the VALIDATION set (not the test set) to avoid peeking.

Evaluates on test and writes plots + a summary CSV.

What changed vs your last version

Added flexible column pickers (_pick_patient_col, _pick_date_col, …) and used them everywhere (intraday, visit, static).

Thresholds now picked on VAL (Youden and PR‑F1) → no test leakage.

Balanced test creation returns X_stat_bal too (keeps seq+static aligned).

Resampling with SMOTE is skipped automatically when static input is enabled (can’t synthesize static safely).

In [ ]:
# ==============================================
# Leak-safe Ramadan features + Balanced LSTM
# (hourly builder + sequences + training utilities)
# ==============================================
import os, time, warnings, random, re
from pathlib import Path
from collections import Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, roc_curve, precision_recall_curve,
    average_precision_score, auc, mean_squared_error
)

from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN, SMOTETomek

import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout, Concatenate
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

warnings.filterwarnings("ignore")

# --------------------
# GLOBAL CONFIG
# --------------------
# Paths (update for your environment)
CSV_INTRADAY_WITH_VISITS = "/kaggle/input/hmcdataset/intraday_with_visits.csv"
STATIC_CSV      = "/kaggle/input/hmc-model-static-variables/outcome_static.csv"
VISIT_WIDE_CSV  = "/kaggle/input/hmc-model-static-variables/outcome_visit_wide_by_variable.csv"
VISIT_LONG_CSV  = "/kaggle/input/hmc-model-static-variables/outcome_visit_long.csv"

OUT_HOURLY_CSV  = "/kaggle/working/dynamic_hourly_features_ramadan.csv"
OUT_SEQ_NPZ     = "/kaggle/working/sequences_leakfree.npz"
OUT_RESULTS_CSV = "/kaggle/working/results_summary_all.csv"
OUT_PLOTS_PNG   = "/kaggle/working/combined_roc_pr_curves.png"

# Ramadan window and label definition
RAMADAN_START = pd.to_datetime("2023-03-22")
RAMADAN_END   = pd.to_datetime("2023-04-19")
HYPO_CUTOFF   = 70.0   # mg/dL
MIN_CGM_PER_H = 4      # minimum CGM points within an hour to keep that hour
SEQ_LEN       = 24     # sliding window length (hours)

# Lifestyle candidates (if present in intraday_with_visits)
LIFESTYLE_COLS_CANDIDATES = [
    "steps","distance","calories","heart_rate","spo2",
    "deep","light","rem","nap","awake"
]

# "final master" feature lists
VISIT_COLS = ["carb","meals","total_daily_dose_u","fasting_percent_29"]
STATIC_COLS = [
    "Age","Gender","BMI","HbA1C","Cholesterol","LDL","HDL","Triglycerides",
    "eGFR","Creatinine","Insulin_units_per_kg","SmartGuard_percent"
]

# Sequence features used for models (you can edit)
DEFAULT_SEQ_FEATURE_COLS = (
    "cgm_mean","cgm_std","pca_cgm1",      # CGM core + CGM PCA#1
    "pc1_activity_energy",                # lifestyle PCA#1 (0 if lifestyle missing)
    "carb","meals","total_daily_dose_u","fasting_percent_29"  # visit features
)

# Training config
RANDOM_STATE     = 42
THR_MIN, THR_MAX = 0.40, 0.60
AUGMENT_SIGMA    = 0.01   # small Gaussian jitter on train (set None to disable)
RESAMPLE_METHODS = [
    "none",            # baseline (class_weight + focal)
    "oversample_seq",  # duplicate minority sequences
    "undersample_seq", # downsample majority sequences
    # SMOTE-family below only when no static input is used
    "smote", "smoteenn", "smotetomek"
]
USE_STATIC_INPUT = True  # set False to ignore static input entirely (enables SMOTE variants safely)

# --------------------
# Utilities (robust column picking)
# --------------------
def set_global_seeds(seed: int = 42):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
set_global_seeds(RANDOM_STATE)

def to_dt(x, utc_ok=True):
    return pd.to_datetime(x, errors="coerce", utc=utc_ok)

def ensure_numeric(df, exclude=("patientID","huaweiID","visit_assigned","period_main","start","date","hour","hour_of_day")):
    ex = set(exclude)
    for c in df.columns:
        if c not in ex:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

def safe_encode_gender(series):
    if series.dtype == "object":
        return (series.str.strip().str.lower().map({"male":1, "m":1, "female":0, "f":0}))
    return pd.to_numeric(series, errors="coerce")

def split_patients(unique_pids, test_size=0.2, val_size=0.1, random_state=RANDOM_STATE):
    train_pids, test_pids = train_test_split(unique_pids, test_size=test_size, random_state=random_state)
    val_fraction = val_size / max(1e-9, (1.0 - test_size))
    train_pids, val_pids = train_test_split(train_pids, test_size=val_fraction, random_state=random_state)
    return np.array(train_pids), np.array(val_pids), np.array(test_pids)

def _normalize_date(s):
    s = pd.to_datetime(s, errors="coerce")
    return s.dt.normalize()

# ---- robust column pickers ----
def _norm_col(s: str) -> str:
    return re.sub(r'[^a-z0-9]+', '', str(s).lower())

def _pick_col_flex(
    df: pd.DataFrame,
    preferred=None,
    required=False,
    name="",
    must_contain_all=None,
    any_contains=None,
):
    cols = list(df.columns)
    norm_map = {c: _norm_col(c) for c in cols}

    # (1) exact by case-insensitive preferred
    if preferred:
        lower_pref = {str(p).strip().lower(): p for p in preferred}
        for c in cols:
            if str(c).strip().lower() in lower_pref:
                return c

    # (2) exact by normalized preferred
    if preferred:
        pref_norm = {_norm_col(p): p for p in preferred}
        for c, n in norm_map.items():
            if n in pref_norm:
                return c

    # (3) heuristics on normalized names
    cands = []
    for c, n in norm_map.items():
        ok = True
        if must_contain_all:
            for tok in must_contain_all:
                if _norm_col(tok) not in n:
                    ok = False
                    break
        if ok and any_contains:
            if not any(_norm_col(tok) in n for tok in any_contains):
                ok = False
        if ok:
            cands.append(c)
    if cands:
        # prioritize names starting with 'patientid' when looking for patient column
        def _priority(col: str):
            n = norm_map[col]
            starts_pid = n.startswith("patientid")
            has_pid    = "patientid" in n
            return (-(starts_pid or has_pid), len(n))
        cands.sort(key=_priority)
        return cands[0]

    if required:
        raise KeyError(
            f"Required column not found for {name}. "
            f"preferred={preferred} | must_contain_all={must_contain_all} | any_contains={any_contains}. "
            f"Available: {cols}"
        )
    return None

def _pick_patient_col(df: pd.DataFrame) -> str:
    preferred = ["patientID", "patientId", "PatientID (Huawei Data)", "subject_id", "patid", "pid", "id", "huaweiid"]
    return _pick_col_flex(df, preferred=preferred, required=True, name="patientID",
                          must_contain_all=["id"], any_contains=["patient","subject","pat","huawei"])

def _pick_date_col(df: pd.DataFrame) -> str:
    preferred = ["date", "visit_date", "Date", "day", "timestamp", "Visit Date", "date_of_visit", "start"]
    return _pick_col_flex(df, preferred=preferred, required=True, name="date",
                          any_contains=["date","visit","day","timestamp","start"])

def _pick_variable_col(df: pd.DataFrame) -> str:
    preferred = ["variable","var","feature","name","measure","metric"]
    return _pick_col_flex(df, preferred=preferred, required=True, name="variable",
                          any_contains=["variable","var","feature","name","measure","metric"])

def _pick_value_col(df: pd.DataFrame) -> str:
    preferred = ["value","val","measure_value","reading","amount","score"]
    return _pick_col_flex(df, preferred=preferred, required=True, name="value",
                          any_contains=["value","val","measurevalue","reading","amount","score"])

# ---------------------------
# Loaders for external files
# ---------------------------
def load_static_df(static_csv=STATIC_CSV, needed=STATIC_COLS):
    if not static_csv or not os.path.exists(static_csv):
        print("⚠️ Static CSV not found; static features will be zero-filled.")
        return None
    df = pd.read_csv(static_csv)
    pid_col = _pick_patient_col(df)
    df = df.rename(columns={pid_col:"patientID"})
    keep = ["patientID"] + [c for c in needed if c in df.columns]
    df = df[keep].drop_duplicates(subset=["patientID"]).copy()
    if "Gender" in df.columns:
        df["Gender"] = safe_encode_gender(df["Gender"])
    for c in keep:
        if c != "patientID":
            df[c] = pd.to_numeric(df[c], errors="coerce")
    print(f"ℹ️ static: using patientID column = '{pid_col}'")
    return df

def load_visit_df(visit_wide_csv=VISIT_WIDE_CSV, visit_long_csv=VISIT_LONG_CSV, needed=VISIT_COLS):
    # Try wide first
    if visit_wide_csv and os.path.exists(visit_wide_csv):
        wide = pd.read_csv(visit_wide_csv)
        pid_col  = _pick_patient_col(wide)
        date_col = _pick_date_col(wide)
        wide = wide.rename(columns={pid_col:"patientID", date_col:"date"})
        wide["date"] = _normalize_date(wide["date"])
        keep = ["patientID","date"] + [c for c in needed if c in wide.columns]
        if len(keep) > 2:
            print(f"ℹ️ visit-wide: patientID='{pid_col}', date='{date_col}', kept={keep[2:]}")
            return wide[keep].copy()
        else:
            print("⚠️ VISIT_WIDE_CSV found but none of the needed visit columns present; will try LONG if available.")

    # Fallback: long -> pivot
    if visit_long_csv and os.path.exists(visit_long_csv):
        long = pd.read_csv(visit_long_csv)
        pid_col   = _pick_patient_col(long)
        date_col  = _pick_date_col(long)
        var_col   = _pick_variable_col(long)
        value_col = _pick_value_col(long)
        long = long.rename(columns={pid_col:"patientID", date_col:"date", var_col:"variable", value_col:"value"})
        long["date"] = _normalize_date(long["date"])
        wide = (long
                .pivot_table(index=["patientID","date"], columns="variable", values="value", aggfunc="mean")
                .reset_index())
        keep = ["patientID","date"] + [c for c in needed if c in wide.columns]
        if len(keep) > 2:
            print(f"ℹ️ visit-long: patientID='{pid_col}', date='{date_col}', variables matched={keep[2:]}")
            return wide[keep].copy()
        print("⚠️ VISIT_LONG_CSV present but none of the needed variables were found in the pivot.")

    print("⚠️ No usable visit CSVs found; visit features will be zero-filled.")
    return None

# ----------------------------------------------------------------
# Part A — Build hourly Ramadan features and leak‑safe transforms
# ----------------------------------------------------------------
def build_hourly_features_with_leak_safe_transforms(
    in_csv=CSV_INTRADAY_WITH_VISITS,
    out_csv=OUT_HOURLY_CSV,
    min_cgm_per_hour=MIN_CGM_PER_H,
    test_size=0.2, val_size=0.1, random_state=RANDOM_STATE,
    static_csv=STATIC_CSV, visit_wide_csv=VISIT_WIDE_CSV, visit_long_csv=VISIT_LONG_CSV
):
    if not os.path.exists(in_csv):
        raise FileNotFoundError(f"Input not found: {in_csv}")

    # Load & parse intraday
    df = pd.read_csv(in_csv)

    # Robust patient column for intraday too
    if "patientID" not in df.columns:
        pid_col = _pick_patient_col(df)
        df = df.rename(columns={pid_col: "patientID"})
        print(f"ℹ️ intraday: using patientID column = '{pid_col}'")

    # timestamps
    start_col = "start" if "start" in df.columns else _pick_date_col(df)
    df[start_col] = to_dt(df[start_col])
    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"], errors="coerce")
    else:
        df["date"] = pd.to_datetime(df[start_col].dt.date)
    df["hour"]       = df[start_col].dt.floor("h")
    df["hour_of_day"]= df["hour"].dt.hour

    df = ensure_numeric(df)

    # Ramadan filter
    df = df[(df["date"] >= RAMADAN_START) & (df["date"] <= RAMADAN_END)].copy()

    # Require CGM
    if "cgm" not in df.columns:
        raise ValueError("❌ Dataset must include 'cgm' column.")
    df_cgm = df.dropna(subset=["cgm"]).copy()

    # Valid hourly windows
    valid_hours = (
        df_cgm.groupby(["patientID","hour"])
              .filter(lambda g: g["cgm"].notna().sum() >= min_cgm_per_hour)
    )

    # Base hourly CGM stats
    hourly = (
        valid_hours.groupby(["patientID","hour"], as_index=False)
                   .agg(
                       cgm_min=("cgm","min"),
                       cgm_max=("cgm","max"),
                       cgm_mean=("cgm","mean"),
                       cgm_std=("cgm","std")
                   )
                   .sort_values(["patientID","hour"])
                   .reset_index(drop=True)
    )
    hourly["hour_of_day"] = hourly["hour"].dt.hour

    # Hypo label
    lab = (
        valid_hours.groupby(["patientID","hour"])["cgm"]
                   .apply(lambda x: int((x < HYPO_CUTOFF).any()))
                   .reset_index(name="hypo_label")
    )
    hourly = hourly.merge(lab, on=["patientID","hour"], how="left")

    # Lifestyle hourly means (if present)
    lifestyle_cols = [c for c in LIFESTYLE_COLS_CANDIDATES if c in df_cgm.columns]
    if lifestyle_cols:
        life_hourly = (
            df_cgm.groupby(["patientID","hour"], as_index=False)[lifestyle_cols]
                  .mean().fillna(0.0)
        )
        hourly = hourly.merge(life_hourly, on=["patientID","hour"], how="left")

    # Composite CGM features
    hourly["cgm_mean_plus_std"]  = hourly["cgm_mean"] + hourly["cgm_std"]
    hourly["cgm_mean_minus_std"] = hourly["cgm_mean"] - hourly["cgm_std"]

    # Patient-level split (NO LEAK)
    pids = hourly["patientID"].dropna().unique()
    train_p, val_p, test_p = split_patients(pids, test_size=test_size, val_size=val_size, random_state=random_state)
    hourly["Split"] = np.where(hourly["patientID"].isin(train_p), "train",
                        np.where(hourly["patientID"].isin(val_p), "val", "test"))

    # CGM PCA fit on TRAIN only
    cgm_cols = ["cgm_min","cgm_max","cgm_mean","cgm_std"]
    tr_mask  = hourly["Split"] == "train"
    scal_cgm = StandardScaler().fit(hourly.loc[tr_mask, cgm_cols].fillna(0.0))
    pca_cgm  = PCA(n_components=3, random_state=random_state).fit(
        scal_cgm.transform(hourly.loc[tr_mask, cgm_cols].fillna(0.0))
    )
    def _apply_cgm_pca(df_in):
        X = scal_cgm.transform(df_in[cgm_cols].fillna(0.0))
        Z = pca_cgm.transform(X)
        out = df_in.copy()
        out["pca_cgm1"], out["pca_cgm2"], out["pca_cgm3"] = Z[:,0], Z[:,1], Z[:,2]
        return out
    hourly = _apply_cgm_pca(hourly)

    # Lifestyle PCA fit on TRAIN only
    if lifestyle_cols:
        scal_life = StandardScaler().fit(hourly.loc[tr_mask, lifestyle_cols].fillna(0.0))
        pca_life  = PCA(n_components=3, random_state=random_state).fit(
            scal_life.transform(hourly.loc[tr_mask, lifestyle_cols].fillna(0.0))
        )
        X_all = scal_life.transform(hourly[lifestyle_cols].fillna(0.0))
        Z_all = pca_life.transform(X_all)
        hourly["pc1_activity_energy"] = Z_all[:,0]
        hourly["pc2_physiology"]      = Z_all[:,1]
        hourly["pc3_sleep_rest"]      = Z_all[:,2]
    else:
        hourly["pc1_activity_energy"] = 0.0
        hourly["pc2_physiology"]      = 0.0
        hourly["pc3_sleep_rest"]      = 0.0

    # Merge VISIT features (daily)
    visit_df = load_visit_df(visit_wide_csv, visit_long_csv, VISIT_COLS)
    hourly["date"] = hourly["hour"].dt.normalize()
    if visit_df is not None:
        visit_df["date"] = pd.to_datetime(visit_df["date"], errors="coerce").dt.normalize()
        visit_df = visit_df[(visit_df["date"] >= RAMADAN_START) & (visit_df["date"] <= RAMADAN_END)].copy()
        hourly = hourly.merge(visit_df, on=["patientID","date"], how="left")
    for c in VISIT_COLS:
        if c not in hourly.columns:
            hourly[c] = 0.0
        hourly[c] = pd.to_numeric(hourly[c], errors="coerce").fillna(0.0)

    # Merge STATIC features (per patient)
    static_df = load_static_df(static_csv, STATIC_COLS)
    if static_df is not None:
        hourly = hourly.merge(static_df, on="patientID", how="left")
    for c in STATIC_COLS:
        if c not in hourly.columns:
            hourly[c] = 0.0
        hourly[c] = pd.to_numeric(hourly[c], errors="coerce").fillna(0.0)

    # Save hourly table
    hourly = hourly.sort_values(["patientID","hour"]).reset_index(drop=True)
    hourly.to_csv(out_csv, index=False)
    print(f"✅ Saved hourly features (leak-safe) → {out_csv}")

    return hourly, (train_p, val_p, test_p)

# ---------------------------------------------------------------
# Part B — Build sequences (optionally with per-patient static)
# ---------------------------------------------------------------
def build_sequences_by_split(
    hourly, splits, seq_len=SEQ_LEN,
    seq_feature_cols=DEFAULT_SEQ_FEATURE_COLS,
    static_cols=STATIC_COLS,
    scale_features=True
):
    for c in ["patientID","hour","hypo_label","Split"]:
        if c not in hourly.columns:
            raise KeyError(f"hourly missing required column: {c}")
    hourly["hour"] = pd.to_datetime(hourly["hour"], errors="coerce")

    seq_feature_cols = list(seq_feature_cols)
    missing_seq = [c for c in seq_feature_cols if c not in hourly.columns]
    if missing_seq:
        raise KeyError(f"Sequence feature(s) not found in hourly: {missing_seq}")

    static_cols_present = [c for c in static_cols if c in hourly.columns]
    if static_cols_present and USE_STATIC_INPUT:
        static_mat = (hourly[["patientID"] + static_cols_present]
                      .drop_duplicates(subset=["patientID"])
                      .set_index("patientID")
                      .astype(float).fillna(0.0))
    else:
        static_mat = None
        static_cols_present = []

    train_p, val_p, test_p = splits

    def _build_for_pidset(pid_set):
        sub = hourly[hourly["patientID"].isin(pid_set)].copy()
        X_seq, X_stat, y = [], [], []
        for pid, grp in sub.groupby("patientID"):
            grp = grp.sort_values("hour").reset_index(drop=True)
            if len(grp) <= seq_len:
                continue
            feats  = grp[seq_feature_cols].astype(float).values
            labels = grp["hypo_label"].astype(int).values
            for i in range(len(grp) - seq_len):
                X_seq.append(feats[i:i+seq_len]); y.append(labels[i+seq_len])
                if static_mat is not None and pid in static_mat.index:
                    X_stat.append(static_mat.loc[pid].values.astype(float))
        X_seq = np.array(X_seq); y = np.array(y).astype(int)
        X_stat = np.array(X_stat) if (static_mat is not None and len(X_stat)>0) else None
        return X_seq, X_stat, y

    Xtr_s, Xtr_stat, ytr = _build_for_pidset(train_p)
    Xva_s, Xva_stat, yva = _build_for_pidset(val_p)
    Xte_s, Xte_stat, yte = _build_for_pidset(test_p)

    # Scale sequence features (fit on TRAIN only), and static (fit on TRAIN only)
    seq_scaler  = None
    stat_scaler = None
    if scale_features and Xtr_s.size > 0:
        n_f = Xtr_s.shape[2]
        seq_scaler = StandardScaler().fit(Xtr_s.reshape(-1, n_f))
        def _scale_seq(X):
            if X is None or X.size==0: return X
            n = X.shape[0]; return seq_scaler.transform(X.reshape(-1, n_f)).reshape(n, SEQ_LEN, n_f)
        Xtr_s = _scale_seq(Xtr_s); Xva_s = _scale_seq(Xva_s); Xte_s = _scale_seq(Xte_s)

    if scale_features and Xtr_stat is not None and Xtr_stat.size>0:
        stat_scaler = StandardScaler().fit(Xtr_stat)
        def _scale_stat(X):
            if X is None or X.size==0: return X
            return stat_scaler.transform(X)
        Xtr_stat = _scale_stat(Xtr_stat); Xva_stat = _scale_stat(Xva_stat); Xte_stat = _scale_stat(Xte_stat)

    print(f"✅ Sequences built | train={Xtr_s.shape}, val={Xva_s.shape}, test={Xte_s.shape}")
    return {
        "train": {"X_seq": Xtr_s, "X_stat": Xtr_stat, "y": ytr},
        "val":   {"X_seq": Xva_s, "X_stat": Xva_stat, "y": yva},
        "test":  {"X_seq": Xte_s, "X_stat": Xte_stat, "y": yte},
        "seq_features_used": seq_feature_cols,
        "static_features_used": static_cols_present,
        "scalers": {"seq": seq_scaler, "stat": stat_scaler}
    }

# ------------------------------------------------------
# Balanced LSTM pipeline utilities (metrics, resampling)
# ------------------------------------------------------
THR_MIN, THR_MAX = THR_MIN, THR_MAX  # keep constants visible here

def _best_threshold_in_range(thresholds, scores, thr_min=THR_MIN, thr_max=THR_MAX):
    thresholds = np.asarray(thresholds, dtype=float)
    scores     = np.asarray(scores, dtype=float)
    mask = np.isfinite(thresholds) & (thresholds >= thr_min) & (thresholds <= thr_max)
    if mask.any():
        idx_in = int(np.nanargmax(scores[mask])); idx = np.where(mask)[0][idx_in]
        return float(thresholds[idx]), True
    idx = int(np.nanargmax(scores))
    return float(np.clip(thresholds[idx], thr_min, thr_max)), False

def focal_loss(gamma=2.0, alpha=0.25):
    bce = tf.keras.losses.BinaryCrossentropy(from_logits=False, reduction=tf.keras.losses.Reduction.NONE)
    eps = tf.keras.backend.epsilon()
    def loss(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, eps, 1.0 - eps)
        ce = bce(y_true, y_pred)
        p_t = y_true * y_pred + (1.0 - y_true) * (1.0 - y_pred)
        alpha_t = y_true * alpha + (1.0 - y_true) * (1.0 - alpha)
        modulating = tf.pow(1.0 - p_t, gamma)
        return alpha_t * modulating * ce
    return loss

def _safe_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred, labels=[0,1])
    if cm.shape != (2,2):
        full = np.zeros((2,2), dtype=int)
        full[:cm.shape[0], :cm.shape[1]] = cm
        cm = full
    return cm

def _specificity_overall(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    return tn / (tn + fp + 1e-8)

def _specificity_per_class(y_true, y_pred, positive_label):
    y_true_bin = (np.asarray(y_true).ravel() == positive_label).astype(int)
    y_pred_bin = (np.asarray(y_pred).ravel() == positive_label).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true_bin, y_pred_bin, labels=[0,1]).ravel()
    return tn / (tn + fp + 1e-8)

def evaluate_full_metrics(y_true, y_pred, y_prob=None):
    y_true = np.asarray(y_true).astype(int).ravel()
    y_pred = np.asarray(y_pred).astype(int).ravel()
    cm = _safe_confusion_matrix(y_true, y_pred)

    metrics = {}
    for lbl in [0,1]:
        metrics[f"Class{lbl}/Precision"]   = precision_score(y_true, y_pred, pos_label=lbl, zero_division=0)
        metrics[f"Class{lbl}/Recall"]      = recall_score(y_true, y_pred,    pos_label=lbl, zero_division=0)
        metrics[f"Class{lbl}/F1"]          = f1_score(y_true, y_pred,        pos_label=lbl, zero_division=0)
        metrics[f"Class{lbl}/Specificity"] = _specificity_per_class(y_true, y_pred, positive_label=lbl)
        metrics[f"Class{lbl}/Support"]     = int(np.sum(y_true == lbl))

    metrics["Overall/Accuracy"]             = accuracy_score(y_true, y_pred)
    metrics["Overall/Precision_macro"]      = precision_score(y_true, y_pred, average='macro',    zero_division=0)
    metrics["Overall/Precision_weighted"]   = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    metrics["Overall/Recall_macro"]         = recall_score(y_true, y_pred,    average='macro',    zero_division=0)
    metrics["Overall/Recall_weighted"]      = recall_score(y_true, y_pred,    average='weighted', zero_division=0)
    metrics["Overall/F1_macro"]             = f1_score(y_true, y_pred,        average='macro',    zero_division=0)
    metrics["Overall/F1_weighted"]          = f1_score(y_true, y_pred,        average='weighted', zero_division=0)
    metrics["Overall/Specificity"]          = _specificity_overall(y_true, y_pred)
    mse_pred                                = mean_squared_error(y_true, y_pred)
    metrics["Overall/MSE_pred"]             = mse_pred
    metrics["Overall/RMSE_pred"]            = float(np.sqrt(mse_pred))

    if y_prob is not None:
        y_prob = np.asarray(y_prob, dtype=float).ravel()
        try:  metrics["Overall/ROC-AUC"] = roc_auc_score(y_true, y_prob)
        except ValueError: metrics["Overall/ROC-AUC"] = np.nan
        try:  metrics["Overall/PR-AUC"]  = average_precision_score(y_true, y_prob)
        except ValueError: metrics["Overall/PR-AUC"] = np.nan
        mse_prob                          = mean_squared_error(y_true, y_prob)
        metrics["Overall/MSE_prob"]       = mse_prob
        metrics["Overall/RMSE_prob"]      = float(np.sqrt(mse_prob))
    else:
        metrics["Overall/ROC-AUC"]  = np.nan
        metrics["Overall/PR-AUC"]   = np.nan
        metrics["Overall/MSE_prob"] = np.nan
        metrics["Overall/RMSE_prob"]= np.nan
    return metrics

def make_class_weight(y):
    y  = np.asarray(y).astype(int).ravel()
    n0 = max(1, (y==0).sum()); n1 = max(1, (y==1).sum()); N = n0+n1
    w0 = N/(2.0*n0); w1 = N/(2.0*n1)
    return {0: float(w0), 1: float(w1)}

def augment_with_static(X_seq, X_stat, y, sigma=AUGMENT_SIGMA):
    if sigma is None or sigma <= 0:
        return X_seq, X_stat, y
    noise = np.random.normal(0, sigma, X_seq.shape)
    X_seq_aug = np.vstack([X_seq, X_seq + noise])
    y_aug     = np.hstack([y, y])
    if X_stat is not None:
        X_stat_aug = np.vstack([X_stat, X_stat])
    else:
        X_stat_aug = None
    return X_seq_aug, X_stat_aug, y_aug

def seq_resample(X, y, method="none", random_state=RANDOM_STATE, return_index=False, allow_smote=True):
    """
    Sequence-level resampling. If return_index=True, also returns the index mapping used so
    static inputs can be resampled consistently. For SMOTE-family, index mapping isn't
    meaningful; we disable SMOTE if allow_smote=False.
    """
    X = np.asarray(X); y = np.asarray(y).astype(int).ravel()
    n, T, F = X.shape
    base_idx = np.arange(n)

    if method == "none":
        return (X, y, base_idx) if return_index else (X, y)

    if method in {"oversample_seq","undersample_seq"}:
        rng = np.random.default_rng(random_state)
        idx0 = np.where(y==0)[0]; idx1 = np.where(y==1)[0]
        n0, n1 = len(idx0), len(idx1)
        if n0==0 or n1==0:
            return (X, y, base_idx) if return_index else (X, y)

        if method == "oversample_seq":
            if n1 < n0:
                add = rng.choice(idx1, size=n0-n1, replace=True)
                keep = np.concatenate([idx0, idx1, add])
            else:
                add = rng.choice(idx0, size=n1-n0, replace=True)
                keep = np.concatenate([idx0, idx1, add])
        else:  # undersample
            if n0 > n1:
                keep0 = rng.choice(idx0, size=n1, replace=False)
                keep  = np.concatenate([keep0, idx1])
            else:
                keep1 = rng.choice(idx1, size=n0, replace=False)
                keep  = np.concatenate([idx0, keep1])

        rng.shuffle(keep)
        Xr, yr = X[keep], y[keep]
        return (Xr, yr, keep) if return_index else (Xr, yr)

    # SMOTE family
    if not allow_smote:
        print(f"⚠️ {method} disabled when static input is used; falling back to 'none'.")
        return (X, y, base_idx) if return_index else (X, y)

    minority_n = int((y==1).sum())
    majority_n = int((y==0).sum())
    if minority_n < 2 or majority_n < 2:
        print("⚠️ Not enough samples for SMOTE/SMOTEENN/SMOTETomek; skipping resampling.")
        return (X, y, base_idx) if return_index else (X, y)

    Xf = X.reshape(n, -1)
    if method == "smote":
        k_neighbors = max(1, min(5, minority_n-1))
        sm = SMOTE(random_state=random_state, k_neighbors=k_neighbors)
        Xr, yr = sm.fit_resample(Xf, y)
    elif method == "smoteenn":
        Xr, yr = SMOTEENN(random_state=random_state).fit_resample(Xf, y)
    elif method == "smotetomek":
        Xr, yr = SMOTETomek(random_state=random_state).fit_resample(Xf, y)
    else:
        raise ValueError(f"Unknown resampling method: {method}")

    Xr = Xr.reshape(-1, T, F)
    return (Xr, yr, None) if return_index else (Xr, yr)

def make_balanced_test(X_test, y_test, X_stat=None, random_state=RANDOM_STATE):
    """Return balanced (by label) subsets of X_test, y_test, and X_stat (if given)."""
    X_test = np.asarray(X_test)
    y_test = np.asarray(y_test).astype(int).ravel()
    idx0, idx1 = np.where(y_test==0)[0], np.where(y_test==1)[0]
    if len(idx0)==0 or len(idx1)==0: 
        return (X_test, y_test, (None if X_stat is None else X_stat))
    m = min(len(idx0), len(idx1))
    rs = np.random.RandomState(random_state)
    keep = np.concatenate([rs.choice(idx0, m, replace=False), rs.choice(idx1, m, replace=False)])
    rs.shuffle(keep)
    Xb, yb = X_test[keep], y_test[keep]
    Xsb = (None if X_stat is None else np.asarray(X_stat)[keep])
    return Xb, yb, Xsb

# ------------------------------------------------------
# Model builders (supports seq-only or seq+static)
# ------------------------------------------------------
def make_model(seq_len, n_seq_f, n_stat_f=0, arch="LSTM_100", lr=1e-3):
    seq_in = Input(shape=(seq_len, n_seq_f), name="seq_in")
    x = seq_in
    if arch == "BiLSTM":
        x = Bidirectional(LSTM(64, return_sequences=True))(x)
        x = Dropout(0.2)(x)
        x = Bidirectional(LSTM(32))(x)
        x = Dropout(0.2)(x)
        x = Dense(16, activation="relu")(x)
    elif arch == "LSTM_50":
        x = LSTM(50, return_sequences=True)(x); x = Dropout(0.2)(x)
        x = LSTM(25)(x);                    x = Dropout(0.2)(x)
        x = Dense(10, activation="relu")(x)
    elif arch == "LSTM_25_L1":
        x = LSTM(50, return_sequences=True, kernel_regularizer=l1(1e-5))(x); x = Dropout(0.2)(x)
        x = LSTM(25, kernel_regularizer=l1(1e-5))(x);                        x = Dropout(0.2)(x)
        x = Dense(10, activation="relu", kernel_regularizer=l1(1e-5))(x)
    elif arch == "LSTM_25_L2":
        x = LSTM(50, return_sequences=True, kernel_regularizer=l2(1e-5))(x); x = Dropout(0.2)(x)
        x = LSTM(25, kernel_regularizer=l2(1e-5))(x);                        x = Dropout(0.2)(x)
        x = Dense(10, activation="relu", kernel_regularizer=l2(1e-5))(x)
    else:  # LSTM_100
        x = LSTM(100, return_sequences=True)(x); x = Dropout(0.2)(x)
        x = LSTM(50)(x);                          x = Dropout(0.2)(x)
        x = Dense(25, activation="relu")(x)

    if n_stat_f and n_stat_f > 0 and USE_STATIC_INPUT:
        stat_in = Input(shape=(n_stat_f,), name="stat_in")
        s = Dense(32, activation="relu")(stat_in)
        s = Dropout(0.2)(s)
        h = Concatenate()([x, s])
        h = Dense(32, activation="relu")(h)
        out = Dense(1, activation="sigmoid")(h)
        model = Model(inputs=[seq_in, stat_in], outputs=out)
    else:
        h = Dense(32, activation="relu")(x)
        out = Dense(1, activation="sigmoid")(h)
        model = Model(inputs=seq_in, outputs=out)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  loss=focal_loss(), metrics=["accuracy"])
    return model

# ------------------------------------------------------
# Training runner (VAL for threshold; TEST for final)
# ------------------------------------------------------
def run_balanced_lstm_pipeline(data,
                               arch_list=("LSTM_100","BiLSTM","LSTM_50"),
                               resample_methods=RESAMPLE_METHODS,
                               thr_min=THR_MIN, thr_max=THR_MAX,
                               random_state=RANDOM_STATE,
                               results_csv=OUT_RESULTS_CSV,
                               plots_png=OUT_PLOTS_PNG):
    os.makedirs(os.path.dirname(results_csv), exist_ok=True)
    os.makedirs(os.path.dirname(plots_png), exist_ok=True)
    os.makedirs("checkpoints", exist_ok=True)

    Xtr, Xtr_stat, ytr = data["train"]["X_seq"], data["train"]["X_stat"], data["train"]["y"]
    Xva, Xva_stat, yva = data["val"]["X_seq"],   data["val"]["X_stat"],   data["val"]["y"]
    Xte, Xte_stat, yte = data["test"]["X_seq"],  data["test"]["X_stat"],  data["test"]["y"]

    # Augment train (and static if present)
    Xtr, Xtr_stat, ytr = augment_with_static(Xtr, Xtr_stat, ytr, sigma=AUGMENT_SIGMA)

    # Balanced copy of test for diagnostic plots
    Xte_bal, yte_bal, Xte_stat_bal = make_balanced_test(Xte, yte, X_stat=Xte_stat)

    results     = {}
    roc_curves  = {}
    pr_curves   = {}

    allow_smote = (Xtr_stat is None or not USE_STATIC_INPUT)

    def train_eval_one(method_name, arch_name):
        nonlocal Xtr, ytr, Xtr_stat

        # Resample TRAIN only
        Xrs, yrs, idx_map = seq_resample(Xtr, ytr, method=method_name, random_state=random_state,
                                         return_index=True, allow_smote=allow_smote)
        if Xtr_stat is not None and USE_STATIC_INPUT:
            if idx_map is None:
                # SMOTE chosen while static present → already disabled in seq_resample
                Xrs_stat = Xtr_stat
            else:
                Xrs_stat = Xtr_stat[idx_map]
        else:
            Xrs_stat = None

        # Build model
        seq_len, n_seq_f = Xrs.shape[1], Xrs.shape[2]
        n_stat_f = 0 if (Xrs_stat is None or not USE_STATIC_INPUT) else Xrs_stat.shape[1]
        model = make_model(seq_len, n_seq_f, n_stat_f=n_stat_f, arch=arch_name, lr=1e-3)

        # Fit with VAL for early stopping (no peeking at test)
        es = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True, verbose=1)
        ckpt_path = f"checkpoints/{method_name}__{arch_name}.h5"
        cp = ModelCheckpoint(ckpt_path, save_best_only=True, monitor="val_loss", verbose=0)

        if n_stat_f > 0 and USE_STATIC_INPUT:
            model.fit([Xrs, Xrs_stat], yrs,
                      validation_data=([Xva, Xva_stat], yva),
                      epochs=12, batch_size=64, callbacks=[es, cp],
                      class_weight=make_class_weight(yrs), verbose=1)
            p_tr  = model.predict([Xrs, Xrs_stat], verbose=0).ravel()
            p_va  = model.predict([Xva, Xva_stat], verbose=0).ravel()
            p_te  = model.predict([Xte, Xte_stat], verbose=0).ravel()
            p_teB = model.predict([Xte_bal, Xte_stat_bal], verbose=0).ravel() if Xte_stat_bal is not None else model.predict(Xte_bal, verbose=0).ravel()
        else:
            model.fit(Xrs, yrs,
                      validation_data=(Xva, yva),
                      epochs=12, batch_size=64, callbacks=[es, cp],
                      class_weight=make_class_weight(yrs), verbose=1)
            p_tr  = model.predict(Xrs, verbose=0).ravel()
            p_va  = model.predict(Xva, verbose=0).ravel()
            p_te  = model.predict(Xte, verbose=0).ravel()
            p_teB = model.predict(Xte_bal, verbose=0).ravel()

        # ---------- choose thresholds on VALIDATION (not TEST) ----------
        try:
            fpr_va, tpr_va, thr_roc_va = roc_curve(yva, p_va); auc_roc = auc(fpr_va, tpr_va)
        except ValueError:
            fpr_va, tpr_va, thr_roc_va, auc_roc = np.array([0,1]), np.array([0,1]), np.array([0.5]), np.nan
        youden_va = tpr_va - fpr_va
        t_roc, _ = _best_threshold_in_range(thr_roc_va, youden_va, thr_min, thr_max)

        prec_va, rec_va, thr_pr_va = precision_recall_curve(yva, p_va)
        f1s_va = 2*prec_va[:-1]*rec_va[:-1] / (prec_va[:-1]+rec_va[:-1]+1e-8)
        t_pr, _ = _best_threshold_in_range(thr_pr_va, f1s_va, thr_min, thr_max)
        ap_val  = average_precision_score(yva, p_va)

        # Curves (validation-based AUC/AP shown)
        roc_curves[(method_name, arch_name)] = (fpr_va, tpr_va, auc_roc)
        pr_curves[(method_name, arch_name)]  = (rec_va, prec_va, ap_val)
        print(f"📌 [{method_name}/{arch_name}] VAL thresholds → Youden={t_roc:.4f}, PR-F1={t_pr:.4f} (window [{thr_min},{thr_max}])")

        eval_ts = sorted(set([thr_min, 0.50, thr_max, float(t_roc), float(t_pr)]))

        # Evaluate at thresholds: TRAIN / VAL / TEST / TEST (balanced)
        for t in eval_ts:
            yhat_tr  = (p_tr  >= t).astype(int)
            yhat_va  = (p_va  >= t).astype(int)
            yhat_te  = (p_te  >= t).astype(int)
            yhat_teB = (p_teB >= t).astype(int)

            results[f"{method_name}__{arch_name}__thr_{t:.2f}__train"]        = evaluate_full_metrics(yrs,  yhat_tr,  p_tr)
            results[f"{method_name}__{arch_name}__thr_{t:.2f}__val"]          = evaluate_full_metrics(yva,  yhat_va,  p_va)
            results[f"{method_name}__{arch_name}__thr_{t:.2f}__test"]         = evaluate_full_metrics(yte,  yhat_te,  p_te)
            results[f"{method_name}__{arch_name}__thr_{t:.2f}__testBalanced"] = evaluate_full_metrics(yte_bal, yhat_teB, p_teB)

    # Loop: resampling methods × architectures
    for METHOD in resample_methods:
        if METHOD in {"smote","smoteenn","smotetomek"} and (data["train"]["X_stat"] is not None and USE_STATIC_INPUT):
            print(f"⏭️  Skipping {METHOD} (static input enabled).")
            continue
        print(f"\n🔁 Resampling: {METHOD} | train y-dist = {Counter(data['train']['y'])}")
        for ARCH in arch_list:
            train_eval_one(METHOD, ARCH)

    # --- Plots (validation curves)
    plt.figure(figsize=(14,6))
    # ROC
    plt.subplot(1,2,1)
    for (meth, arch), (fpr, tpr, auc_roc) in roc_curves.items():
        plt.plot(fpr, tpr, label=f'{meth}/{arch} (VAL AUC={auc_roc:.3f})')
    plt.plot([0,1],[0,1],'--',label='Random')
    plt.xlabel('FPR'); plt.ylabel('TPR'); plt.title('ROC (Validation)'); plt.legend(fontsize=8)
    # PR
    plt.subplot(1,2,2)
    for (meth, arch), (rec, prec, ap) in pr_curves.items():
        plt.plot(rec, prec, label=f'{meth}/{arch} (VAL AP={ap:.3f})')
    plt.xlabel('Recall'); plt.ylabel('Precision'); plt.title('PR (Validation)'); plt.legend(fontsize=8)
    plt.tight_layout(); plt.savefig(plots_png, dpi=300); plt.show()
    print(f"🖼️ Saved plots → {plots_png}")

    # --- Results CSV
    results_df = pd.DataFrame(results).T.reset_index().rename(columns={"index":"Key"})
    k = results_df["Key"].str.strip()
    results_df["Split"]  = np.where(k.str.endswith("__train"), "train",
                             np.where(k.str.endswith("__val"), "val",
                             np.where(k.str.endswith("__testBalanced"), "testBalanced",
                             np.where(k.str.endswith("__test"), "test", np.nan))))
    parts = k.str.split("__")
    results_df["Method"]    = parts.str[0]
    results_df["Model"]     = parts.str[1]
    results_df["Threshold"] = pd.to_numeric(parts.str[2].str.replace("thr_","", regex=False), errors="coerce")
    results_df.round(6).to_csv(results_csv, index=False)
    print(f"📑 Saved results → {results_csv}")

    return results_df

# --------------------
# Run end-to-end
# --------------------
if __name__ == "__main__":
    # A) Hourly features with leak‑safe PCA & merges
    hourly, splits = build_hourly_features_with_leak_safe_transforms(
        in_csv=CSV_INTRADAY_WITH_VISITS,
        out_csv=OUT_HOURLY_CSV,
        min_cgm_per_hour=MIN_CGM_PER_H,
        test_size=0.2, val_size=0.1, random_state=RANDOM_STATE,
        static_csv=STATIC_CSV, visit_wide_csv=VISIT_WIDE_CSV, visit_long_csv=VISIT_LONG_CSV
    )

    # B) Sequences with visit + static (no leakage; scalers fit on TRAIN only)
    data = build_sequences_by_split(
        hourly, splits,
        seq_len=SEQ_LEN,
        seq_feature_cols=DEFAULT_SEQ_FEATURE_COLS,
        static_cols=STATIC_COLS,
        scale_features=True
    )

    # (Optional) Save arrays for later reuse
    np.savez_compressed(
        OUT_SEQ_NPZ,
        Xtr=data["train"]["X_seq"],  Xtr_stat=(data["train"]["X_stat"] if data["train"]["X_stat"] is not None else np.empty((0,0))),
        ytr=data["train"]["y"],
        Xva=data["val"]["X_seq"],    Xva_stat=(data["val"]["X_stat"] if data["val"]["X_stat"] is not None else np.empty((0,0))),
        yva=data["val"]["y"],
        Xte=data["test"]["X_seq"],   Xte_stat=(data["test"]["X_stat"] if data["test"]["X_stat"] is not None else np.empty((0,0))),
        yte=data["test"]["y"],
        seq_features_used=np.array(data["seq_features_used"], dtype=object),
        static_features_used=np.array(data["static_features_used"], dtype=object)
    )
    print(f"💾 Saved sequences → {OUT_SEQ_NPZ}")

    # C) Train + evaluate (leak-safe thresholds chosen on VAL)
    results_df = run_balanced_lstm_pipeline(
        data,
        arch_list=("LSTM_100","BiLSTM","LSTM_50"),
        resample_methods=RESAMPLE_METHODS,
        thr_min=THR_MIN, thr_max=THR_MAX,
        random_state=RANDOM_STATE,
        results_csv=OUT_RESULTS_CSV,
        plots_png=OUT_PLOTS_PNG
    )


Here’s a concise read‑out of the results you uploaded in **`/mnt/data/results_summary_all.csv`** and what they mean in practice.

---

## What I analyzed

From your results file I looked at, per split:

* **Resampling method**: `none`, `oversample_seq`, `undersample_seq`
* **Model**: `LSTM_100`, `BiLSTM`, `LSTM_50`
* **Thresholds** evaluated: 0.40, 0.50, 0.60 (and the full probability curves to compute ROC‑AUC/PR‑AUC)
* **Metrics**: Overall Accuracy/F1 (macro & weighted), per‑class Precision/Recall/F1/Specificity, plus ROC‑AUC and PR‑AUC

Splits present in the file: **train**, **test**, and a **testBalanced** (your balanced diagnostic set with 1:1 class ratio).

---

##  Best practical setting

If the goal is **reliably catching hypoglycemia (Class 1)** with strong precision:

* **Resampling**: `oversample_seq`
* **Model**: `LSTM_100`
* **Decision threshold**: **0.50**

**On the balanced test set (diagnostic 1:1):**

* **Weighted F1**: **0.801**
* **Accuracy**: 0.801
* **Class 1 (hypo) — Precision**: **0.956**
* **Class 1 — Recall (Sensitivity)**: **0.642**
* **ROC‑AUC**: 0.906
* **PR‑AUC**: 0.903 *(note: PR‑AUC depends on prevalence; this is the balanced view)*

**Confusion matrix (balanced test, approximate counts):**

|            |       Pred 0 |      Pred 1 |
| ---------- | -----------: | ----------: |
| **True 0** | **TN = 130** |  **FP = 4** |
| **True 1** |  **FN = 48** | **TP = 86** |

That’s **very few false alarms** (FP) while catching ~64% of hypos.

**If you want a bit more sensitivity** (catch more hypos) and can tolerate a few more false positives, use **threshold = 0.40** for the same model/setup:

* Class 1 **Recall** ↑ to **0.664**
* Class 1 **Precision** → 0.935
* Weighted F1 ≈ 0.799
* Confusion matrix (balanced): **TN 126, FP 8, FN 45, TP 89**

---

## Why the “original test” looks deceptively great

On the **original (imbalanced) test**, the top rows (e.g., `none / LSTM_100 / thr 0.60`) show **very high weighted F1 (≈0.85–0.96)** but **very low Class 1 recall (~0.13–0.37)**. That’s because the dataset is dominated by Class 0, so a model that predicts negatives most of the time can look “great” overall while **missing most hypos**. This is a classic class‑imbalance effect.

That’s why your **testBalanced** view is important: it reveals how well the model actually detects positives.

---

## Method & architecture comparison (balanced test)

Top performer per **resampling method** (sorted by Weighted F1):

1. **oversample_seq + LSTM_100, thr 0.50**

   * Weighted F1 **0.801**, Acc 0.801, **Precision₁ 0.956**, **Recall₁ 0.642**
2. **undersample_seq + BiLSTM, thr 0.40**

   * Weighted F1 0.792, Acc 0.792, Precision₁ 0.860, **Recall₁ 0.687** *(best sensitivity among the top)*
3. **none + LSTM_50, thr 0.50**

   * Weighted F1 0.740, Acc 0.740, Precision₁ 0.831, Recall₁ 0.642

**Takeaway:**

* **`LSTM_100`** is consistently the strongest backbone.
* **Oversampling** improves **precision while retaining good recall**; **undersampling** nudges recall highest (but with more false alarms).
* **No resampling** underperforms for the positive class.

---

## AUC perspective (threshold‑free)

* For **oversample_seq + LSTM_100**:

  * **ROC‑AUC (original test)**: ~**0.886**
  * **PR‑AUC (original test)**: ~**0.336** (low due to class rarity; typical)
  * **ROC‑AUC (balanced test)**: ~**0.906**
  * **PR‑AUC (balanced test)**: ~**0.903** *(inflated by 50% prevalence; use for diagnostics only)*

The ROC‑AUCs are stable and indicate a **strong ranking ability**. PR‑AUC on the original test is more honest about the difficulty of the rare positives.

---

## Generalization check (same method/model/threshold across splits)

For **oversample_seq + LSTM_100 @ thr 0.50**:

* **Train** Weighted F1 ≈ **0.972**
* **Val** Weighted F1 ≈ **0.946**
* **Test (original)** Weighted F1 ≈ **0.950**
* **TestBalanced** Weighted F1 ≈ **0.801**

The drop on **testBalanced** is expected because the class prior is forced to 50/50; it does **not** indicate overfitting. Train/Val/Test are tightly aligned.

---

## Recommendations

1. **Deploy default:** `oversample_seq + LSTM_100` with **threshold = 0.50**

   * Great precision on hypos (few false alarms) with reasonable sensitivity.

2. **Sensitivity mode:** set **threshold = 0.40**

   * Use when **missing a hypo is costlier** than an extra false alert.

3. **If you want even more recall**, consider `undersample_seq + BiLSTM @ thr 0.40` (Recall₁ ≈ **0.687** on balanced test), but expect more false positives.

4. **Calibrate to clinical preference:** You can choose threshold by optimizing **Fβ** (e.g., β=2 for recall‑heavy) on the **validation set**, then lock that threshold for the test/deployment.

5. **Next steps to squeeze more recall without losing precision:**

   * Try adding **pca_cgm2/3** and **hour_of_day** to sequence features.
   * Small **temporal dropout/label smoothing** to stabilize.
   * **Patient‑grouped CV** to confirm robustness.
   * **Threshold per risk period** (e.g., nocturnal vs daytime) using hour‑of‑day.

---

### Quick reference (best configurations)

* **Best balanced overall**: `oversample_seq / LSTM_100 / thr=0.50`
  **Weighted F1 0.801 · Acc 0.801 · Prec₁ 0.956 · Rec₁ 0.642 · ROC‑AUC 0.906 · PR‑AUC 0.903**

* **More recall**: `oversample_seq / LSTM_100 / thr=0.40`
  **Weighted F1 0.799 · Acc 0.799 · Prec₁ 0.935 · Rec₁ 0.664**

* **Highest recall among top‑2**: `undersample_seq / BiLSTM / thr=0.40`
  **Weighted F1 0.792 · Acc 0.792 · Prec₁ 0.860 · Rec₁ 0.687**

If you want, I can generate a compact leaderboard table (or plots) from this file showing the top N runs for each split and highlight the trade‑offs between precision and recall.


In [ ]:
# ============================
# Results analysis & reporting
# ============================
import os
import io
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

from sklearn.metrics import (
    confusion_matrix, roc_curve, precision_recall_curve, auc, average_precision_score
)

import tensorflow as tf

# ---------- Configuration ----------
RESULTS_CSV_CANDIDATES = [
    "/mnt/data/results_summary_all (9).csv",
    "/mnt/data/results_summary_all.csv",
    "/kaggle/working/results_summary_all.csv",
    "/kaggle/working/outputs/results_summary_all.csv",
    "outputs/results_summary_all.csv"
]
SEQ_NPZ_CANDIDATES = [
    "/kaggle/working/sequences_leakfree.npz",
    "/mnt/data/sequences_leakfree.npz",
    "sequences_leakfree.npz"
]
CHECKPOINT_DIR = "checkpoints"  # expects files like: checkpoints/{Method}__{Model}.h5

# Your canonical visit/static lists (for feature breakdown)
VISIT_COLS = ["carb","meals","total_daily_dose_u","fasting_percent_29"]
STATIC_COLS = [
    "Age","Gender","BMI","HbA1C","Cholesterol","LDL","HDL","Triglycerides",
    "eGFR","Creatinine","Insulin_units_per_kg","SmartGuard_percent"
]
# Which methods to summarize
METHODS_FOR_TOP5 = ["none", "oversample_seq", "undersample_seq"]

# ---------- Helpers ----------
def _first_existing(path_list):
    for p in path_list:
        if os.path.exists(p):
            return p
    return None

def _ensure_columns(df: pd.DataFrame):
    """
    Ensure df has Method/Model/Threshold/Split. If missing, try to parse from 'Key'.
    """
    need = {"Method","Model","Threshold","Split"}
    if need.issubset(df.columns):
        return df
    if "Key" not in df.columns:
        raise KeyError("Results file is missing Method/Model/Threshold/Split and has no 'Key' column to parse.")
    parts = df["Key"].astype(str).str.split("__")
    df["Method"] = parts.str[0]
    df["Model"]  = parts.str[1]
    # Threshold is typically in the 3rd token as 'thr_0.50'
    thr_str = parts.str[2].str.replace("thr_","", regex=False)
    df["Threshold"] = pd.to_numeric(thr_str, errors="coerce")
    # Split can be at the end of the Key; back off to explicit Split col if present
    key = df["Key"].astype(str)
    df["Split"] = np.where(key.str.endswith("__train"), "train",
                   np.where(key.str.endswith("__val"), "val",
                   np.where(key.str.endswith("__testBalanced"), "testBalanced",
                   np.where(key.str.endswith("__test"), "test", df.get("Split", np.nan)))))
    return df

def _round_or_none(x, nd=3):
    try:
        return round(float(x), nd)
    except Exception:
        return np.nan

def _safe_confmat_from_row(row: pd.Series):
    """
    Reconstructs an integer confusion matrix from supports + recall/specificity metrics in the row.
    Assumes:
      - Class1/Recall = TP / P
      - Class1/Specificity = TN / N (specificity wrt positives decision among negatives)
      - Class0/Support = N, Class1/Support = P
    """
    s0 = int(row.get("Class0/Support", 0))  # negatives
    s1 = int(row.get("Class1/Support", 0))  # positives
    rec1 = float(row.get("Class1/Recall", np.nan))
    sp1  = float(row.get("Class1/Specificity", np.nan))

    if any([not np.isfinite(v) for v in [s0, s1, rec1, sp1]]):
        raise ValueError("Cannot reconstruct confusion matrix: missing supports/recall/specificity in row.")

    tp = int(round(rec1 * s1))
    fn = max(0, s1 - tp)
    tn = int(round(sp1 * s0))
    fp = max(0, s0 - tn)

    # Small adjustments to keep sums consistent
    tn = max(0, min(tn, s0))
    fp = s0 - tn
    tp = max(0, min(tp, s1))
    fn = s1 - tp

    return np.array([[tn, fp],
                     [fn, tp]], dtype=int)

def focal_loss(gamma=2.0, alpha=0.25):
    # For loading custom-loss models
    bce = tf.keras.losses.BinaryCrossentropy(from_logits=False, reduction=tf.keras.losses.Reduction.NONE)
    eps = tf.keras.backend.epsilon()
    def loss(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, eps, 1.0 - eps)
        ce = bce(y_true, y_pred)
        p_t = y_true * y_pred + (1.0 - y_true) * (1.0 - y_pred)
        alpha_t = y_true * alpha + (1.0 - y_true) * (1.0 - alpha)
        modulating = tf.pow(1.0 - p_t, gamma)
        return alpha_t * modulating * ce
    return loss

def _try_load_sequences(npz_candidates):
    p = _first_existing(npz_candidates)
    if not p:
        print("⚠️ sequences_leakfree.npz not found. Feature counts, shapes and ROC/PR will be limited.")
        return None
    npz = np.load(p, allow_pickle=True)
    # unpack with fallbacks
    data = {
        "train": {"X_seq": npz["Xtr"], "y": npz["ytr"]},
        "val":   {"X_seq": npz["Xva"], "y": npz["yva"]},
        "test":  {"X_seq": npz["Xte"], "y": npz["yte"]},
        "seq_features_used": list(npz["seq_features_used"].tolist()) if "seq_features_used" in npz.files else [],
        "static_features_used": list(npz["static_features_used"].tolist()) if "static_features_used" in npz.files else []
    }
    # Optional static inputs
    if "Xtr_stat" in npz.files and npz["Xtr_stat"].size > 0:
        data["train"]["X_stat"] = npz["Xtr_stat"]
    else:
        data["train"]["X_stat"] = None

    if "Xva_stat" in npz.files and npz["Xva_stat"].size > 0:
        data["val"]["X_stat"] = npz["Xva_stat"]
    else:
        data["val"]["X_stat"] = None

    if "Xte_stat" in npz.files and npz["Xte_stat"].size > 0:
        data["test"]["X_stat"] = npz["Xte_stat"]
    else:
        data["test"]["X_stat"] = None

    data["npz_path"] = p
    return data

def _predict_loaded_model(model, X_seq, X_stat=None):
    # Allow models with one or two inputs
    try:
        if isinstance(model.input, list) or len(model.inputs) > 1:
            if X_stat is None:
                raise ValueError("Model expects static input but none provided.")
            preds = model.predict([X_seq, X_stat], verbose=0).ravel()
        else:
            preds = model.predict(X_seq, verbose=0).ravel()
        return preds
    except Exception as e:
        print(f"⚠️ Prediction failed: {e}")
        return None

# ---------- 1) Load results ----------
res_path = _first_existing(RESULTS_CSV_CANDIDATES)
if not res_path:
    raise FileNotFoundError("Could not find results CSV. Please update RESULTS_CSV_CANDIDATES.")
print(f"📄 Using results file: {res_path}")
df = pd.read_csv(res_path)
df = _ensure_columns(df)

# ---------- 1) Top-5 tables per method (by VAL F1_weighted) ----------
want_cols_map = {
    "Model": "Model",
    "Split": "Split",
    "Threshold": "Threshold",
    "Accuracy": "Overall/Accuracy",
    "Precision_weighted": "Overall/Precision_weighted",
    "Recall_weighted": "Overall/Recall_weighted",
    "F1_weighted": "Overall/F1_weighted",
    "Precision_1": "Class1/Precision",
    "Recall_1": "Class1/Recall",
    "F1_1": "Class1/F1",
    "Specificity_1": "Class1/Specificity",
    "ROC_AUC": "Overall/ROC-AUC",
    "PR_AUC": "Overall/PR-AUC",
    "Brier": "Overall/MSE_prob"
}

df_val = df[df["Split"].str.lower() == "val"].copy()
all_top5 = []
for m in METHODS_FOR_TOP5:
    sub = df_val[df_val["Method"] == m].copy()
    sub = sub.dropna(subset=["Overall/F1_weighted"])
    sub = sub.sort_values("Overall/F1_weighted", ascending=False).head(5)
    if sub.empty:
        continue
    out = pd.DataFrame({
        k: sub[v].values if v in sub.columns else np.nan
        for k, v in want_cols_map.items()
    })
    out.insert(0, "Method", m)
    all_top5.append(out)

top5_df = pd.concat(all_top5, ignore_index=True) if all_top5 else pd.DataFrame(columns=["Method"]+list(want_cols_map.keys()))
top5_df_rounded = top5_df.copy()
for c in ["Threshold","Accuracy","Precision_weighted","Recall_weighted","F1_weighted",
          "Precision_1","Recall_1","F1_1","Specificity_1","ROC_AUC","PR_AUC","Brier"]:
    if c in top5_df_rounded.columns:
        top5_df_rounded[c] = top5_df_rounded[c].apply(lambda x: _round_or_none(x, 4))

print("\n=== Top-5 by VAL F1_weighted for each method (none / oversample_seq / undersample_seq) ===")
print(top5_df_rounded.to_string(index=False))

# Save
top5_out_path = "/kaggle/working/top5_summary_per_method.csv" if os.path.exists("/kaggle/working") else "top5_summary_per_method.csv"
top5_df_rounded.to_csv(top5_out_path, index=False)
print(f"\n💾 Saved top-5 summary → {top5_out_path}")

# ---------- 2) Confusion matrix for BEST VAL F1 model ----------
if df_val.empty:
    print("\n⚠️ No validation rows found; cannot select best VAL F1 model.")
else:
    best_row = df_val.sort_values("Overall/F1_weighted", ascending=False).iloc[0]
    cm = _safe_confmat_from_row(best_row)
    print("\n=== Best VAL model (by F1_weighted) ===")
    print(f"Method={best_row['Method']} | Model={best_row['Model']} | thr={best_row['Threshold']:.2f}")
    print("Confusion Matrix [VAL]:")
    print(pd.DataFrame(cm, index=["True 0","True 1"], columns=["Pred 0","Pred 1"]).to_string())

    # Plot & save PNG
    fig, ax = plt.subplots(figsize=(4.5, 4))
    im = ax.imshow(cm, cmap="Blues")
    ax.set_title(f"Confusion Matrix (VAL)\n{best_row['Method']} / {best_row['Model']} @ thr={best_row['Threshold']:.2f}")
    ax.set_xlabel("Predicted"); ax.set_ylabel("Actual")
    ax.set_xticks([0,1]); ax.set_xticklabels(["0","1"])
    ax.set_yticks([0,1]); ax.set_yticklabels(["0","1"])
    # text
    for (i,j), v in np.ndenumerate(cm):
        ax.text(j, i, str(v), ha="center", va="center", fontsize=12, color=("white" if cm[i,j]>cm.max()/2 else "black"))
    fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    plt.tight_layout()
    cm_png_path = "/kaggle/working/confusion_matrix_best_val.png" if os.path.exists("/kaggle/working") else "confusion_matrix_best_val.png"
    plt.savefig(cm_png_path, dpi=200)
    plt.close(fig)
    print(f"🖼️ Saved confusion matrix PNG → {cm_png_path}")

# ---------- 3) ROC/PR curves for best 5 (by VAL F1) ----------
# We will try to load the checkpoints and the sequences NPZ.
seq_data = _try_load_sequences(SEQ_NPZ_CANDIDATES)
if seq_data is None:
    print("\n⚠️ Skipping ROC/PR (sequences NPZ not found).")
else:
    # pick top 5 by VAL F1 overall (unique Method/Model)
    top5_overall = (df_val
                    .dropna(subset=["Overall/F1_weighted"])
                    .sort_values("Overall/F1_weighted", ascending=False))
    # keep first occurrence per (Method, Model)
    top5_overall = top5_overall.drop_duplicates(subset=["Method","Model"]).head(5)
    if top5_overall.empty:
        print("\n⚠️ No candidates for ROC/PR plotting.")
    else:
        roc_handles = []
        pr_handles  = []
        fig_roc, ax_roc = plt.subplots(figsize=(6.5,5))
        fig_pr,  ax_pr  = plt.subplots(figsize=(6.5,5))

        yte = seq_data["test"]["y"].astype(int)
        Xte = seq_data["test"]["X_seq"]
        Xte_stat = seq_data["test"].get("X_stat", None)

        for _, r in top5_overall.iterrows():
            meth, arch = r["Method"], r["Model"]
            ckpt = os.path.join(CHECKPOINT_DIR, f"{meth}__{arch}.h5")
            if not os.path.exists(ckpt):
                print(f"⚠️ Checkpoint not found for {meth}/{arch}: {ckpt} — skipping.")
                continue

            try:
                model = tf.keras.models.load_model(ckpt, custom_objects={"loss": focal_loss()}, compile=False)
            except Exception as e:
                print(f"⚠️ Failed to load model {ckpt}: {e}")
                continue

            y_prob = _predict_loaded_model(model, Xte, X_stat=Xte_stat)
            if y_prob is None:
                continue

            # ROC
            try:
                fpr, tpr, _ = roc_curve(yte, y_prob)
                roc_auc = auc(fpr, tpr)
                ax_roc.plot(fpr, tpr, label=f'{meth}/{arch} (AUC={roc_auc:.3f})')
            except Exception as e:
                print(f"⚠️ ROC failed for {meth}/{arch}: {e}")

            # PR
            try:
                prec, rec, _ = precision_recall_curve(yte, y_prob)
                ap = average_precision_score(yte, y_prob)
                ax_pr.plot(rec, prec, label=f'{meth}/{arch} (AP={ap:.3f})')
            except Exception as e:
                print(f"⚠️ PR failed for {meth}/{arch}: {e}")

        # finalize ROC
        ax_roc.plot([0,1],[0,1],'--', color='gray', label='Random')
        ax_roc.set_title("ROC — Best 5 by VAL F1")
        ax_roc.set_xlabel("False Positive Rate")
        ax_roc.set_ylabel("True Positive Rate")
        ax_roc.legend(fontsize=8)
        plt.tight_layout()
        roc_png = "/kaggle/working/best5_roc.png" if os.path.exists("/kaggle/working") else "best5_roc.png"
        fig_roc.savefig(roc_png, dpi=250); plt.close(fig_roc)
        print(f"🖼️ Saved ROC curves → {roc_png}")

        # finalize PR
        ax_pr.set_title("Precision–Recall — Best 5 by VAL F1")
        ax_pr.set_xlabel("Recall")
        ax_pr.set_ylabel("Precision")
        ax_pr.legend(fontsize=8)
        plt.tight_layout()
        pr_png = "/kaggle/working/best5_pr.png" if os.path.exists("/kaggle/working") else "best5_pr.png"
        fig_pr.savefig(pr_png, dpi=250); plt.close(fig_pr)
        print(f"🖼️ Saved PR curves → {pr_png}")

# ---------- 4) Feature counts (hourly vs visit vs static) ----------
if seq_data is not None:
    seq_feats = seq_data.get("seq_features_used", []) or []
    static_feats = seq_data.get("static_features_used", []) or []
    visit_used = [f for f in seq_feats if f in VISIT_COLS]
    hourly_used = [f for f in seq_feats if f not in VISIT_COLS]

    print("\n=== Feature sets used (from NPZ) ===")
    print(f"Hourly features after transform: {len(hourly_used)} → {hourly_used}")
    print(f"Static features after transform: {len(static_feats)} → {static_feats}")
    print(f"Visit features: {len(visit_used)} → {visit_used}")
else:
    print("\n⚠️ Feature counts unavailable (NPZ not found).")

# ---------- 5) Sequence shapes & class distribution ----------
def _shape_or_na(arr):
    try:
        return tuple(arr.shape)
    except Exception:
        return "(NA)"

def _dist_str(y):
    if y is None or len(y)==0:
        return "{ } (pos=NA)"
    cnt = Counter(np.asarray(y).astype(int).tolist())
    pos = cnt.get(1,0); tot = sum(cnt.values())
    pct = 100.0*pos/max(1,tot)
    return f"{dict(cnt)} (pos={pct:.1f}%)"

if seq_data is not None:
    trX, vaX, teX = seq_data["train"]["X_seq"], seq_data["val"]["X_seq"], seq_data["test"]["X_seq"]
    print("\n=== Sequence shapes ===")
    print(f"Train seq: {_shape_or_na(trX)}")
    print(f"Val   seq: {_shape_or_na(vaX)}")
    print(f"Test  seq: {_shape_or_na(teX)}")

    print("\n=== Class distribution ===")
    print(f"🔎 Train sequences: {_dist_str(seq_data['train']['y'])}")
    print(f"🔎 Val sequences:   {_dist_str(seq_data['val']['y'])}")
    print(f"🔎 Test sequences:  {_dist_str(seq_data['test']['y'])}")
else:
    print("\n⚠️ Sequence shapes & class distribution unavailable (NPZ not found).")

# ---------- 6) Print structure of the BEST model ----------
if df_val.empty:
    print("\n⚠️ No validation rows → cannot determine best model for summary.")
else:
    best_row = df_val.sort_values("Overall/F1_weighted", ascending=False).iloc[0]
    meth, arch = best_row["Method"], best_row["Model"]
    ckpt = os.path.join(CHECKPOINT_DIR, f"{meth}__{arch}.h5")
    if not os.path.exists(ckpt):
        print(f"\n⚠️ Best model checkpoint not found: {ckpt}")
    else:
        try:
            model = tf.keras.models.load_model(ckpt, custom_objects={"loss": focal_loss()}, compile=False)
            s = io.StringIO()
            model.summary(print_fn=lambda x: s.write(x + "\n"))
            summary_text = s.getvalue()
            print("\n=== Best model structure (Keras summary) ===")
            print(summary_text)
            # Save to file
            summary_path = "/kaggle/working/best_model_summary.txt" if os.path.exists("/kaggle/working") else "best_model_summary.txt"
            with open(summary_path, "w") as f:
                f.write(summary_text)
            print(f"💾 Saved best model summary → {summary_path}")
        except Exception as e:
            print(f"\n⚠️ Failed to load/print model summary: {e}")


In [ ]:
# ===========================================
# Extra: fixed-confusion-matrix PNGs for
#  - oversample_seq + LSTM_100 @ thr 0.50
#  - undersample_seq + BiLSTM  @ thr 0.40
#  - none          + LSTM_50   @ thr 0.50
# ===========================================
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---- where to find the results file ----
RESULTS_CSV_CANDIDATES = [
    "/mnt/data/results_summary_all (9).csv",
    "/mnt/data/results_summary_all.csv",
    "/kaggle/working/results_summary_all.csv",
    "/kaggle/working/outputs/results_summary_all.csv",
    "outputs/results_summary_all.csv"
]

def _first_existing(paths):
    for p in paths:
        if os.path.exists(p):
            return p
    return None

def _ensure_columns(df: pd.DataFrame):
    """Make sure Method/Model/Threshold/Split are present (parse from Key if needed)."""
    need = {"Method","Model","Threshold","Split"}
    if need.issubset(df.columns):
        return df
    if "Key" not in df.columns:
        raise KeyError("Results file missing Method/Model/Threshold/Split and no 'Key' to parse.")
    parts = df["Key"].astype(str).str.split("__")
    df["Method"] = parts.str[0]
    df["Model"]  = parts.str[1]
    thr_str = parts.str[2].str.replace("thr_","", regex=False)
    df["Threshold"] = pd.to_numeric(thr_str, errors="coerce")
    key = df["Key"].astype(str)
    df["Split"] = np.where(key.str.endswith("__train"), "train",
                   np.where(key.str.endswith("__val"),   "val",
                   np.where(key.str.endswith("__testBalanced"), "testBalanced",
                   np.where(key.str.endswith("__test"), "test", np.nan))))
    return df

def _confmat_from_row(row: pd.Series):
    """
    Build confusion matrix from supports + recall/spec recorded in results CSV:
      - Class1/Recall = TP / P
      - Class1/Specificity = TN / N  (fallback to Class0/Recall if needed)
      - Class0/Support = N, Class1/Support = P
    """
    s0 = int(row.get("Class0/Support", 0))  # negatives
    s1 = int(row.get("Class1/Support", 0))  # positives
    rec1 = float(row.get("Class1/Recall", np.nan))

    sp1 = row.get("Class1/Specificity", np.nan)
    if not np.isfinite(sp1):
        # fallback: recall for class 0 equals specificity wrt class 1
        sp1 = float(row.get("Class0/Recall", np.nan))

    if not all(np.isfinite([s0, s1, rec1, sp1])):
        raise ValueError("Row lacks required metrics to reconstruct confusion matrix.")

    tp = int(round(rec1 * s1))
    fn = s1 - tp
    tn = int(round(sp1 * s0))
    fp = s0 - tn

    # clamp for safety
    tn = max(0, min(tn, s0)); fp = s0 - tn
    tp = max(0, min(tp, s1)); fn = s1 - tp

    return np.array([[tn, fp],
                     [fn, tp]], dtype=int)

def _plot_cm(cm: np.ndarray, title: str, out_path: str):
    fig, ax = plt.subplots(figsize=(4.8, 4.2))
    im = ax.imshow(cm, cmap="Blues")
    ax.set_title(title)
    ax.set_xlabel("Predicted"); ax.set_ylabel("Actual")
    ax.set_xticks([0,1]); ax.set_xticklabels(["0","1"])
    ax.set_yticks([0,1]); ax.set_yticklabels(["0","1"])
    for (i,j), v in np.ndenumerate(cm):
        ax.text(j, i, str(v),
                ha="center", va="center",
                color=("white" if v > cm.max()/2 else "black"),
                fontsize=12)
    fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    plt.tight_layout()
    plt.savefig(out_path, dpi=220)
    plt.close(fig)

# ---- load results ----
res_path = _first_existing(RESULTS_CSV_CANDIDATES)
if not res_path:
    raise FileNotFoundError("Could not find results_summary_all*.csv — update RESULTS_CSV_CANDIDATES.")
print(f"📄 Using results file: {res_path}")
res = pd.read_csv(res_path)
res = _ensure_columns(res)

# We'll take the 'test' split by default (change SPLIT to "val" if you want VAL instead)
SPLIT = "test"

# ---- combos you requested ----
requests = [
    ("oversample_seq", "LSTM_100", 0.50),
    ("undersample_seq", "BiLSTM",  0.40),
    ("none",           "LSTM_50",  0.50),
]

# ---- make outputs dir if on Kaggle ----
out_dir = "/kaggle/working" if os.path.exists("/kaggle/working") else "."
outputs = []

for method, model, thr in requests:
    # filter rows (case-insensitive Split; Threshold rounded to 2dp in pipeline)
    sub = res[
        (res["Method"] == method) &
        (res["Model"]  == model)  &
        (res["Split"].astype(str).str.lower() == SPLIT.lower())
    ].copy()

    if sub.empty:
        print(f"⚠️ No rows for {method} + {model} on split='{SPLIT}'. Skipping.")
        continue

    # match threshold to 2 decimals
    # accept both exact and near-equal due to float rounding
    def _thr_match(x):
        try:
            return (round(float(x), 2) == round(float(thr), 2)) or (abs(float(x) - float(thr)) < 1e-6)
        except Exception:
            return False

    sub = sub[sub["Threshold"].apply(_thr_match)]
    if sub.empty:
        print(f"⚠️ No threshold={thr:.2f} row for {method} + {model} on split='{SPLIT}'. Skipping.")
        continue

    # If multiple rows (rare), take the first
    row = sub.iloc[0]
    cm = _confmat_from_row(row)

    # Print nicely
    print(f"\n=== Confusion Matrix — {method} + {model} @ thr={thr:.2f} [{SPLIT}] ===")
    print(pd.DataFrame(cm, index=["True 0","True 1"], columns=["Pred 0","Pred 1"]).to_string())

    # Save PNG
    safe_method = method.replace("/","-")
    safe_model  = model.replace("/","-")
    out_png = os.path.join(out_dir, f"cm_{safe_method}__{safe_model}__thr_{thr:.2f}__{SPLIT}.png")
    _plot_cm(cm, f"{method} + {model} @ thr={thr:.2f} [{SPLIT}]", out_png)
    outputs.append(out_png)

print("\n🖼️ Saved confusion matrix images:")
for p in outputs:
    print(" -", p)


#  fixed-confusion-matrix PNGs

In [ ]:
# ===========================================
# Extra: fixed-confusion-matrix PNGs for
#  - oversample_seq + LSTM_100 @ thr 0.50
#  - undersample_seq + BiLSTM  @ thr 0.40
#  - none          + LSTM_50   @ thr 0.50
# ===========================================
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---- where to find the results file ----
RESULTS_CSV_CANDIDATES = [
    "/mnt/data/results_summary_all (9).csv",
    "/mnt/data/results_summary_all.csv",
    "/kaggle/working/results_summary_all.csv",
    "/kaggle/working/outputs/results_summary_all.csv",
    "outputs/results_summary_all.csv"
]

def _first_existing(paths):
    for p in paths:
        if os.path.exists(p):
            return p
    return None

def _ensure_columns(df: pd.DataFrame):
    """Make sure Method/Model/Threshold/Split are present (parse from Key if needed)."""
    need = {"Method","Model","Threshold","Split"}
    if need.issubset(df.columns):
        return df
    if "Key" not in df.columns:
        raise KeyError("Results file missing Method/Model/Threshold/Split and no 'Key' to parse.")
    parts = df["Key"].astype(str).str.split("__")
    df["Method"] = parts.str[0]
    df["Model"]  = parts.str[1]
    thr_str = parts.str[2].str.replace("thr_","", regex=False)
    df["Threshold"] = pd.to_numeric(thr_str, errors="coerce")
    key = df["Key"].astype(str)
    df["Split"] = np.where(key.str.endswith("__train"), "train",
                   np.where(key.str.endswith("__val"),   "val",
                   np.where(key.str.endswith("__testBalanced"), "testBalanced",
                   np.where(key.str.endswith("__test"), "test", np.nan))))
    return df

def _confmat_from_row(row: pd.Series):
    """
    Build confusion matrix from supports + recall/spec recorded in results CSV:
      - Class1/Recall = TP / P
      - Class1/Specificity = TN / N  (fallback to Class0/Recall if needed)
      - Class0/Support = N, Class1/Support = P
    """
    s0 = int(row.get("Class0/Support", 0))  # negatives
    s1 = int(row.get("Class1/Support", 0))  # positives
    rec1 = float(row.get("Class1/Recall", np.nan))

    sp1 = row.get("Class1/Specificity", np.nan)
    if not np.isfinite(sp1):
        # fallback: recall for class 0 equals specificity wrt class 1
        sp1 = float(row.get("Class0/Recall", np.nan))

    if not all(np.isfinite([s0, s1, rec1, sp1])):
        raise ValueError("Row lacks required metrics to reconstruct confusion matrix.")

    tp = int(round(rec1 * s1))
    fn = s1 - tp
    tn = int(round(sp1 * s0))
    fp = s0 - tn

    # clamp for safety
    tn = max(0, min(tn, s0)); fp = s0 - tn
    tp = max(0, min(tp, s1)); fn = s1 - tp

    return np.array([[tn, fp],
                     [fn, tp]], dtype=int)

def _plot_cm(cm: np.ndarray, title: str, out_path: str):
    fig, ax = plt.subplots(figsize=(4.8, 4.2))
    im = ax.imshow(cm, cmap="Blues")
    ax.set_title(title)
    ax.set_xlabel("Predicted"); ax.set_ylabel("Actual")
    ax.set_xticks([0,1]); ax.set_xticklabels(["0","1"])
    ax.set_yticks([0,1]); ax.set_yticklabels(["0","1"])
    for (i,j), v in np.ndenumerate(cm):
        ax.text(j, i, str(v),
                ha="center", va="center",
                color=("white" if v > cm.max()/2 else "black"),
                fontsize=12)
    fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    plt.tight_layout()
    plt.savefig(out_path, dpi=220)
    plt.close(fig)

# ---- load results ----
res_path = _first_existing(RESULTS_CSV_CANDIDATES)
if not res_path:
    raise FileNotFoundError("Could not find results_summary_all*.csv — update RESULTS_CSV_CANDIDATES.")
print(f"📄 Using results file: {res_path}")
res = pd.read_csv(res_path)
res = _ensure_columns(res)

# We'll take the 'test' split by default (change SPLIT to "val" if you want VAL instead)
SPLIT = "test"

# ---- combos you requested ----
requests = [
    ("oversample_seq", "LSTM_100", 0.50),
    ("undersample_seq", "BiLSTM",  0.40),
    ("none",           "LSTM_50",  0.50),
]

# ---- make outputs dir if on Kaggle ----
out_dir = "/kaggle/working" if os.path.exists("/kaggle/working") else "."
outputs = []

for method, model, thr in requests:
    # filter rows (case-insensitive Split; Threshold rounded to 2dp in pipeline)
    sub = res[
        (res["Method"] == method) &
        (res["Model"]  == model)  &
        (res["Split"].astype(str).str.lower() == SPLIT.lower())
    ].copy()

    if sub.empty:
        print(f"⚠️ No rows for {method} + {model} on split='{SPLIT}'. Skipping.")
        continue

    # match threshold to 2 decimals
    # accept both exact and near-equal due to float rounding
    def _thr_match(x):
        try:
            return (round(float(x), 2) == round(float(thr), 2)) or (abs(float(x) - float(thr)) < 1e-6)
        except Exception:
            return False

    sub = sub[sub["Threshold"].apply(_thr_match)]
    if sub.empty:
        print(f"⚠️ No threshold={thr:.2f} row for {method} + {model} on split='{SPLIT}'. Skipping.")
        continue

    # If multiple rows (rare), take the first
    row = sub.iloc[0]
    cm = _confmat_from_row(row)

    # Print nicely
    print(f"\n=== Confusion Matrix — {method} + {model} @ thr={thr:.2f} [{SPLIT}] ===")
    print(pd.DataFrame(cm, index=["True 0","True 1"], columns=["Pred 0","Pred 1"]).to_string())

    # Save PNG
    safe_method = method.replace("/","-")
    safe_model  = model.replace("/","-")
    out_png = os.path.join(out_dir, f"cm_{safe_method}__{safe_model}__thr_{thr:.2f}__{SPLIT}.png")
    _plot_cm(cm, f"{method} + {model} @ thr={thr:.2f} [{SPLIT}]", out_png)
    outputs.append(out_png)

print("\n🖼️ Saved confusion matrix images:")
for p in outputs:
    print(" -", p)


# Shape 
### Fixed analysis + sequence utilities (single-source, de-duplicated)
### - build_sequences_by_split   (scales using actual X.shape, not SEQ_LEN)
### - _build_sequence_index_map  (one canonical copy)
### - compute_visit_shap         (robust, seq_len inferred if omitted)
### - aggregate_hourly_to_daily_risk (no hidden deps; direct model.predict)
### - make_balanced_test         (shape-safe)

In [ ]:
# ======================================================
# Fixed analysis + sequence utilities + RUNNER
#  - build_sequences_by_split   (scales using actual X.shape, not SEQ_LEN)
#  - _build_sequence_index_map  (one canonical copy)
#  - compute_visit_shap         (robust; seq_len inferred if omitted)
#  - aggregate_hourly_to_daily_risk (direct model.predict; no hidden deps)
#  - make_balanced_test         (shape-safe)
#  - run_analyses_all           (calls everything and prints outputs)
#  - quick model fit if none    (small LSTM; early stopping)
# ======================================================

import os
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# Optional TF imports only used in the quick-fit path
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout, Concatenate
from tensorflow.keras.callbacks import EarlyStopping

# ---------------------------
# Global guards / safe fallbacks
# ---------------------------
def _bool_env(name, default=False):
    v = os.environ.get(name)
    if v is None: return default
    return str(v).strip().lower() in {"1","true","yes","y","on"}

try:
    USE_STATIC_INPUT
except NameError:
    USE_STATIC_INPUT = True

try:
    RANDOM_STATE
except NameError:
    RANDOM_STATE = 42

try:
    DEFAULT_SEQ_FEATURE_COLS
except NameError:
    DEFAULT_SEQ_FEATURE_COLS = (
        "cgm_mean","cgm_std","pca_cgm1",
        "pc1_activity_energy",
        "carb","meals","total_daily_dose_u","fasting_percent_29"
    )

try:
    STATIC_COLS
except NameError:
    STATIC_COLS = [
        "Age","Gender","BMI","HbA1C","Cholesterol","LDL","HDL","Triglycerides",
        "eGFR","Creatinine","Insulin_units_per_kg","SmartGuard_percent"
    ]

try:
    VISIT_COLS
except NameError:
    VISIT_COLS = ["carb","meals","total_daily_dose_u","fasting_percent_29"]

# Analysis defaults
VISIT_FEATURES       = VISIT_COLS
SHAP_BACKGROUND_SIZE = 512
SHAP_TEST_SAMPLES    = 1024
RISK_THRESHOLD       = 0.50

def _check_globals():
    """No-op guard used by helpers that previously referenced external globals."""
    pass

# ======================================================
# build_sequences_by_split (uses actual X.shape[1] when reshaping)
# ======================================================
def build_sequences_by_split(
    hourly: pd.DataFrame,
    splits,
    seq_len: int,
    seq_feature_cols=DEFAULT_SEQ_FEATURE_COLS,
    static_cols=STATIC_COLS,
    scale_features: bool = True
):
    """
    Build (X_seq, X_stat, y) arrays for train/val/test given hourly data and patient splits.
    - Scalers are fit on TRAIN only.
    - Uses actual window length (T) from arrays when reshaping; no reliance on global SEQ_LEN.
    """
    # Required columns
    for c in ["patientID","hour","hypo_label","Split"]:
        if c not in hourly.columns:
            raise KeyError(f"hourly missing required column: {c}")
    hourly = hourly.copy()
    hourly["hour"] = pd.to_datetime(hourly["hour"], errors="coerce")

    # Sequence features presence
    seq_feature_cols = list(seq_feature_cols)
    missing_seq = [c for c in seq_feature_cols if c not in hourly.columns]
    if missing_seq:
        raise KeyError(f"Sequence feature(s) not found in hourly: {missing_seq}")

    # Static matrix per patient (fill zeros for missing patients)
    static_cols_present = [c for c in static_cols if c in hourly.columns]
    if static_cols_present and USE_STATIC_INPUT:
        static_mat = (hourly[["patientID"] + static_cols_present]
                      .drop_duplicates(subset=["patientID"])
                      .set_index("patientID")
                      .astype(float)
                      .fillna(0.0))
    else:
        static_mat = None
        static_cols_present = []

    train_p, val_p, test_p = splits

    def _build_for_pidset(pid_set):
        sub = hourly[hourly["patientID"].isin(pid_set)].copy()
        sub = sub.sort_values(["patientID","hour"]).reset_index(drop=True)
        X_seq, X_stat, y = [], [], []
        for pid, grp in sub.groupby("patientID", sort=True):
            grp = grp.sort_values("hour").reset_index(drop=True)
            if len(grp) <= seq_len:
                continue
            feats  = grp[seq_feature_cols].astype(float).values
            labels = grp["hypo_label"].astype(int).values
            for i in range(len(grp) - seq_len):
                X_seq.append(feats[i:i+seq_len])
                y.append(labels[i+seq_len])
                if USE_STATIC_INPUT and static_mat is not None:
                    if pid in static_mat.index:
                        X_stat.append(static_mat.loc[pid].values.astype(float))
                    else:
                        X_stat.append(np.zeros(len(static_cols_present), dtype=float))
        X_seq = np.array(X_seq)
        y     = np.array(y).astype(int)
        X_stat = np.array(X_stat) if (USE_STATIC_INPUT and static_mat is not None and len(X_stat)>0) else None
        return X_seq, X_stat, y

    Xtr_s, Xtr_stat, ytr = _build_for_pidset(train_p)
    Xva_s, Xva_stat, yva = _build_for_pidset(val_p)
    Xte_s, Xte_stat, yte = _build_for_pidset(test_p)

    # -------- Scale on TRAIN only (use actual T, F) --------
    seq_scaler  = None
    stat_scaler = None
    if scale_features and Xtr_s is not None and Xtr_s.size > 0:
        n_f = Xtr_s.shape[2]
        seq_scaler = StandardScaler().fit(Xtr_s.reshape(-1, n_f))
        def _scale_seq(X):
            if X is None or X.size == 0:
                return X
            n, T, F = X.shape
            return seq_scaler.transform(X.reshape(-1, F)).reshape(n, T, F)
        Xtr_s = _scale_seq(Xtr_s); Xva_s = _scale_seq(Xva_s); Xte_s = _scale_seq(Xte_s)

    if scale_features and Xtr_stat is not None and Xtr_stat.size > 0:
        stat_scaler = StandardScaler().fit(Xtr_stat)
        def _scale_stat(X):
            if X is None or X.size == 0:
                return X
            return stat_scaler.transform(X)
        Xtr_stat = _scale_stat(Xtr_stat); Xva_stat = _scale_stat(Xva_stat); Xte_stat = _scale_stat(Xte_stat)

    print(f"✅ Sequences built | train={getattr(Xtr_s,'shape',None)}, val={getattr(Xva_s,'shape',None)}, test={getattr(Xte_s,'shape',None)}")
    return {
        "train": {"X_seq": Xtr_s, "X_stat": Xtr_stat, "y": ytr},
        "val":   {"X_seq": Xva_s, "X_stat": Xva_stat, "y": yva},
        "test":  {"X_seq": Xte_s, "X_stat": Xte_stat, "y": yte},
        "seq_features_used": seq_feature_cols,
        "static_features_used": static_cols_present,
        "scalers": {"seq": seq_scaler, "stat": stat_scaler}
    }

# ======================================================
# Sequence index map (single canonical copy)
# ======================================================
def _build_sequence_index_map(hourly_df: pd.DataFrame, split: str, seq_len: int) -> pd.DataFrame:
    """
    Recreate the exact sequence ordering used in build_sequences_by_split so that
    index i maps to the (i+seq_len)-th hour row for each patient.

    Returns: ['seq_idx','patientID','hour','date','visit_assigned','period_main','row_idx']
    """
    sub = (hourly_df[hourly_df["Split"].astype(str).str.lower() == split.lower()]
           .sort_values(["patientID","hour"])
           .reset_index())
    sub = sub.rename(columns={"index": "row_idx"})

    rows = []
    for pid, grp in sub.groupby("patientID", sort=True):
        grp = grp.sort_values("hour").reset_index(drop=True)
        for i in range(len(grp) - seq_len):
            tgt = grp.loc[i+seq_len]
            rows.append({
                "seq_idx": len(rows),
                "patientID": pid,
                "hour": pd.to_datetime(tgt["hour"]),
                "date": pd.to_datetime(tgt.get("date", pd.NaT)),
                "visit_assigned": tgt.get("visit_assigned", np.nan),
                "period_main": tgt.get("period_main", np.nan),
                "row_idx": int(tgt["row_idx"]),
            })
    return pd.DataFrame(rows)

# ======================================================
# SHAP on visit features (robust; seq_len inference; shape checks)
# ======================================================
def compute_visit_shap(
    model,
    data,
    hourly: pd.DataFrame,
    seq_features_used,
    visit_features=None,
    split: str = "test",
    background_size: int = SHAP_BACKGROUND_SIZE,
    max_test_windows: int = SHAP_TEST_SAMPLES,
    out_dir: str = "/kaggle/working",
    seq_len: int = None
):
    """
    Computes global and per-visit SHAP importance for visit features that are INCLUDED
    in the sequence feature tensor. Saves two CSVs:
      - shap_visit_global.csv
      - shap_visit_per_visit.csv
    """
    _check_globals()
    os.makedirs(out_dir, exist_ok=True)
    visit_features = list(visit_features) if visit_features is not None else list(VISIT_FEATURES)

    # Infer seq_len from arrays if not provided
    Xte = data[split]["X_seq"]
    if seq_len is None:
        if Xte is None or Xte.ndim != 3:
            raise RuntimeError("Cannot infer seq_len from data arrays.")
        seq_len = Xte.shape[1]

    # Map visit features -> indices in the sequence features
    feat_to_idx = {f: i for i, f in enumerate(seq_features_used)}
    visit_in_seq = [f for f in visit_features if f in feat_to_idx]
    if not visit_in_seq:
        raise ValueError(
            f"None of the visit features are present in seq_features_used={seq_features_used}. "
            f"Ensure your DEFAULT_SEQ_FEATURE_COLS includes items from VISIT_COLS."
        )

    Xtr, Xtr_stat = data["train"]["X_seq"], data["train"]["X_stat"]
    Xte, Xte_stat = data[split]["X_seq"],  data[split]["X_stat"]

    bg_n = min(int(background_size), len(Xtr))
    te_n = min(int(max_test_windows), len(Xte))
    if bg_n < 1 or te_n < 1:
        raise RuntimeError(f"Not enough sequences for SHAP (bg_n={bg_n}, te_n={te_n}).")

    rng   = np.random.default_rng(42)
    bg_ix = rng.choice(np.arange(len(Xtr)), size=bg_n, replace=False)
    te_ix = rng.choice(np.arange(len(Xte)), size=te_n, replace=False)

    bg_seq, te_seq = Xtr[bg_ix], Xte[te_ix]
    if USE_STATIC_INPUT and (Xtr_stat is not None and Xte_stat is not None and Xtr_stat.size>0 and Xte_stat.size>0):
        bg_static, te_static = Xtr_stat[bg_ix], Xte_stat[te_ix]
    else:
        bg_static = te_static = None

    # ---- SHAP explainer
    try:
        import shap
    except Exception as e:
        raise ImportError(
            "This SHAP analysis requires the 'shap' package. Install with: pip install shap"
        ) from e

    try:
        if bg_static is not None:
            explainer   = shap.DeepExplainer(model, [bg_seq, bg_static])
            shap_values = explainer.shap_values([te_seq, te_static])
        else:
            explainer   = shap.DeepExplainer(model, bg_seq)
            shap_values = explainer.shap_values(te_seq)
    except Exception as e:
        print(f"[WARN] DeepExplainer failed ({e}). Falling back to GradientExplainer…")
        if bg_static is not None:
            explainer   = shap.GradientExplainer(model, [bg_seq, bg_static])
            shap_values = explainer.shap_values([te_seq, te_static])
        else:
            explainer   = shap.GradientExplainer(model, bg_seq)
            shap_values = explainer.shap_values(te_seq)

    shap_seq = shap_values[0] if isinstance(shap_values, list) else shap_values
    if shap_seq.ndim != 3:
        raise RuntimeError(f"Unexpected SHAP shape for sequence input: {shap_seq.shape}")

    # Reduce over time → mean |SHAP| across the window
    shap_abs_time = np.mean(np.abs(shap_seq), axis=1)  # [n_samples, F]

    # ---- GLOBAL visit feature importance
    rows = [{"feature": f, "mean_abs_shap": float(np.mean(shap_abs_time[:, feat_to_idx[f]]))}
            for f in visit_in_seq]
    global_visit_df = pd.DataFrame(rows).sort_values("mean_abs_shap", ascending=False)
    gpath = os.path.join(out_dir, "shap_visit_global.csv")
    global_visit_df.to_csv(gpath, index=False)
    print("✅ Saved global visit SHAP →", gpath)

    # ---- PER‑VISIT importance
    seq_map = _build_sequence_index_map(hourly, split=split, seq_len=seq_len)
    if len(seq_map) != len(Xte):
        raise RuntimeError(
            f"Mapping length {len(seq_map)} != X_{split} length {len(Xte)}. "
            f"seq_len={seq_len}. Rebuild hourly/sequences consistently."
        )
    seq_map_sub = seq_map.iloc[te_ix].reset_index(drop=True)

    per_rows = []
    for i in range(len(seq_map_sub)):
        pid = seq_map_sub.loc[i, "patientID"]
        dte = pd.to_datetime(seq_map_sub.loc[i, "date"])
        for f in visit_in_seq:
            per_rows.append({
                "patientID": pid,
                "date": dte,
                "feature": f,
                "mean_abs_shap": float(shap_abs_time[i, feat_to_idx[f]])
            })
    per_visit_df = (pd.DataFrame(per_rows)
                    .groupby(["patientID","date","feature"], as_index=False)["mean_abs_shap"].mean()
                    .sort_values(["patientID","date","mean_abs_shap"], ascending=[True, True, False]))
    ppath = os.path.join(out_dir, "shap_visit_per_visit.csv")
    per_visit_df.to_csv(ppath, index=False)
    print("✅ Saved per‑visit SHAP →", ppath)

    return global_visit_df, per_visit_df

# ======================================================
# Aggregate hourly predictions to daily risk
# ======================================================
def aggregate_hourly_to_daily_risk(
    model,
    data,
    hourly: pd.DataFrame,
    split: str = "test",
    threshold: float = RISK_THRESHOLD,
    out_dir: str = "/kaggle/working",
    seq_len: int = None
):
    """
    Aggregate sequence‑window predictions to daily risk summaries.
    Saves 'daily_risk_<split>.csv' and (best effort) a small example plot.
    """
    os.makedirs(out_dir, exist_ok=True)

    # Predictions
    Xs      = data[split]["X_seq"]
    Xs_stat = data[split]["X_stat"]
    y_true  = np.asarray(data[split]["y"]).astype(int).ravel()

    if seq_len is None:
        if Xs is None or Xs.ndim != 3:
            raise RuntimeError("Cannot infer seq_len from data arrays.")
        seq_len = Xs.shape[1]

    if USE_STATIC_INPUT and (Xs_stat is not None and Xs_stat.size>0):
        y_prob = model.predict([Xs, Xs_stat], verbose=0).ravel()
    else:
        y_prob = model.predict(Xs, verbose=0).ravel()
    y_pred = (y_prob >= float(threshold)).astype(int)

    # Map sequence rows back to hours/dates
    seq_map = _build_sequence_index_map(hourly, split=split, seq_len=seq_len)
    if len(seq_map) != len(y_true):
        raise RuntimeError(
            f"Sequence map length {len(seq_map)} != prediction length {len(y_true)}.\n"
            f"seq_len={seq_len}, X_{split}.shape={getattr(Xs, 'shape', None)}. "
            "Use the same hourly/sequences and seq_len that were used for training."
        )

    pred_df = pd.DataFrame({
        "patientID": seq_map["patientID"].values,
        "hour":      pd.to_datetime(seq_map["hour"].values),
        "date":      pd.to_datetime(seq_map["date"].values),
        "visit_assigned": seq_map.get("visit_assigned", pd.Series([np.nan]*len(seq_map))).values,
        "period_main":    seq_map.get("period_main", pd.Series([np.nan]*len(seq_map))).values,
        "y_true":    y_true,
        "y_prob":    y_prob,
        "y_pred":    y_pred
    })

    # Daily aggregates
    grp = pred_df.groupby(["patientID","date"], as_index=False)
    daily = grp.agg(
        n_windows=("y_true","size"),
        true_positives=("y_true","sum"),
        pred_positives=("y_pred","sum"),
        risk_mean=("y_prob","mean"),
        risk_max=("y_prob","max"),
        risk_p95=("y_prob", lambda x: float(np.quantile(x, 0.95))),
        hours_above_thr=("y_pred","sum")
    )
    daily["prevalence"] = daily["true_positives"] / daily["n_windows"].replace(0, np.nan)
    daily_csv = os.path.join(out_dir, f"daily_risk_{split}.csv")
    daily.to_csv(daily_csv, index=False)
    print(f"✅ Saved daily risk aggregates → {daily_csv}")

    # Optional quick plot
    try:
        example_pid = daily["patientID"].iloc[0]
        dsub = daily[daily["patientID"] == example_pid].sort_values("date")
        plt.figure(figsize=(8,3))
        plt.plot(dsub["date"], dsub["risk_mean"], label="Mean daily risk")
        plt.plot(dsub["date"], dsub["risk_max"],  label="Max daily risk")
        plt.axhline(threshold, linestyle="--", label=f"thr={threshold:.2f}")
        plt.xlabel("Date"); plt.ylabel("Risk"); plt.title(f"Daily risk — patient {example_pid}")
        plt.legend(); plt.tight_layout()
        png = os.path.join(out_dir, f"daily_risk_trend_patient_{example_pid}.png")
        plt.savefig(png, dpi=200); plt.close()
        print(f"🖼️ Saved example daily trend → {png}")
    except Exception as e:
        print(f"[WARN] Could not plot daily trend example: {e}")

    return pred_df, daily

# ======================================================
# Balanced test subset (shape-safe)
# ======================================================
def make_balanced_test(X_test, y_test, X_stat=None, random_state: int = RANDOM_STATE):
    X_test = np.asarray(X_test)
    y_test = np.asarray(y_test).astype(int).ravel()
    idx0, idx1 = np.where(y_test==0)[0], np.where(y_test==1)[0]
    if len(idx0)==0 or len(idx1)==0:
        return (X_test, y_test, (None if X_stat is None else np.asarray(X_stat)))
    m = min(len(idx0), len(idx1))
    rs = np.random.RandomState(random_state)
    keep = np.concatenate([rs.choice(idx0, m, replace=False), rs.choice(idx1, m, replace=False)])
    rs.shuffle(keep)
    Xb, yb = X_test[keep], y_test[keep]
    Xsb = (None if X_stat is None else np.asarray(X_stat)[keep])
    return Xb, yb, Xsb

# ======================================================
# ---------- Helper: tiny LSTM builder for quick run ----------
# ======================================================
def _make_quick_model(seq_len, n_seq_f, n_stat_f=0, lr=1e-3):
    seq_in = Input(shape=(seq_len, n_seq_f), name="seq_in")
    x = LSTM(64, return_sequences=True)(seq_in)
    x = Dropout(0.2)(x)
    x = LSTM(32)(x)
    x = Dropout(0.2)(x)
    x = Dense(16, activation="relu")(x)

    if n_stat_f and n_stat_f > 0 and USE_STATIC_INPUT:
        stat_in = Input(shape=(n_stat_f,), name="stat_in")
        s = Dense(16, activation="relu")(stat_in)
        s = Dropout(0.2)(s)
        h = Concatenate()([x, s])
        h = Dense(16, activation="relu")(h)
        out = Dense(1, activation="sigmoid")(h)
        model = Model(inputs=[seq_in, stat_in], outputs=out)
    else:
        h = Dense(16, activation="relu")(x)
        out = Dense(1, activation="sigmoid")(h)
        model = Model(inputs=seq_in, outputs=out)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  loss="binary_crossentropy", metrics=["accuracy"])
    return model

# ======================================================
# ---------- Helper: load sequences from NPZ ----------
# ======================================================
def _load_sequences_npz(npz_path):
    d = np.load(npz_path, allow_pickle=True)
    def _arr(name):
        return None if name not in d or d[name].size==0 else d[name]
    data = {
        "train": {"X_seq": _arr("Xtr"), "X_stat": _arr("Xtr_stat"), "y": d["ytr"]},
        "val":   {"X_seq": _arr("Xva"), "X_stat": _arr("Xva_stat"), "y": d["yva"]},
        "test":  {"X_seq": _arr("Xte"), "X_stat": _arr("Xte_stat"), "y": d["yte"]},
        "seq_features_used": list(d["seq_features_used"]),
        "static_features_used": list(d["static_features_used"]) if "static_features_used" in d else []
    }
    return data

# ======================================================
# ---------- Helper: find files & build splits ----------
# ======================================================
def _first_existing(paths):
    for p in paths:
        if p and os.path.exists(p):
            return p
    return None

def _splits_from_hourly(hourly: pd.DataFrame):
    train_p = np.array(sorted(hourly.loc[hourly["Split"].str.lower()=="train","patientID"].unique()))
    val_p   = np.array(sorted(hourly.loc[hourly["Split"].str.lower()=="val","patientID"].unique()))
    test_p  = np.array(sorted(hourly.loc[hourly["Split"].str.lower()=="test","patientID"].unique()))
    if len(train_p)==0 or len(val_p)==0 or len(test_p)==0:
        raise RuntimeError("hourly must include a 'Split' column with 'train'/'val'/'test' assignments.")
    return (train_p, val_p, test_p)

# ======================================================
# --------------------- RUNNER -------------------------
# ======================================================
def run_analyses_all(model=None, data=None, hourly=None,
                     split="test", out_dir=None, seq_len_default=24,
                     do_shap=True, train_epochs=6):
    """
    If model/data/hourly are not supplied, this runner tries to:
    - load sequences from NPZ (sequences_leakfree.npz)
    - load hourly CSV (dynamic_hourly_features_ramadan.csv)
    - fit a small LSTM quickly
    Then it computes:
    - SHAP CSVs (if 'shap' is installed and do_shap=True)
    - Daily risk CSV + a small PNG
    """
    # --- output dir
    if out_dir is None:
        if os.path.exists("/kaggle/working"): out_dir = "/kaggle/working"
        else:
            out_dir = os.path.join(".", "outputs")
            os.makedirs(out_dir, exist_ok=True)

    # --- locate files if needed
    if hourly is None:
        HOURLY_CANDIDATES = [
            "/kaggle/working/dynamic_hourly_features_ramadan.csv",
            "/mnt/data/dynamic_hourly_features_ramadan.csv",
            "/kaggle/input/hmc-model-static-variables/dynamic_hourly_features_ramadan.csv"
        ]
        hp = _first_existing(HOURLY_CANDIDATES)
        if not hp:
            raise FileNotFoundError("Could not find hourly CSV. Please pass 'hourly' DataFrame.")
        hourly = pd.read_csv(hp)
        print(f"📄 Loaded hourly table: {hp} | shape={hourly.shape}")

    if data is None:
        NPZ_CANDIDATES = [
            "/kaggle/working/sequences_leakfree.npz",
            "/kaggle/working/outputs/sequences_leakfree.npz",
            "/mnt/data/sequences_leakfree.npz"
        ]
        npz = _first_existing(NPZ_CANDIDATES)
        if npz:
            data = _load_sequences_npz(npz)
            print(f"📦 Loaded sequences NPZ: {npz}")
        else:
            # rebuild sequences from hourly
            splits = _splits_from_hourly(hourly)
            data = build_sequences_by_split(hourly, splits, seq_len=seq_len_default,
                                            seq_feature_cols=DEFAULT_SEQ_FEATURE_COLS,
                                            static_cols=STATIC_COLS, scale_features=True)

    # --- fit quick model if none provided
    if model is None:
        Xtr, ytr = data["train"]["X_seq"], data["train"]["y"]
        Xva, yva = data["val"]["X_seq"],   data["val"]["y"]
        Xtr_stat = data["train"]["X_stat"]; Xva_stat = data["val"]["X_stat"]
        seq_len  = Xtr.shape[1]; n_seq_f = Xtr.shape[2]
        n_stat_f = 0 if (Xtr_stat is None or not USE_STATIC_INPUT) else Xtr_stat.shape[1]

        model = _make_quick_model(seq_len, n_seq_f, n_stat_f=n_stat_f, lr=1e-3)
        es = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True, verbose=1)

        if n_stat_f > 0 and USE_STATIC_INPUT:
            model.fit([Xtr, Xtr_stat], ytr, validation_data=([Xva, Xva_stat], yva),
                      epochs=train_epochs, batch_size=64, callbacks=[es], verbose=1)
        else:
            model.fit(Xtr, ytr, validation_data=(Xva, yva),
                      epochs=train_epochs, batch_size=64, callbacks=[es], verbose=1)
        print("✅ Model trained (quick fit).")

    # --- SHAP (optional)
    if do_shap:
        try:
            _ = compute_visit_shap(model, data, hourly, data["seq_features_used"],
                                   visit_features=VISIT_FEATURES, split=split, out_dir=out_dir)
        except ImportError as e:
            print(f"⚠️ SHAP not available; skipping. ({e})")
        except Exception as e:
            print(f"⚠️ SHAP step skipped due to error: {e}")

    # --- Daily risk aggregation
    pred_df, daily = aggregate_hourly_to_daily_risk(model, data, hourly,
                                                    split=split, threshold=RISK_THRESHOLD,
                                                    out_dir=out_dir)
    # Print small previews so you "see output"
    with pd.option_context("display.width", 160, "display.max_columns", 20):
        print("\n🔎 Predictions (head):")
        print(pred_df.head(8).to_string(index=False))
        print("\n📊 Daily risk (head):")
        print(daily.head(8).to_string(index=False))

    print("\n🎯 Done. Outputs saved under:", out_dir)
    return model, data, hourly

# ======================================================
# Example: run automatically if this cell/file is executed
# (You can comment this out if you already have model/data/hourly in memory.)
# ======================================================
if __name__ == "__main__":
    _ = run_analyses_all(
        model=None,          # set to your trained model object to skip quick fit
        data=None,           # set to your 'data' dict to reuse existing arrays
        hourly=None,         # set to your hourly DataFrame if already loaded
        split="test",
        out_dir=None,        # default (/kaggle/working or ./outputs)
        seq_len_default=24,  # used only if rebuilding sequences from hourly
        do_shap=True,        # requires 'pip install shap'
        train_epochs=6       # small quick fit; adjust as you like
    )


# forecasts

Notes & options

Change patient: set _patient = <your id> before running.

Change K: set K_HOURS to the forecast horizon you want (e.g., 12).

Change threshold: set THRESH if you prefer the VAL‑optimized threshold you saw in your summary.

Better inputs for multi‑step: replace the “naive persistence” block with proper hour‑ahead forecasts of your sequence features (CGM, lifestyle PCs, visit vars) for stronger multi‑hour accuracy.


Notes for stronger multi‑hour forecasts

You’re currently using naive persistence for the future input features when rolling forward (i.e., you reuse the last observed hour’s feature vector for each next hour). That’s OK to get started, but for better K>6 accuracy consider:

CGM features (cgm_mean, cgm_std, …): replace persistence with a small CGM forecaster (e.g., ARIMA/Prophet/LightGBM) to generate hour‑ahead CGM summaries, then feed those into your LSTM rolling window.

Lifestyle PCs (pc1_activity_energy, …): if you have “typical daily patterns”, a daily seasonal baseline (e.g., by hour‑of‑day) can outperform pure persistence.

Visit features (carb, meals, …): these are daily; step them forward from the last known day or incorporate the next known visit day if available.

If you want, tell me:

a different patientID (from the list above),

your preferred K_HOURS, and

whether you want a more conservative (fewer alerts) or more sensitive (more alerts) threshold,

and I’ll output a tailored snippet with those values baked in

In [ ]:
# ==========================
# Forecasting & time-compare (fixed + improved)
# ==========================
import os, numpy as np, pandas as pd, matplotlib.pyplot as plt, matplotlib.dates as mdates
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score, precision_recall_curve
import tensorflow as tf

# --------- Fallbacks for global symbols if not already defined in the notebook ---------
try:
    OUT_RESULTS_CSV
except NameError:
    OUT_RESULTS_CSV = "/kaggle/working/results_summary_all.csv"

try:
    USE_STATIC_INPUT
except NameError:
    USE_STATIC_INPUT = True  # your training used static inputs

# focal_loss (redefine here so load_model(custom_objects=...) always works)
def focal_loss(gamma=2.0, alpha=0.25):
    bce = tf.keras.losses.BinaryCrossentropy(from_logits=False, reduction=tf.keras.losses.Reduction.NONE)
    eps = tf.keras.backend.epsilon()
    def loss(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, eps, 1.0 - eps)
        ce = bce(y_true, y_pred)
        p_t = y_true * y_pred + (1.0 - y_true) * (1.0 - y_pred)
        alpha_t = y_true * alpha + (1.0 - y_true) * (1.0 - alpha)
        modulating = tf.pow(1.0 - p_t, gamma)
        return alpha_t * modulating * ce
    return loss

# --------------------------
# 🔧 Parameters you can edit
# --------------------------
OVERRIDE_PATIENT_ID        = 69    # force this patient (else it auto-suggests)
K_HOURS                    = 12    # forecast horizon
AUTO_USE_VAL_OPTIMAL_THR   = True  # use best VAL threshold from your results CSV
THRESH_MANUAL              = 0.49  # fallback if the VAL-optimal isn’t found
FORECAST_METHOD            = "ema" # "ema" | "linear" | "persistence"
EMA_ALPHA                  = 0.6
LIN_STEPS                  = 6
PATIENT_SELECTION_STRATEGY = "positives"
TOP_N_PATIENTS_TO_PLOT     = 3
SAVE_DIR                   = "/kaggle/working"

# Improvement toggles
USE_HORIZON_THRESHOLDS     = True  # calibrate per-horizon thresholds on VAL (boosts recall at longer horizons)
FBETA_FOR_CAL              = 1.5   # β>1 favors recall
APPLY_NOFM                 = True  # apply N-of-M decision smoothing
NOFM_N, NOFM_M             = 2, 3  # 2 of last 3 positives -> positive

os.makedirs(SAVE_DIR, exist_ok=True)

# --- Datetime helpers: make all times tz-naive in UTC ---
def _to_naive_utc(s):
    s = pd.to_datetime(s, errors="coerce")
    # If tz-aware, convert to UTC then drop tz; if tz-naive, leave as is
    if getattr(s.dt, "tz", None) is not None:
        s = s.dt.tz_convert("UTC").dt.tz_localize(None)
    return s

# ----------------------------------------------------
# 0) Utilities: choose best checkpoint & VAL threshold
# ----------------------------------------------------
def pick_best_checkpoint(results_csv=OUT_RESULTS_CSV, ckpt_dir="checkpoints"):
    if not os.path.exists(results_csv):
        raise FileNotFoundError(f"Results CSV not found: {results_csv}")
    df = pd.read_csv(results_csv)
    dfv = df[df["Split"].astype(str).str.lower().eq("val")].copy()
    if dfv.empty:
        raise RuntimeError("No VAL rows found in results; cannot pick best model.")
    dfv = dfv.sort_values("Overall/F1_weighted", ascending=False)
    best = dfv.iloc[0]
    method = str(best["Method"]); model = str(best["Model"])
    ckpt_path = os.path.join(ckpt_dir, f"{method}__{model}.h5")
    if not os.path.exists(ckpt_path):
        files = [os.path.join(ckpt_dir, f) for f in os.listdir(ckpt_dir) if f.endswith(".h5")]
        if not files: raise FileNotFoundError("No .h5 checkpoints found in 'checkpoints'.")
        ckpt_path = files[0]
        print(f"[WARN] Expected checkpoint not found; using {ckpt_path}")
    print(f"✅ Best (VAL F1) → {method}/{model} | ckpt = {ckpt_path}")
    return ckpt_path, method, model

def get_val_optimal_threshold(results_csv, method, model):
    try:
        df = pd.read_csv(results_csv)
        dfv = df[(df["Split"].astype(str).str.lower()=="val") &
                 (df["Method"].astype(str)==method) &
                 (df["Model"].astype(str)==model)].copy()
        if dfv.empty:
            dfv = df[df["Split"].astype(str).str.lower()=="val"].copy()
        dfv = dfv.sort_values("Overall/F1_weighted", ascending=False)
        t = float(dfv.iloc[0]["Threshold"])
        if np.isfinite(t): return t
    except Exception as e:
        print(f"[WARN] Could not read VAL-optimal threshold: {e}")
    return None

# ------------------------------------------------
# 1) Align predictions to hours for a given split
# ------------------------------------------------
def predict_split_prob_df(model, data, hourly, split="test", threshold=0.50):
    Xs = data[split]["X_seq"]; Xstat = data[split]["X_stat"]
    ytrue = data[split]["y"].astype(int).ravel()
    if Xs is None or Xs.ndim != 3:
        raise ValueError(f"No sequences for split={split}.")
    yprob = model.predict([Xs, Xstat], verbose=0).ravel() if (Xstat is not None and Xstat.size>0) else model.predict(Xs, verbose=0).ravel()
    ypred = (yprob >= float(threshold)).astype(int)
    seq_len = Xs.shape[1]

    sub = (hourly[hourly["Split"].astype(str).str.lower() == split.lower()]
           .sort_values(["patientID","hour"]).reset_index())
    sub = sub.rename(columns={"index":"row_idx"})
    sub["hour"] = _to_naive_utc(sub["hour"])  # <<< tz-fix
    rows = []
    for pid, grp in sub.groupby("patientID", sort=True):
        grp = grp.sort_values("hour").reset_index(drop=True)
        for i in range(len(grp) - seq_len):
            tgt = grp.loc[i+seq_len]
            rows.append({"seq_idx":len(rows), "patientID":pid,
                         "hour":pd.to_datetime(tgt["hour"]),
                         "date":pd.to_datetime(tgt.get("date", pd.NaT))})
    idx_map = pd.DataFrame(rows)
    if len(idx_map) != len(ytrue):
        raise RuntimeError(f"Mapping length {len(idx_map)} != predictions length {len(ytrue)}.")
    out = (pd.DataFrame({
                "patientID": idx_map["patientID"].values,
                "hour":      pd.to_datetime(idx_map["hour"].values),
                "date":      pd.to_datetime(idx_map["date"].values),
                "y_true":    ytrue,
                "y_prob":    yprob,
                "y_pred":    ypred
            })
            .sort_values(["patientID","hour"]).reset_index(drop=True))
    out["hour"] = _to_naive_utc(out["hour"])  # belt-and-suspenders
    return out

# ------------------------------------------------------------
# 2) Mini feature forecaster for multi-step rolling predictions
# ------------------------------------------------------------
def _ema_next(v, alpha=0.6):
    s = v[0]
    for x in v[1:]:
        s = alpha*x + (1-alpha)*s
    return float(s)

def _linear_next(v):
    n = len(v)
    if n < 2: return float(v[-1])
    x = np.arange(n, dtype=float)
    try:
        b1, b0 = np.polyfit(x, v.astype(float), 1)  # y = b1*x + b0
        return float(b1*(n) + b0)
    except Exception:
        return float(v[-1])

def next_feature_vector(hist_raw, feat_names, method="ema", ema_alpha=0.6, lin_steps=6):
    T, F = hist_raw.shape
    out = np.zeros(F, dtype=float)
    for j, name in enumerate(feat_names):
        col = hist_raw[:, j]
        if method == "persistence":
            out[j] = float(col[-1])
        elif method == "linear":
            w = min(len(col), max(2, lin_steps))
            out[j] = _linear_next(col[-w:])
        else:  # "ema" default for dynamic signals
            if any(k in str(name).lower() for k in ["cgm","pca","pc1","pc2","pc3","steps","calories","heart"]):
                w = min(len(col), max(2, lin_steps))
                out[j] = _ema_next(col[-w:], alpha=ema_alpha)
            else:
                out[j] = float(col[-1])
    return out

# -----------------------------------------------------------------
# 3) Prepare one anchor window & rolling multi-step patient forecast
# -----------------------------------------------------------------
def _prepare_window_for_patient_index(hourly, data, patient_id, idx, split="test"):
    seq_feats = list(data["seq_features_used"])
    seq_len   = int(data["train"]["X_seq"].shape[1])
    sub = hourly[(hourly["Split"].astype(str).str.lower()==split.lower()) &
                 (hourly["patientID"]==patient_id)].sort_values("hour").reset_index(drop=True)
    sub["hour"] = _to_naive_utc(sub["hour"])  # <<< tz-fix
    if idx < seq_len: raise ValueError("idx must be >= seq_len")
    hist_raw   = sub.loc[idx-seq_len:idx-1, seq_feats].astype(float).values
    seq_scaler = data["scalers"]["seq"]
    hist_scaled= seq_scaler.transform(hist_raw) if seq_scaler is not None else hist_raw
    # static
    if USE_STATIC_INPUT and data["train"]["X_stat"] is not None:
        stat_feats = list(data.get("static_features_used", []))
        srow = (hourly[["patientID"]+stat_feats].drop_duplicates(subset=["patientID"]).set_index("patientID"))
        s = (srow.loc[patient_id].astype(float).values if patient_id in srow.index else np.zeros(len(stat_feats), dtype=float))
        stat_scaler = data["scalers"]["stat"]
        if stat_scaler is not None and s.size>0:
            s = stat_scaler.transform(s.reshape(1,-1)).ravel()
    else:
        s = None
    last_hour  = pd.to_datetime(sub.loc[idx-1, "hour"])
    return hist_scaled, s, hist_raw, last_hour, sub

def rolling_forecast_patient(model, data, hourly, patient_id, k=6, split="test", threshold=0.50,
                             method=FORECAST_METHOD, ema_alpha=EMA_ALPHA, lin_steps=LIN_STEPS):
    seq_feats   = list(data["seq_features_used"])
    seq_len     = int(data["train"]["X_seq"].shape[1])
    seq_scaler  = data["scalers"]["seq"]

    sub = hourly[(hourly["Split"].astype(str).str.lower()==split.lower()) &
                 (hourly["patientID"]==patient_id)].sort_values("hour").reset_index(drop=True)
    sub["hour"] = _to_naive_utc(sub["hour"])  # <<< tz-fix
    rows = []
    for anchor_idx in range(seq_len, len(sub)-1):  # each anchor predicts +1..+k
        last_needed = anchor_idx + k
        if last_needed >= len(sub): break
        Xwin_scaled, svec, raw_hist, _, _ = _prepare_window_for_patient_index(hourly, data, patient_id, anchor_idx, split=split)
        cur_scaled = Xwin_scaled.copy()
        cur_raw    = raw_hist.copy()
        F = cur_scaled.shape[1]
        for h in range(1, k+1):
            xin = cur_scaled.reshape(1, seq_len, F)
            prob = (model.predict([xin, svec.reshape(1,-1)], verbose=0).ravel()[0]
                    if (svec is not None and USE_STATIC_INPUT) else
                    model.predict(xin, verbose=0).ravel()[0])
            tgt_idx  = anchor_idx + h
            tgt_hour = pd.to_datetime(sub.loc[tgt_idx, "hour"])
            y_true   = int(sub.loc[tgt_idx, "hypo_label"])
            # horizon-specific threshold support
            thr_h = (threshold.get(h, float(threshold.get(1, 0.50)))
                     if isinstance(threshold, dict) else float(threshold))
            rows.append({
                "patientID": patient_id,
                "anchor_hour": pd.to_datetime(sub.loc[anchor_idx, "hour"]),
                "forecast_hour": tgt_hour,
                "horizon": h,
                "y_prob": float(prob),
                "y_pred": int(prob >= thr_h),
                "y_true": y_true
            })
            # roll features using mini forecaster
            next_raw    = next_feature_vector(cur_raw, seq_feats, method=method, ema_alpha=ema_alpha, lin_steps=lin_steps)
            next_scaled = seq_scaler.transform(next_raw.reshape(1,-1)).ravel() if seq_scaler is not None else next_raw
            cur_scaled  = np.vstack([cur_scaled[1:], next_scaled])
            cur_raw     = np.vstack([cur_raw[1:], next_raw])
    out = (pd.DataFrame(rows).sort_values(["forecast_hour","horizon"]).reset_index(drop=True))
    out["forecast_hour"] = _to_naive_utc(out["forecast_hour"])  # <<< tz-fix
    out["anchor_hour"]   = _to_naive_utc(out["anchor_hour"])
    return out

# -----------------------------------------
# 4) Metrics by horizon + patient suggestion
# -----------------------------------------
def metrics_by_horizon(df_forecast):
    out = []
    for h, g in df_forecast.groupby("horizon"):
        y = g["y_true"].astype(int).values
        p = g["y_prob"].astype(float).values
        yhat = g["y_pred"].astype(int).values
        try: roc = roc_auc_score(y, p)
        except: roc = np.nan
        try: pr  = average_precision_score(y, p)
        except: pr  = np.nan
        out.append({
            "horizon": h, "n": len(g),
            "Accuracy": accuracy_score(y, yhat),
            "Precision": precision_score(y, yhat, zero_division=0),
            "Recall": recall_score(y, yhat, zero_division=0),
            "F1": f1_score(y, yhat, zero_division=0),
            "ROC_AUC": roc, "PR_AUC": pr
        })
    return pd.DataFrame(out).sort_values("horizon")

def metrics_by_horizon_on_col(df_forecast, col="y_pred"):
    rows = []
    for h, g in df_forecast.groupby("horizon"):
        y = g["y_true"].astype(int).values
        yhat = g[col].astype(int).values
        try: roc = roc_auc_score(y, g["y_prob"].astype(float).values)
        except: roc = np.nan
        try: pr  = average_precision_score(y, g["y_prob"].astype(float).values)
        except: pr  = np.nan
        rows.append({
            "horizon": h, "n": len(g),
            "Accuracy": accuracy_score(y, yhat),
            "Precision": precision_score(y, yhat, zero_division=0),
            "Recall": recall_score(y, yhat, zero_division=0),
            "F1": f1_score(y, yhat, zero_division=0),
            "ROC_AUC": roc, "PR_AUC": pr,
            "decision": col
        })
    return pd.DataFrame(rows).sort_values("horizon")

def suggest_patients_for_plots(df_nowcast, strategy="positives", top_n=3):
    g = df_nowcast.groupby("patientID", as_index=False)
    if strategy == "coverage":
        s = g.size().rename(columns={"size":"n"}).sort_values("n", ascending=False)
        ids = list(s["patientID"].head(top_n))
    elif strategy == "auc":
        rows = []
        for pid, grp in df_nowcast.groupby("patientID"):
            y, p = grp["y_true"].values, grp["y_prob"].values
            try: pr = average_precision_score(y, p)
            except: pr = np.nan
            rows.append({"patientID":pid, "PR_AUC":pr, "n":len(grp)})
        s = (pd.DataFrame(rows).dropna(subset=["PR_AUC"])
             .sort_values(["PR_AUC","n"], ascending=[False, False]))
        ids = list(s["patientID"].head(top_n)) if not s.empty else list(df_nowcast["patientID"].value_counts().head(top_n).index)
    else:  # "positives" default
        pos = g["y_true"].sum().rename(columns={"y_true":"positives"}) \
               .sort_values("positives", ascending=False)
        ids = list(pos["patientID"].head(top_n))
        if pos["positives"].max() <= 0:
            ids = list(df_nowcast["patientID"].value_counts().head(top_n).index)
    return ids

# -----------------------
# 5) Post-processing (N-of-M)
# -----------------------
def apply_n_of_m_rule(df, n=2, m=3):
    """df must be one patient; columns: forecast_hour, y_pred."""
    df = df.sort_values("forecast_hour").copy()
    flags = df["y_pred"].astype(int).values
    out = np.zeros_like(flags)
    for i in range(len(flags)):
        win = flags[max(0, i-m+1):i+1]
        out[i] = 1 if win.sum() >= n else 0
    df["y_pred_nofm"] = out
    return df

# -----------------------
# 6) Plotting functions
# -----------------------
def plot_nowcast_and_forecast_timeline(df_nowcast, df_forecast, patient_id,
                                       hours_back=72, out_png=None, threshold=0.50):
    past = df_nowcast[df_nowcast["patientID"]==patient_id].sort_values("hour").copy()
    past["hour"] = _to_naive_utc(past["hour"])  # <<< tz-fix
    if not past.empty and hours_back and hours_back>0:
        cutoff = past["hour"].max() - pd.Timedelta(hours=hours_back)
        past = past[past["hour"] >= cutoff]
    if df_forecast.empty: raise ValueError("df_forecast is empty.")
    last_anchor = df_forecast[df_forecast["patientID"]==patient_id]["anchor_hour"].max()
    fut = (df_forecast[(df_forecast["patientID"]==patient_id) &
                       (df_forecast["anchor_hour"]==last_anchor)]
           .sort_values("forecast_hour").copy())
    fut["forecast_hour"] = _to_naive_utc(fut["forecast_hour"])  # <<< tz-fix

    fig, ax = plt.subplots(figsize=(10,4))
    if not past.empty:
        ax.plot(past["hour"], past["y_prob"], lw=2, label="Nowcast (test) prob")
        ax.step(past["hour"], past["y_true"], where="post", alpha=0.35, label="True label (past)")
    ax.plot(fut["forecast_hour"], fut["y_prob"], lw=2, marker="o",
            label=f"Forecast next {int(fut['horizon'].max())}h prob")
    if "y_true" in fut.columns and fut["y_true"].notna().any():
        ax.step(fut["forecast_hour"], fut["y_true"], where="post", alpha=0.35, label="True label (future)")
    ax.axhline(float(threshold) if not isinstance(threshold, dict) else float(threshold.get(1, 0.50)),
               ls="--", label="Decision threshold")
    ax.set_ylim(-0.05, 1.05)
    ax.set_xlabel("Hour"); ax.set_ylabel("Risk / Label")
    ax.set_title(f"Patient {patient_id} — past nowcast + future forecast")
    ax.legend(loc="upper left")
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%m-%d %H:%M"))
    fig.autofmt_xdate(); fig.tight_layout()
    if out_png:
        fig.savefig(out_png, dpi=200); print(f"🖼️ Saved timeline → {out_png}")
    plt.show()

def plot_nowcast_vs_forecast_h1(df_nowcast, df_forecast, patient_id, out_png=None):
    f1 = df_forecast[(df_forecast["patientID"]==patient_id) & (df_forecast["horizon"]==1)].copy()
    f1 = f1.rename(columns={"forecast_hour":"hour", "y_prob":"y_prob_forecast_h1"})
    past = df_nowcast[df_nowcast["patientID"]==patient_id][["hour","y_prob","y_true"]].copy()

    # --- force both keys to the same dtype/timezone ---
    past["hour"] = _to_naive_utc(past["hour"])
    f1["hour"]   = _to_naive_utc(f1["hour"])

    j = past.merge(f1[["hour","y_prob_forecast_h1"]], on="hour", how="inner").sort_values("hour")
    if j.empty:
        print("[INFO] No overlap between nowcast timeline and horizon-1 forecasts for this patient."); return
    fig, ax = plt.subplots(figsize=(10,4))
    ax.plot(j["hour"], j["y_prob"], lw=2, label="Nowcast prob (test)")
    ax.plot(j["hour"], j["y_prob_forecast_h1"], lw=2, linestyle="--", label="Forecast@+1h prob")
    ax.step(j["hour"], j["y_true"], where="post", alpha=0.35, label="True label")
    ax.set_ylim(-0.05, 1.05)
    ax.set_xlabel("Hour"); ax.set_ylabel("Risk / Label")
    ax.set_title(f"Patient {patient_id} — nowcast vs forecast@+1h")
    ax.legend(loc="upper left")
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%m-%d %H:%M"))
    fig.autofmt_xdate(); fig.tight_layout()
    if out_png:
        fig.savefig(out_png, dpi=200); print(f"🖼️ Saved compare@+1h → {out_png}")
    plt.show()

# -------------------------------------------------
# 7) Threshold calibration on VAL (per horizon)
# -------------------------------------------------
def _best_thr_by_fbeta(y, p, beta=1.0, thr_min=0.10, thr_max=0.70):
    prec, rec, thr = precision_recall_curve(y, p)
    prec, rec, thr = prec[:-1], rec[:-1], thr
    mask = np.isfinite(thr) & (thr >= thr_min) & (thr <= thr_max)
    if not mask.any():
        return 0.50
    fb = (1+beta**2) * (prec[mask]*rec[mask]) / (beta**2*prec[mask] + rec[mask] + 1e-9)
    return float(thr[mask][np.nanargmax(fb)])

def calibrate_horizon_thresholds_on_val(model, data, hourly, k=12, beta=1.5, method="ema"):
    vals = []
    val_mask = hourly["Split"].astype(str).str.lower()=="val"
    val_ids = sorted(hourly.loc[val_mask, "patientID"].unique())
    for pid in val_ids:
        df_fc_val = rolling_forecast_patient(
            model, data, hourly, patient_id=pid, k=k, split="val",
            threshold=0.50, method=method  # temporary; only need probs
        )
        if not df_fc_val.empty:
            vals.append(df_fc_val)
    if not vals:
        return {}
    allv = pd.concat(vals, ignore_index=True)
    out = {}
    for h, g in allv.groupby("horizon"):
        y = g["y_true"].astype(int).values
        p = g["y_prob"].astype(float).values
        if len(np.unique(y)) < 2:
            continue
        out[h] = _best_thr_by_fbeta(y, p, beta=beta, thr_min=0.10, thr_max=0.70)
    return out

# ======================
# 8) Run: load, forecast, plot
# ======================
# Preconditions
if "hourly" not in globals() or "data" not in globals():
    raise RuntimeError("hourly/data not found. Please run your feature builder and sequence builder cells first.")

CKPT_PATH, _best_method, _best_model = pick_best_checkpoint(OUT_RESULTS_CSV, "checkpoints")
model = tf.keras.models.load_model(CKPT_PATH, custom_objects={"loss": focal_loss, "focal_loss": focal_loss})

# (b) Threshold: VAL-optimal or manual
if AUTO_USE_VAL_OPTIMAL_THR:
    thr_val = get_val_optimal_threshold(OUT_RESULTS_CSV, _best_method, _best_model)
    THRESH   = float(thr_val) if (thr_val is not None) else float(THRESH_MANUAL)
    if thr_val is None:
        print(f"[INFO] Using manual THRESH={THRESH_MANUAL:.2f} (VAL-optimal not found).")
    else:
        print(f"✅ Using VAL-optimal THRESH={THRESH:.2f}")
else:
    THRESH = float(THRESH_MANUAL)
    print(f"ℹ️ Using manual THRESH={THRESH:.2f}")

# (c) Nowcast on TEST + suggestions
df_test_nowcast = predict_split_prob_df(model, data, hourly, split="test", threshold=THRESH)
if OVERRIDE_PATIENT_ID is not None and OVERRIDE_PATIENT_ID in set(df_test_nowcast["patientID"].unique()):
    suggested = [OVERRIDE_PATIENT_ID]
else:
    suggested = suggest_patients_for_plots(df_test_nowcast, strategy=PATIENT_SELECTION_STRATEGY,
                                           top_n=TOP_N_PATIENTS_TO_PLOT)
print("📌 Suggested patientID(s) to plot:", suggested)

# (d) Optional: learn per-horizon thresholds on VAL
HORIZON_THR = None
if USE_HORIZON_THRESHOLDS:
    HORIZON_THR = calibrate_horizon_thresholds_on_val(
        model, data, hourly, k=K_HOURS, beta=FBETA_FOR_CAL, method=FORECAST_METHOD
    )
    print("Horizon thresholds (VAL):", HORIZON_THR)

# (e) Forecast & plot
for pid in suggested:
    print(f"\n=== Forecasting patient {pid} | K={K_HOURS}, method={FORECAST_METHOD} ===")
    thr_to_use = (HORIZON_THR if (isinstance(HORIZON_THR, dict) and len(HORIZON_THR)>0) else THRESH)
    df_fc = rolling_forecast_patient(model, data, hourly, patient_id=pid, k=K_HOURS,
                                     split="test", threshold=thr_to_use,
                                     method=FORECAST_METHOD, ema_alpha=EMA_ALPHA, lin_steps=LIN_STEPS)
    print("Forecast rows:", len(df_fc))

    # Optional N-of-M smoothing
    if APPLY_NOFM and not df_fc.empty:
        df_fc = df_fc.groupby("patientID", group_keys=False).apply(apply_n_of_m_rule, n=NOFM_N, m=NOFM_M)

    # Metrics
    hz_base = metrics_by_horizon(df_fc)
    print("Horizon metrics (base decision):\n", hz_base)
    if APPLY_NOFM and "y_pred_nofm" in df_fc.columns:
        hz_nofm = metrics_by_horizon_on_col(df_fc, col="y_pred_nofm")
        print("Horizon metrics (N-of-M decision):\n", hz_nofm)

    # Plots
    outA = os.path.join(SAVE_DIR, f"nowcast_plus_forecast_patient_{pid}.png")
    plot_nowcast_and_forecast_timeline(df_test_nowcast, df_fc, patient_id=pid,
                                       hours_back=72, threshold=thr_to_use, out_png=outA)
    outB = os.path.join(SAVE_DIR, f"nowcast_vs_forecast_h1_patient_{pid}.png")
    plot_nowcast_vs_forecast_h1(df_test_nowcast, df_fc, patient_id=pid, out_png=outB)

print("✅ Done.")


In [ ]:
# ==============================================
# Leak-safe Ramadan features + Balanced LSTM
# (hourly builder + sequences + training utilities + significance testing)
# ==============================================
import os, time, warnings, random, re
from pathlib import Path
from collections import Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, roc_curve, precision_recall_curve,
    average_precision_score, auc, mean_squared_error
)
from scipy.stats import norm  # <-- added
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN, SMOTETomek

import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout, Concatenate
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

warnings.filterwarnings("ignore")

# --------------------
# GLOBAL CONFIG
# --------------------
CSV_INTRADAY_WITH_VISITS = "/kaggle/input/hmcdataset/intraday_with_visits.csv"
STATIC_CSV      = "/kaggle/input/hmc-model-static-variables/outcome_static.csv"
VISIT_WIDE_CSV  = "/kaggle/input/hmc-model-static-variables/outcome_visit_wide_by_variable.csv"
VISIT_LONG_CSV  = "/kaggle/input/hmc-model-static-variables/outcome_visit_long.csv"

OUT_HOURLY_CSV  = "/kaggle/working/dynamic_hourly_features_ramadan.csv"
OUT_SEQ_NPZ     = "/kaggle/working/sequences_leakfree.npz"
OUT_RESULTS_CSV = "/kaggle/working/results_summary_all.csv"
OUT_PLOTS_PNG   = "/kaggle/working/combined_roc_pr_curves.png"

RAMADAN_START = pd.to_datetime("2023-03-22")
RAMADAN_END   = pd.to_datetime("2023-04-19")
HYPO_CUTOFF   = 70.0
MIN_CGM_PER_H = 4
SEQ_LEN       = 24

LIFESTYLE_COLS_CANDIDATES = [
    "steps","distance","calories","heart_rate","spo2",
    "deep","light","rem","nap","awake"
]

VISIT_COLS = ["carb","meals","total_daily_dose_u","fasting_percent_29"]
STATIC_COLS = [
    "Age","Gender","BMI","HbA1C","Cholesterol","LDL","HDL","Triglycerides",
    "eGFR","Creatinine","Insulin_units_per_kg","SmartGuard_percent"
]

DEFAULT_SEQ_FEATURE_COLS = (
    "cgm_mean","cgm_std","pca_cgm1",
    "pc1_activity_energy",
    "carb","meals","total_daily_dose_u","fasting_percent_29"
)

RANDOM_STATE     = 42
THR_MIN, THR_MAX = 0.40, 0.60
AUGMENT_SIGMA    = 0.01
RESAMPLE_METHODS = ["none","oversample_seq","undersample_seq","smote","smoteenn","smotetomek"]
USE_STATIC_INPUT = True

# --------------------
# Utilities (robust column picking)
# --------------------
def set_global_seeds(seed: int = 42):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
set_global_seeds(RANDOM_STATE)

def to_dt(x, utc_ok=True):
    return pd.to_datetime(x, errors="coerce", utc=utc_ok)

def ensure_numeric(df, exclude=("patientID","huaweiID","visit_assigned","period_main","start","date","hour","hour_of_day")):
    ex = set(exclude)
    for c in df.columns:
        if c not in ex:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

def safe_encode_gender(series):
    if series.dtype == "object":
        return (series.str.strip().str.lower().map({"male":1, "m":1, "female":0, "f":0}))
    return pd.to_numeric(series, errors="coerce")

def split_patients(unique_pids, test_size=0.2, val_size=0.1, random_state=RANDOM_STATE):
    train_pids, test_pids = train_test_split(unique_pids, test_size=test_size, random_state=random_state)
    val_fraction = val_size / max(1e-9, (1.0 - test_size))
    train_pids, val_pids = train_test_split(train_pids, test_size=val_fraction, random_state=random_state)
    return np.array(train_pids), np.array(val_pids), np.array(test_pids)

def _normalize_date(s):
    s = pd.to_datetime(s, errors="coerce")
    return s.dt.normalize()

def _norm_col(s: str) -> str:
    return re.sub(r'[^a-z0-9]+', '', str(s).lower())

def _pick_col_flex(df: pd.DataFrame, preferred=None, required=False, name="", must_contain_all=None, any_contains=None):
    cols = list(df.columns)
    norm_map = {c: _norm_col(c) for c in cols}
    if preferred:
        lower_pref = {str(p).strip().lower(): p for p in preferred}
        for c in cols:
            if str(c).strip().lower() in lower_pref:
                return c
    if preferred:
        pref_norm = {_norm_col(p): p for p in preferred}
        for c, n in norm_map.items():
            if n in pref_norm:
                return c
    cands = []
    for c, n in norm_map.items():
        ok = True
        if must_contain_all:
            for tok in must_contain_all:
                if _norm_col(tok) not in n:
                    ok = False; break
        if ok and any_contains:
            if not any(_norm_col(tok) in n for tok in any_contains):
                ok = False
        if ok: cands.append(c)
    if cands:
        def _priority(col: str):
            n = norm_map[col]
            starts_pid = n.startswith("patientid")
            has_pid    = "patientid" in n
            return (-(starts_pid or has_pid), len(n))
        cands.sort(key=_priority)
        return cands[0]
    if required:
        raise KeyError(f"Required column not found for {name}. preferred={preferred} | must_contain_all={must_contain_all} | any_contains={any_contains}. Available: {cols}")
    return None

def _pick_patient_col(df: pd.DataFrame) -> str:
    preferred = ["patientID","patientId","PatientID (Huawei Data)","subject_id","patid","pid","id","huaweiid"]
    return _pick_col_flex(df, preferred=preferred, required=True, name="patientID",
                          must_contain_all=["id"], any_contains=["patient","subject","pat","huawei"])

def _pick_date_col(df: pd.DataFrame) -> str:
    preferred = ["date","visit_date","Date","day","timestamp","Visit Date","date_of_visit","start"]
    return _pick_col_flex(df, preferred=preferred, required=True, name="date",
                          any_contains=["date","visit","day","timestamp","start"])

def _pick_variable_col(df: pd.DataFrame) -> str:
    preferred = ["variable","var","feature","name","measure","metric"]
    return _pick_col_flex(df, preferred=preferred, required=True, name="variable",
                          any_contains=["variable","var","feature","name","measure","metric"])

def _pick_value_col(df: pd.DataFrame) -> str:
    preferred = ["value","val","measure_value","reading","amount","score"]
    return _pick_col_flex(df, preferred=preferred, required=True, name="value",
                          any_contains=["value","val","measurevalue","reading","amount","score"])

# ---------------------------
# Loaders for external files
# ---------------------------
def load_static_df(static_csv=STATIC_CSV, needed=STATIC_COLS):
    if not static_csv or not os.path.exists(static_csv):
        print("⚠️ Static CSV not found; static features will be zero-filled.")
        return None
    df = pd.read_csv(static_csv)
    pid_col = _pick_patient_col(df)
    df = df.rename(columns={pid_col:"patientID"})
    keep = ["patientID"] + [c for c in needed if c in df.columns]
    df = df[keep].drop_duplicates(subset=["patientID"]).copy()
    if "Gender" in df.columns:
        df["Gender"] = safe_encode_gender(df["Gender"])
    for c in keep:
        if c != "patientID":
            df[c] = pd.to_numeric(df[c], errors="coerce")
    print(f"ℹ️ static: using patientID column = '{pid_col}'")
    return df

def load_visit_df(visit_wide_csv=VISIT_WIDE_CSV, visit_long_csv=VISIT_LONG_CSV, needed=VISIT_COLS):
    if visit_wide_csv and os.path.exists(visit_wide_csv):
        wide = pd.read_csv(visit_wide_csv)
        pid_col  = _pick_patient_col(wide)
        date_col = _pick_date_col(wide)
        wide = wide.rename(columns={pid_col:"patientID", date_col:"date"})
        wide["date"] = _normalize_date(wide["date"])
        keep = ["patientID","date"] + [c for c in needed if c in wide.columns]
        if len(keep) > 2:
            print(f"ℹ️ visit-wide: patientID='{pid_col}', date='{date_col}', kept={keep[2:]}")
            return wide[keep].copy()
        else:
            print("⚠️ VISIT_WIDE_CSV found but none of the needed visit columns present; will try LONG if available.")
    if visit_long_csv and os.path.exists(visit_long_csv):
        long = pd.read_csv(visit_long_csv)
        pid_col   = _pick_patient_col(long)
        date_col  = _pick_date_col(long)
        var_col   = _pick_variable_col(long)
        value_col = _pick_value_col(long)
        long = long.rename(columns={pid_col:"patientID", date_col:"date", var_col:"variable", value_col:"value"})
        long["date"] = _normalize_date(long["date"])
        wide = (long
                .pivot_table(index=["patientID","date"], columns="variable", values="value", aggfunc="mean")
                .reset_index())
        keep = ["patientID","date"] + [c for c in needed if c in wide.columns]
        if len(keep) > 2:
            print(f"ℹ️ visit-long: patientID='{pid_col}', date='{date_col}', variables matched={keep[2:]}")
            return wide[keep].copy()
        print("⚠️ VISIT_LONG_CSV present but none of the needed variables were found in the pivot.")
    print("⚠️ No usable visit CSVs found; visit features will be zero-filled.")
    return None

# ----------------------------------------------------------------
# Part A — Build hourly Ramadan features and leak-safe transforms
# ----------------------------------------------------------------
def build_hourly_features_with_leak_safe_transforms(
    in_csv=CSV_INTRADAY_WITH_VISITS,
    out_csv=OUT_HOURLY_CSV,
    min_cgm_per_hour=MIN_CGM_PER_H,
    test_size=0.2, val_size=0.1, random_state=RANDOM_STATE,
    static_csv=STATIC_CSV, visit_wide_csv=VISIT_WIDE_CSV, visit_long_csv=VISIT_LONG_CSV
):
    if not os.path.exists(in_csv):
        raise FileNotFoundError(f"Input not found: {in_csv}")
    df = pd.read_csv(in_csv)
    if "patientID" not in df.columns:
        pid_col = _pick_patient_col(df)
        df = df.rename(columns={pid_col: "patientID"})
        print(f"ℹ️ intraday: using patientID column = '{pid_col}'")

    start_col = "start" if "start" in df.columns else _pick_date_col(df)
    df[start_col] = to_dt(df[start_col])
    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"], errors="coerce")
    else:
        df["date"] = pd.to_datetime(df[start_col].dt.date)
    df["hour"]       = df[start_col].dt.floor("h")
    df["hour_of_day"]= df["hour"].dt.hour

    df = ensure_numeric(df)
    df = df[(df["date"] >= RAMADAN_START) & (df["date"] <= RAMADAN_END)].copy()

    if "cgm" not in df.columns:
        raise ValueError("❌ Dataset must include 'cgm' column.")
    df_cgm = df.dropna(subset=["cgm"]).copy()

    valid_hours = (
        df_cgm.groupby(["patientID","hour"])
              .filter(lambda g: g["cgm"].notna().sum() >= min_cgm_per_hour)
    )

    hourly = (
        valid_hours.groupby(["patientID","hour"], as_index=False)
                   .agg(
                       cgm_min=("cgm","min"),
                       cgm_max=("cgm","max"),
                       cgm_mean=("cgm","mean"),
                       cgm_std=("cgm","std")
                   )
                   .sort_values(["patientID","hour"])
                   .reset_index(drop=True)
    )
    hourly["hour_of_day"] = hourly["hour"].dt.hour

    lab = (
        valid_hours.groupby(["patientID","hour"])["cgm"]
                   .apply(lambda x: int((x < HYPO_CUTOFF).any()))
                   .reset_index(name="hypo_label")
    )
    hourly = hourly.merge(lab, on=["patientID","hour"], how="left")

    lifestyle_cols = [c for c in LIFESTYLE_COLS_CANDIDATES if c in df_cgm.columns]
    if lifestyle_cols:
        life_hourly = (
            df_cgm.groupby(["patientID","hour"], as_index=False)[lifestyle_cols]
                  .mean().fillna(0.0)
        )
        hourly = hourly.merge(life_hourly, on=["patientID","hour"], how="left")

    hourly["cgm_mean_plus_std"]  = hourly["cgm_mean"] + hourly["cgm_std"]
    hourly["cgm_mean_minus_std"] = hourly["cgm_mean"] - hourly["cgm_std"]

    pids = hourly["patientID"].dropna().unique()
    train_p, val_p, test_p = split_patients(pids, test_size=test_size, val_size=val_size, random_state=random_state)
    hourly["Split"] = np.where(hourly["patientID"].isin(train_p), "train",
                        np.where(hourly["patientID"].isin(val_p), "val", "test"))

    cgm_cols = ["cgm_min","cgm_max","cgm_mean","cgm_std"]
    tr_mask  = hourly["Split"] == "train"
    scal_cgm = StandardScaler().fit(hourly.loc[tr_mask, cgm_cols].fillna(0.0))
    pca_cgm  = PCA(n_components=3, random_state=random_state).fit(
        scal_cgm.transform(hourly.loc[tr_mask, cgm_cols].fillna(0.0))
    )
    def _apply_cgm_pca(df_in):
        X = scal_cgm.transform(df_in[cgm_cols].fillna(0.0))
        Z = pca_cgm.transform(X)
        out = df_in.copy()
        out["pca_cgm1"], out["pca_cgm2"], out["pca_cgm3"] = Z[:,0], Z[:,1], Z[:,2]
        return out
    hourly = _apply_cgm_pca(hourly)

    if lifestyle_cols:
        scal_life = StandardScaler().fit(hourly.loc[tr_mask, lifestyle_cols].fillna(0.0))
        pca_life  = PCA(n_components=3, random_state=random_state).fit(
            scal_life.transform(hourly.loc[tr_mask, lifestyle_cols].fillna(0.0))
        )
        X_all = scal_life.transform(hourly[lifestyle_cols].fillna(0.0))
        Z_all = pca_life.transform(X_all)
        hourly["pc1_activity_energy"] = Z_all[:,0]
        hourly["pc2_physiology"]      = Z_all[:,1]
        hourly["pc3_sleep_rest"]      = Z_all[:,2]
    else:
        hourly["pc1_activity_energy"] = 0.0
        hourly["pc2_physiology"]      = 0.0
        hourly["pc3_sleep_rest"]      = 0.0

    visit_df = load_visit_df(visit_wide_csv, visit_long_csv, VISIT_COLS)
    hourly["date"] = hourly["hour"].dt.normalize()
    if visit_df is not None:
        visit_df["date"] = pd.to_datetime(visit_df["date"], errors="coerce").dt.normalize()
        visit_df = visit_df[(visit_df["date"] >= RAMADAN_START) & (visit_df["date"] <= RAMADAN_END)].copy()
        hourly = hourly.merge(visit_df, on=["patientID","date"], how="left")
    for c in VISIT_COLS:
        if c not in hourly.columns:
            hourly[c] = 0.0
        hourly[c] = pd.to_numeric(hourly[c], errors="coerce").fillna(0.0)

    static_df = load_static_df(static_csv, STATIC_COLS)
    if static_df is not None:
        hourly = hourly.merge(static_df, on="patientID", how="left")
    for c in STATIC_COLS:
        if c not in hourly.columns:
            hourly[c] = 0.0
        hourly[c] = pd.to_numeric(cast := hourly[c], errors="coerce").fillna(0.0)

    hourly = hourly.sort_values(["patientID","hour"]).reset_index(drop=True)
    hourly.to_csv(out_csv, index=False)
    print(f"✅ Saved hourly features (leak-safe) → {out_csv}")

    return hourly, (train_p, val_p, test_p)

# ---------------------------------------------------------------
# Part B — Build sequences (optionally with per-patient static)
# ---------------------------------------------------------------
def build_sequences_by_split(hourly, splits, seq_len=SEQ_LEN, seq_feature_cols=DEFAULT_SEQ_FEATURE_COLS,
                             static_cols=STATIC_COLS, scale_features=True):
    for c in ["patientID","hour","hypo_label","Split"]:
        if c not in hourly.columns:
            raise KeyError(f"hourly missing required column: {c}")
    hourly["hour"] = pd.to_datetime(hourly["hour"], errors="coerce")

    seq_feature_cols = list(seq_feature_cols)
    missing_seq = [c for c in seq_feature_cols if c not in hourly.columns]
    if missing_seq:
        raise KeyError(f"Sequence feature(s) not found in hourly: {missing_seq}")

    static_cols_present = [c for c in static_cols if c in hourly.columns]
    if static_cols_present and USE_STATIC_INPUT:
        static_mat = (hourly[["patientID"] + static_cols_present]
                      .drop_duplicates(subset=["patientID"])
                      .set_index("patientID")
                      .astype(float).fillna(0.0))
    else:
        static_mat = None
        static_cols_present = []

    train_p, val_p, test_p = splits

    def _build_for_pidset(pid_set):
        sub = hourly[hourly["patientID"].isin(pid_set)].copy()
        X_seq, X_stat, y = [], [], []
        for pid, grp in sub.groupby("patientID"):
            grp = grp.sort_values("hour").reset_index(drop=True)
            if len(grp) <= seq_len:
                continue
            feats  = grp[seq_feature_cols].astype(float).values
            labels = grp["hypo_label"].astype(int).values
            for i in range(len(grp) - seq_len):
                X_seq.append(feats[i:i+seq_len]); y.append(labels[i+seq_len])
                if static_mat is not None and pid in static_mat.index:
                    X_stat.append(static_mat.loc[pid].values.astype(float))
        X_seq = np.array(X_seq); y = np.array(y).astype(int)
        X_stat = np.array(X_stat) if (static_mat is not None and len(X_stat)>0) else None
        return X_seq, X_stat, y

    Xtr_s, Xtr_stat, ytr = _build_for_pidset(train_p)
    Xva_s, Xva_stat, yva = _build_for_pidset(val_p)
    Xte_s, Xte_stat, yte = _build_for_pidset(test_p)

    seq_scaler  = None
    stat_scaler = None
    if scale_features and Xtr_s.size > 0:
        n_f = Xtr_s.shape[2]
        seq_scaler = StandardScaler().fit(Xtr_s.reshape(-1, n_f))
        def _scale_seq(X):
            if X is None or X.size==0: return X
            n = X.shape[0]; return seq_scaler.transform(X.reshape(-1, n_f)).reshape(n, SEQ_LEN, n_f)
        Xtr_s = _scale_seq(Xtr_s); Xva_s = _scale_seq(Xva_s); Xte_s = _scale_seq(Xte_s)

    if scale_features and Xtr_stat is not None and Xtr_stat.size>0:
        stat_scaler = StandardScaler().fit(Xtr_stat)
        def _scale_stat(X):
            if X is None or X.size==0: return X
            return stat_scaler.transform(X)
        Xtr_stat = _scale_stat(Xtr_stat); Xva_stat = _scale_stat(Xva_stat); Xte_stat = _scale_stat(Xte_stat)

    print(f"✅ Sequences built | train={Xtr_s.shape}, val={Xva_s.shape}, test={Xte_s.shape}")
    return {
        "train": {"X_seq": Xtr_s, "X_stat": Xtr_stat, "y": ytr},
        "val":   {"X_seq": Xva_s, "X_stat": Xva_stat, "y": yva},
        "test":  {"X_seq": Xte_s, "X_stat": Xte_stat, "y": yte},
        "seq_features_used": seq_feature_cols,
        "static_features_used": static_cols_present,
        "scalers": {"seq": seq_scaler, "stat": stat_scaler}
    }

# ------------------------------------------------------
# Balanced LSTM pipeline utilities (metrics, resampling)
# ------------------------------------------------------
THR_MIN, THR_MAX = THR_MIN, THR_MAX

def _best_threshold_in_range(thresholds, scores, thr_min=THR_MIN, thr_max=THR_MAX):
    thresholds = np.asarray(thresholds, dtype=float)
    scores     = np.asarray(scores, dtype=float)
    mask = np.isfinite(thresholds) & (thresholds >= thr_min) & (thresholds <= thr_max)
    if mask.any():
        idx_in = int(np.nanargmax(scores[mask])); idx = np.where(mask)[0][idx_in]
        return float(thresholds[idx]), True
    idx = int(np.nanargmax(scores))
    return float(np.clip(thresholds[idx], thr_min, thr_max)), False

def focal_loss(gamma=2.0, alpha=0.25):
    bce = tf.keras.losses.BinaryCrossentropy(from_logits=False, reduction=tf.keras.losses.Reduction.NONE)
    eps = tf.keras.backend.epsilon()
    def loss(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, eps, 1.0 - eps)
        ce = bce(y_true, y_pred)
        p_t = y_true * y_pred + (1.0 - y_true) * (1.0 - y_pred)
        alpha_t = y_true * alpha + (1.0 - y_true) * (1.0 - alpha)
        modulating = tf.pow(1.0 - p_t, gamma)
        return alpha_t * modulating * ce
    return loss

def _safe_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred, labels=[0,1])
    if cm.shape != (2,2):
        full = np.zeros((2,2), dtype=int)
        full[:cm.shape[0], :cm.shape[1]] = cm
        cm = full
    return cm

def _specificity_overall(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    return tn / (tn + fp + 1e-8)

def _specificity_per_class(y_true, y_pred, positive_label):
    y_true_bin = (np.asarray(y_true).ravel() == positive_label).astype(int)
    y_pred_bin = (np.asarray(y_pred).ravel() == positive_label).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true_bin, y_pred_bin, labels=[0,1]).ravel()
    return tn / (tn + fp + 1e-8)

def evaluate_full_metrics(y_true, y_pred, y_prob=None):
    y_true = np.asarray(y_true).astype(int).ravel()
    y_pred = np.asarray(y_pred).astype(int).ravel()
    cm = _safe_confusion_matrix(y_true, y_pred)

    metrics = {}
    for lbl in [0,1]:
        metrics[f"Class{lbl}/Precision"]   = precision_score(y_true, y_pred, pos_label=lbl, zero_division=0)
        metrics[f"Class{lbl}/Recall"]      = recall_score(y_true, y_pred,    pos_label=lbl, zero_division=0)
        metrics[f"Class{lbl}/F1"]          = f1_score(y_true, y_pred,        pos_label=lbl, zero_division=0)
        metrics[f"Class{lbl}/Specificity"] = _specificity_per_class(y_true, y_pred, positive_label=lbl)
        metrics[f"Class{lbl}/Support"]     = int(np.sum(y_true == lbl))

    metrics["Overall/Accuracy"]             = accuracy_score(y_true, y_pred)
    metrics["Overall/Precision_macro"]      = precision_score(y_true, y_pred, average='macro',    zero_division=0)
    metrics["Overall/Precision_weighted"]   = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    metrics["Overall/Recall_macro"]         = recall_score(y_true, y_pred,    average='macro',    zero_division=0)
    metrics["Overall/Recall_weighted"]      = recall_score(y_true, y_pred,    average='weighted', zero_division=0)
    metrics["Overall/F1_macro"]             = f1_score(y_true, y_pred,        average='macro',    zero_division=0)
    metrics["Overall/F1_weighted"]          = f1_score(y_true, y_pred,        average='weighted', zero_division=0)
    metrics["Overall/Specificity"]          = _specificity_overall(y_true, y_pred)
    mse_pred                                = mean_squared_error(y_true, y_pred)
    metrics["Overall/MSE_pred"]             = mse_pred
    metrics["Overall/RMSE_pred"]            = float(np.sqrt(mse_pred))

    if y_prob is not None:
        y_prob = np.asarray(y_prob, dtype=float).ravel()
        try:  metrics["Overall/ROC-AUC"] = roc_auc_score(y_true, y_prob)
        except ValueError: metrics["Overall/ROC-AUC"] = np.nan
        try:  metrics["Overall/PR-AUC"]  = average_precision_score(y_true, y_prob)
        except ValueError: metrics["Overall/PR-AUC"] = np.nan
        mse_prob                          = mean_squared_error(y_true, y_prob)
        metrics["Overall/MSE_prob"]       = mse_prob
        metrics["Overall/RMSE_prob"]      = float(np.sqrt(mse_prob))
    else:
        metrics["Overall/ROC-AUC"]  = np.nan
        metrics["Overall/PR-AUC"]   = np.nan
        metrics["Overall/MSE_prob"] = np.nan
        metrics["Overall/RMSE_prob"]= np.nan
    return metrics

def make_class_weight(y):
    y  = np.asarray(y).astype(int).ravel()
    n0 = max(1, (y==0).sum()); n1 = max(1, (y==1).sum()); N = n0+n1
    w0 = N/(2.0*n0); w1 = N/(2.0*n1)
    return {0: float(w0), 1: float(w1)}

def augment_with_static(X_seq, X_stat, y, sigma=AUGMENT_SIGMA):
    if sigma is None or sigma <= 0:
        return X_seq, X_stat, y
    noise = np.random.normal(0, sigma, X_seq.shape)
    X_seq_aug = np.vstack([X_seq, X_seq + noise])
    y_aug     = np.hstack([y, y])
    if X_stat is not None:
        X_stat_aug = np.vstack([X_stat, X_stat])
    else:
        X_stat_aug = None
    return X_seq_aug, X_stat_aug, y_aug

def seq_resample(X, y, method="none", random_state=RANDOM_STATE, return_index=False, allow_smote=True):
    X = np.asarray(X); y = np.asarray(y).astype(int).ravel()
    n, T, F = X.shape
    base_idx = np.arange(n)

    if method == "none":
        return (X, y, base_idx) if return_index else (X, y)

    if method in {"oversample_seq","undersample_seq"}:
        rng = np.random.default_rng(random_state)
        idx0 = np.where(y==0)[0]; idx1 = np.where(y==1)[0]
        n0, n1 = len(idx0), len(idx1)
        if n0==0 or n1==0:
            return (X, y, base_idx) if return_index else (X, y)

        if method == "oversample_seq":
            if n1 < n0:
                add = rng.choice(idx1, size=n0-n1, replace=True)
                keep = np.concatenate([idx0, idx1, add])
            else:
                add = rng.choice(idx0, size=n1-n0, replace=True)
                keep = np.concatenate([idx0, idx1, add])
        else:
            if n0 > n1:
                keep0 = rng.choice(idx0, size=n1, replace=False)
                keep  = np.concatenate([keep0, idx1])
            else:
                keep1 = rng.choice(idx1, size=n0, replace=False)
                keep  = np.concatenate([idx0, keep1])

        rng.shuffle(keep)
        Xr, yr = X[keep], y[keep]
        return (Xr, yr, keep) if return_index else (Xr, yr)

    if not allow_smote:
        print(f"⚠️ {method} disabled when static input is used; falling back to 'none'.")
        return (X, y, base_idx) if return_index else (X, y)

    minority_n = int((y==1).sum())
    majority_n = int((y==0).sum())
    if minority_n < 2 or majority_n < 2:
        print("⚠️ Not enough samples for SMOTE/SMOTEENN/SMOTETomek; skipping resampling.")
        return (X, y, base_idx) if return_index else (X, y)

    Xf = X.reshape(n, -1)
    if method == "smote":
        k_neighbors = max(1, min(5, minority_n-1))
        sm = SMOTE(random_state=random_state, k_neighbors=k_neighbors)
        Xr, yr = sm.fit_resample(Xf, y)
    elif method == "smoteenn":
        Xr, yr = SMOTEENN(random_state=random_state).fit_resample(Xf, y)
    elif method == "smotetomek":
        Xr, yr = SMOTETomek(random_state=random_state).fit_resample(Xf, y)
    else:
        raise ValueError(f"Unknown resampling method: {method}")

    Xr = Xr.reshape(-1, T, F)
    return (Xr, yr, None) if return_index else (Xr, yr)

def make_balanced_test(X_test, y_test, X_stat=None, random_state=RANDOM_STATE):
    X_test = np.asarray(X_test)
    y_test = np.asarray(y_test).astype(int).ravel()
    idx0, idx1 = np.where(y_test==0)[0], np.where(y_test==1)[0]
    if len(idx0)==0 or len(idx1)==0:
        return (X_test, y_test, (None if X_stat is None else X_stat))
    m = min(len(idx0), len(idx1))
    rs = np.random.RandomState(random_state)
    keep = np.concatenate([rs.choice(idx0, m, replace=False), rs.choice(idx1, m, replace=False)])
    rs.shuffle(keep)
    Xb, yb = X_test[keep], y_test[keep]
    Xsb = (None if X_stat is None else np.asarray(X_stat)[keep])
    return Xb, yb, Xsb

# ------------------------------------------------------
# Model builders (supports seq-only or seq+static)
# ------------------------------------------------------
def make_model(seq_len, n_seq_f, n_stat_f=0, arch="LSTM_100", lr=1e-3):
    seq_in = Input(shape=(seq_len, n_seq_f), name="seq_in")
    x = seq_in
    if arch == "BiLSTM":
        x = Bidirectional(LSTM(64, return_sequences=True))(x)
        x = Dropout(0.2)(x)
        x = Bidirectional(LSTM(32))(x)
        x = Dropout(0.2)(x)
        x = Dense(16, activation="relu")(x)
    elif arch == "LSTM_50":
        x = LSTM(50, return_sequences=True)(x); x = Dropout(0.2)(x)
        x = LSTM(25)(x);                    x = Dropout(0.2)(x)
        x = Dense(10, activation="relu")(x)
    elif arch == "LSTM_25_L1":
        x = LSTM(50, return_sequences=True, kernel_regularizer=l1(1e-5))(x); x = Dropout(0.2)(x)
        x = LSTM(25, kernel_regularizer=l1(1e-5))(x);                        x = Dropout(0.2)(x)
        x = Dense(10, activation="relu", kernel_regularizer=l1(1e-5))(x)
    elif arch == "LSTM_25_L2":
        x = LSTM(50, return_sequences=True, kernel_regularizer=l2(1e-5))(x); x = Dropout(0.2)(x)
        x = LSTM(25, kernel_regularizer=l2(1e-5))(x);                        x = Dropout(0.2)(x)
        x = Dense(10, activation="relu", kernel_regularizer=l2(1e-5))(x)
    else:  # LSTM_100
        x = LSTM(100, return_sequences=True)(x); x = Dropout(0.2)(x)
        x = LSTM(50)(x);                          x = Dropout(0.2)(x)
        x = Dense(25, activation="relu")(x)

    if n_stat_f and n_stat_f > 0 and USE_STATIC_INPUT:
        stat_in = Input(shape=(n_stat_f,), name="stat_in")
        s = Dense(32, activation="relu")(stat_in)
        s = Dropout(0.2)(s)
        h = Concatenate()([x, s])
        h = Dense(32, activation="relu")(h)
        out = Dense(1, activation="sigmoid")(h)
        model = Model(inputs=[seq_in, stat_in], outputs=out)
    else:
        h = Dense(32, activation="relu")(x)
        out = Dense(1, activation="sigmoid")(h)
        model = Model(inputs=seq_in, outputs=out)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  loss=focal_loss(), metrics=["accuracy"])
    return model

# ==========================
# Significance / Uncertainty
# ==========================
def delong_bootstrap_auc(y_true, p1, p2, n_boot=2000, random_state=42):
    """
    Bootstrap test for ΔAUC = AUC(p1) - AUC(p2).
    Returns (delta_auc, se, z, p_value_two_sided).
    """
    rng = np.random.default_rng(random_state)
    y_true = np.asarray(y_true).astype(int).ravel()
    p1 = np.asarray(p1).astype(float).ravel()
    p2 = np.asarray(p2).astype(float).ravel()

    diffs = []
    n = len(y_true)
    for _ in range(n_boot):
        idx = rng.integers(0, n, n)
        yb, p1b, p2b = y_true[idx], p1[idx], p2[idx]
        try:
            diffs.append(roc_auc_score(yb, p1b) - roc_auc_score(yb, p2b))
        except ValueError:
            continue
    diffs = np.array(diffs, dtype=float)
    delta = float(np.nanmean(diffs))
    se    = float(np.nanstd(diffs, ddof=1))
    z     = 0.0 if se == 0.0 else delta / se
    pval  = 2.0 * (1.0 - norm.cdf(abs(z)))
    return delta, se, z, pval

def bootstrap_ci_auc(y_true, p, n_boot=2000, alpha=0.05, random_state=42):
    """
    Percentile bootstrap CI for ROC-AUC. Returns (auc_hat, [lo, hi]).
    """
    rng = np.random.default_rng(random_state)
    y_true = np.asarray(y_true).astype(int).ravel()
    p = np.asarray(p).astype(float).ravel()

    stats = []
    n = len(y_true)
    for _ in range(n_boot):
        idx = rng.integers(0, n, n)
        yb, pb = y_true[idx], p[idx]
        try:
            stats.append(roc_auc_score(yb, pb))
        except ValueError:
            continue
    stats = np.array(stats, dtype=float)
    auc_hat = float(roc_auc_score(y_true, p))
    lo = float(np.nanpercentile(stats, 2.5))
    hi = float(np.nanpercentile(stats, 97.5))
    return auc_hat, [lo, hi]

# ------------------------------------------------------
# Training runner (VAL for threshold; TEST for final)
# ------------------------------------------------------
def run_balanced_lstm_pipeline(data,
                               arch_list=("LSTM_100","BiLSTM","LSTM_50"),
                               resample_methods=RESAMPLE_METHODS,
                               thr_min=THR_MIN, thr_max=THR_MAX,
                               random_state=RANDOM_STATE,
                               results_csv=OUT_RESULTS_CSV,
                               plots_png=OUT_PLOTS_PNG):

    os.makedirs(os.path.dirname(results_csv), exist_ok=True)
    os.makedirs(os.path.dirname(plots_png), exist_ok=True)
    os.makedirs("checkpoints", exist_ok=True)

    Xtr, Xtr_stat, ytr = data["train"]["X_seq"], data["train"]["X_stat"], data["train"]["y"]
    Xva, Xva_stat, yva = data["val"]["X_seq"],   data["val"]["X_stat"],   data["val"]["y"]
    Xte, Xte_stat, yte = data["test"]["X_seq"],  data["test"]["X_stat"],  data["test"]["y"]

    Xtr, Xtr_stat, ytr = augment_with_static(Xtr, Xtr_stat, ytr, sigma=AUGMENT_SIGMA)

    Xte_bal, yte_bal, Xte_stat_bal = make_balanced_test(Xte, yte, X_stat=Xte_stat)

    results     = {}
    roc_curves  = {}
    pr_curves   = {}

    # For post-hoc significance tests
    prob_store = {}  # (method, arch, split) -> (y_true, y_prob)

    allow_smote = (Xtr_stat is None or not USE_STATIC_INPUT)

    def train_eval_one(method_name, arch_name):
        nonlocal Xtr, ytr, Xtr_stat

        Xrs, yrs, idx_map = seq_resample(Xtr, ytr, method=method_name, random_state=random_state,
                                         return_index=True, allow_smote=allow_smote)
        if Xtr_stat is not None and USE_STATIC_INPUT:
            Xrs_stat = Xtr_stat if idx_map is None else Xtr_stat[idx_map]
        else:
            Xrs_stat = None

        seq_len, n_seq_f = Xrs.shape[1], Xrs.shape[2]
        n_stat_f = 0 if (Xrs_stat is None or not USE_STATIC_INPUT) else Xrs_stat.shape[1]
        model = make_model(seq_len, n_seq_f, n_stat_f=n_stat_f, arch=arch_name, lr=1e-3)

        es = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True, verbose=1)
        ckpt_path = f"checkpoints/{method_name}__{arch_name}.h5"
        cp = ModelCheckpoint(ckpt_path, save_best_only=True, monitor="val_loss", verbose=0)

        if n_stat_f > 0 and USE_STATIC_INPUT:
            model.fit([Xrs, Xrs_stat], yrs,
                      validation_data=([Xva, Xva_stat], yva),
                      epochs=12, batch_size=64, callbacks=[es, cp],
                      class_weight=make_class_weight(yrs), verbose=1)
            p_tr  = model.predict([Xrs, Xrs_stat], verbose=0).ravel()
            p_va  = model.predict([Xva, Xva_stat], verbose=0).ravel()
            p_te  = model.predict([Xte, Xte_stat], verbose=0).ravel()
            p_teB = model.predict([Xte_bal, Xte_stat_bal], verbose=0).ravel() if Xte_stat_bal is not None else model.predict(Xte_bal, verbose=0).ravel()
        else:
            model.fit(Xrs, yrs,
                      validation_data=(Xva, yva),
                      epochs=12, batch_size=64, callbacks=[es, cp],
                      class_weight=make_class_weight(yrs), verbose=1)
            p_tr  = model.predict(Xrs, verbose=0).ravel()
            p_va  = model.predict(Xva, verbose=0).ravel()
            p_te  = model.predict(Xte, verbose=0).ravel()
            p_teB = model.predict(Xte_bal, verbose=0).ravel()

        # thresholds on VAL
        try:
            fpr_va, tpr_va, thr_roc_va = roc_curve(yva, p_va); auc_roc = auc(fpr_va, tpr_va)
        except ValueError:
            fpr_va, tpr_va, thr_roc_va, auc_roc = np.array([0,1]), np.array([0,1]), np.array([0.5]), np.nan
        youden_va = tpr_va - fpr_va
        t_roc, _ = _best_threshold_in_range(thr_roc_va, youden_va, thr_min, thr_max)

        prec_va, rec_va, thr_pr_va = precision_recall_curve(yva, p_va)
        f1s_va = 2*prec_va[:-1]*rec_va[:-1] / (prec_va[:-1]+rec_va[:-1]+1e-8)
        t_pr, _ = _best_threshold_in_range(thr_pr_va, f1s_va, thr_min, thr_max)
        ap_val  = average_precision_score(yva, p_va)

        roc_curves[(method_name, arch_name)] = (fpr_va, tpr_va, auc_roc)
        pr_curves[(method_name, arch_name)]  = (rec_va, prec_va, ap_val)
        print(f"📌 [{method_name}/{arch_name}] VAL thresholds → Youden={t_roc:.4f}, PR-F1={t_pr:.4f} (window [{thr_min},{thr_max}])")

        eval_ts = sorted(set([thr_min, 0.50, thr_max, float(t_roc), float(t_pr)]))

        for t in eval_ts:
            yhat_tr  = (p_tr  >= t).astype(int)
            yhat_va  = (p_va  >= t).astype(int)
            yhat_te  = (p_te  >= t).astype(int)
            yhat_teB = (p_teB >= t).astype(int)

            results[f"{method_name}__{arch_name}__thr_{t:.2f}__train"]        = evaluate_full_metrics(yrs,     yhat_tr,  p_tr)
            results[f"{method_name}__{arch_name}__thr_{t:.2f}__val"]          = evaluate_full_metrics(yva,     yhat_va,  p_va)
            results[f"{method_name}__{arch_name}__thr_{t:.2f}__test"]         = evaluate_full_metrics(yte,     yhat_te,  p_te)
            results[f"{method_name}__{arch_name}__thr_{t:.2f}__testBalanced"] = evaluate_full_metrics(yte_bal, yhat_teB, p_teB)

        # --- store probabilities for A/B significance (threshold-free AUC) ---
        prob_store[(method_name, arch_name, "test")]         = (yte,     p_te)
        prob_store[(method_name, arch_name, "testBalanced")] = (yte_bal, p_teB)

    # Loop: resampling methods × architectures
    for METHOD in resample_methods:
        if METHOD in {"smote","smoteenn","smotetomek"} and (data["train"]["X_stat"] is not None and USE_STATIC_INPUT):
            print(f"⏭️  Skipping {METHOD} (static input enabled).")
            continue
        print(f"\n🔁 Resampling: {METHOD} | train y-dist = {Counter(data['train']['y'])}")
        for ARCH in arch_list:
            train_eval_one(METHOD, ARCH)

    # --- Plots (validation curves)
    plt.figure(figsize=(14,6))
    plt.subplot(1,2,1)
    for (meth, arch), (fpr, tpr, auc_roc) in roc_curves.items():
        plt.plot(fpr, tpr, label=f'{meth}/{arch} (VAL AUC={auc_roc:.3f})')
    plt.plot([0,1],[0,1],'--',label='Random')
    plt.xlabel('FPR'); plt.ylabel('TPR'); plt.title('ROC (Validation)'); plt.legend(fontsize=8)
    plt.subplot(1,2,2)
    for (meth, arch), (rec, prec, ap) in pr_curves.items():
        plt.plot(rec, prec, label=f'{meth}/{arch} (VAL AP={ap:.3f})')
    plt.xlabel('Recall'); plt.ylabel('Precision'); plt.title('PR (Validation)'); plt.legend(fontsize=8)
    plt.tight_layout(); plt.savefig(plots_png, dpi=300); plt.show()
    print(f"🖼️ Saved plots → {plots_png}")

    # --- Results CSV
    results_df = pd.DataFrame(results).T.reset_index().rename(columns={"index":"Key"})
    k = results_df["Key"].str.strip()
    results_df["Split"]  = np.where(k.str.endswith("__train"), "train",
                             np.where(k.str.endswith("__val"), "val",
                             np.where(k.str.endswith("__testBalanced"), "testBalanced",
                             np.where(k.str.endswith("__test"), "test", np.nan))))
    parts = k.str.split("__")
    results_df["Method"]    = parts.str[0]
    results_df["Model"]     = parts.str[1]
    results_df["Threshold"] = pd.to_numeric(parts.str[2].str.replace("thr_","", regex=False), errors="coerce")
    results_df.round(6).to_csv(results_csv, index=False)
    print(f"📑 Saved results → {results_csv}")

    # ================================
    # Pairwise AUC significance (A/B)
    # ================================
    pairs_to_compare = [
        (("none","LSTM_100"), ("oversample_seq","LSTM_100")),
        (("none","LSTM_100"), ("undersample_seq","BiLSTM")),
        (("oversample_seq","LSTM_100"), ("undersample_seq","BiLSTM")),
    ]
    splits_to_use = ["test", "testBalanced"]

    sig_rows = []
    for split_name in splits_to_use:
        for (A, B) in pairs_to_compare:
            methA, archA = A
            methB, archB = B
            keyA = (methA, archA, split_name)
            keyB = (methB, archB, split_name)
            if keyA not in prob_store or keyB not in prob_store:
                continue
            (yA, pA) = prob_store[keyA]
            (yB, pB) = prob_store[keyB]
            y_true = yA  # same split -> same ground truth
            delta, se, z, p = delong_bootstrap_auc(y_true, pA, pB, n_boot=2000, random_state=random_state)
            aucA, ciA = bootstrap_ci_auc(y_true, pA, n_boot=2000, alpha=0.05, random_state=random_state)
            aucB, ciB = bootstrap_ci_auc(y_true, pB, n_boot=2000, alpha=0.05, random_state=random_state)
            sig_rows.append({
                "Split": split_name,
                "ModelA": f"{methA}/{archA}",
                "ModelB": f"{methB}/{archB}",
                "AUC_A": aucA, "AUC_A_CI95_L": ciA[0], "AUC_A_CI95_U": ciA[1],
                "AUC_B": aucB, "AUC_B_CI95_L": ciB[0], "AUC_B_CI95_U": ciB[1],
                "Delta_AUC": delta, "SE_Delta": se, "Z": z, "P_value": p
            })

    sig_df = pd.DataFrame(sig_rows)
    out_sig_csv = (os.path.join(os.path.dirname(results_csv), "auc_significance.csv")
                   if os.path.dirname(results_csv) else "auc_significance.csv")
    sig_df.to_csv(out_sig_csv, index=False)
    print(f"📑 Saved AUC significance table → {out_sig_csv}")
    if not sig_df.empty:
        print(sig_df.head(10).to_string(index=False))

    return results_df

# --------------------
# Run end-to-end
# --------------------
if __name__ == "__main__":
    hourly, splits = build_hourly_features_with_leak_safe_transforms(
        in_csv=CSV_INTRADAY_WITH_VISITS,
        out_csv=OUT_HOURLY_CSV,
        min_cgm_per_hour=MIN_CGM_PER_H,
        test_size=0.2, val_size=0.1, random_state=RANDOM_STATE,
        static_csv=STATIC_CSV, visit_wide_csv=VISIT_WIDE_CSV, visit_long_csv=VISIT_LONG_CSV
    )

    data = build_sequences_by_split(
        hourly, splits,
        seq_len=SEQ_LEN,
        seq_feature_cols=DEFAULT_SEQ_FEATURE_COLS,
        static_cols=STATIC_COLS,
        scale_features=True
    )

    np.savez_compressed(
        OUT_SEQ_NPZ,
        Xtr=data["train"]["X_seq"],  Xtr_stat=(data["train"]["X_stat"] if data["train"]["X_stat"] is not None else np.empty((0,0))),
        ytr=data["train"]["y"],
        Xva=data["val"]["X_seq"],    Xva_stat=(data["val"]["X_stat"] if data["val"]["X_stat"] is not None else np.empty((0,0))),
        yva=data["val"]["y"],
        Xte=data["test"]["X_seq"],   Xte_stat=(data["test"]["X_stat"] if data["test"]["X_stat"] is not None else np.empty((0,0))),
        yte=data["test"]["y"],
        seq_features_used=np.array(data["seq_features_used"], dtype=object),
        static_features_used=np.array(data["static_features_used"], dtype=object)
    )
    print(f"💾 Saved sequences → {OUT_SEQ_NPZ}")

    results_df = run_balanced_lstm_pipeline(
        data,
        arch_list=("LSTM_100","BiLSTM","LSTM_50"),
        resample_methods=RESAMPLE_METHODS,
        thr_min=THR_MIN, thr_max=THR_MAX,
        random_state=RANDOM_STATE,
        results_csv=OUT_RESULTS_CSV,
        plots_png=OUT_PLOTS_PNG
    )


In [ ]:
# ============================================================
# Cross-Phase Validation (Leak-safe)
#   Train: Ramadan  → Test: Shawwal
#   Train: Shawwal  → Test: Ramadan
# Models: LSTM_100, BiLSTM, LSTM_50, LSTM_25_L2
# ============================================================

import os, re, random, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, roc_curve, precision_recall_curve,
    average_precision_score, mean_squared_error
)

import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2

# --------------------------
# CONFIG
# --------------------------
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE); random.seed(RANDOM_STATE); tf.random.set_seed(RANDOM_STATE)

CSV_INTRADAY_WITH_VISITS = "/kaggle/input/hmcdataset/intraday_with_visits.csv"
# If you don't have static/visit files ready, leave them None
STATIC_CSV      = None
VISIT_WIDE_CSV  = None
VISIT_LONG_CSV  = None

RAMADAN_START  = pd.to_datetime("2023-03-22"); RAMADAN_END  = pd.to_datetime("2023-04-19")
SHAWWAL_START  = pd.to_datetime("2023-04-20"); SHAWWAL_END  = pd.to_datetime("2023-05-19")

SEQ_LEN       = 24
HYPO_CUTOFF   = 70.0
MIN_CGM_PER_H = 4
VAL_FRACTION  = 0.15  # within *training phase* patients

USE_STATIC_INPUT = False  # this script runs seq-only (set True if you extend with static branch)
ARCH_LIST = ("LSTM_100","BiLSTM","LSTM_50","LSTM_25_L2")
THR_MIN, THR_MAX = 0.40, 0.60

# Features (sequence)
LIFESTYLE_COLS_CANDIDATES = ["steps","distance","calories","heart_rate","spo2","deep","light","rem","nap","awake"]
DEFAULT_SEQ_FEATURE_COLS = (
    "cgm_mean","cgm_std","pca_cgm1","pc1_activity_energy"
)

# --------------------------
# UTILS
# --------------------------
def _norm_col(s: str) -> str:
    return re.sub(r'[^a-z0-9]+', '', str(s).lower())

def to_dt(x, utc_ok=True):
    return pd.to_datetime(x, errors="coerce", utc=utc_ok)

def ensure_numeric(df, exclude=("patientID","huaweiID","visit_assigned","period_main","start","date","hour","hour_of_day")):
    ex = set(exclude)
    for c in df.columns:
        if c not in ex:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

def focal_loss(gamma=2.0, alpha=0.25):
    bce = tf.keras.losses.BinaryCrossentropy(from_logits=False, reduction=tf.keras.losses.Reduction.NONE)
    eps = tf.keras.backend.epsilon()
    def loss(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, eps, 1.0 - eps)
        ce = bce(y_true, y_pred)
        p_t = y_true * y_pred + (1.0 - y_true) * (1.0 - y_pred)
        alpha_t = y_true * alpha + (1.0 - y_true) * (1.0 - alpha)
        modulating = tf.pow(1.0 - p_t, gamma)
        return alpha_t * modulating * ce
    return loss

def make_class_weight(y):
    y  = np.asarray(y).astype(int).ravel()
    n0 = max(1, (y==0).sum()); n1 = max(1, (y==1).sum()); N = n0+n1
    return {0: float(N/(2.0*n0)), 1: float(N/(2.0*n1))}

# --------------------------
# DATA: build hourly per window
# --------------------------
def build_hourly_for_window(in_csv, start_date, end_date, min_cgm_per_hour=MIN_CGM_PER_H):
    if not os.path.exists(in_csv):
        raise FileNotFoundError(in_csv)
    df = pd.read_csv(in_csv)

    # normalize patient column
    if "patientID" not in df.columns:
        for c in df.columns:
            if _norm_col(c).startswith("patientid") or _norm_col(c) in {"patientid","id","huaweiid"}:
                df = df.rename(columns={c:"patientID"})
                break
        if "patientID" not in df.columns:
            raise KeyError("patientID column not found.")

    # time columns
    time_col = "start" if "start" in df.columns else ("timestamp" if "timestamp" in df.columns else None)
    if time_col is None and "date" not in df.columns:
        raise KeyError("No start/timestamp/date column found.")
    if time_col is not None:
        df[time_col] = to_dt(df[time_col])
        df["date"] = pd.to_datetime(df[time_col].dt.date)
        df["hour"] = df[time_col].dt.floor("h")
    else:
        df["date"] = pd.to_datetime(df["date"], errors="coerce")
        df["hour"] = df["date"].dt.floor("h")

    df["hour_of_day"] = pd.to_datetime(df["hour"]).dt.hour
    df = ensure_numeric(df)

    # filter window
    df = df[(df["date"] >= start_date) & (df["date"] <= end_date)].copy()

    # CGM present?
    if "cgm" not in df.columns:
        raise ValueError("Dataset must include 'cgm' column.")
    df_cgm = df.dropna(subset=["cgm"]).copy()

    # valid hours
    valid_hours = (
        df_cgm.groupby(["patientID","hour"])
              .filter(lambda g: g["cgm"].notna().sum() >= min_cgm_per_hour)
    )

    # hourly summaries
    hourly = (
        valid_hours.groupby(["patientID","hour"], as_index=False)
                   .agg(cgm_min=("cgm","min"),
                        cgm_max=("cgm","max"),
                        cgm_mean=("cgm","mean"),
                        cgm_std=("cgm","std"))
                   .sort_values(["patientID","hour"]).reset_index(drop=True)
    )
    hourly["hour_of_day"] = pd.to_datetime(hourly["hour"]).dt.hour

    # labels
    lab = (
        valid_hours.groupby(["patientID","hour"])["cgm"]
                   .apply(lambda x: int((x < HYPO_CUTOFF).any()))
                   .reset_index(name="hypo_label")
    )
    hourly = hourly.merge(lab, on=["patientID","hour"], how="left")

    # composites
    hourly["cgm_mean_plus_std"]  = hourly["cgm_mean"] + hourly["cgm_std"]
    hourly["cgm_mean_minus_std"] = hourly["cgm_mean"] - hourly["cgm_std"]

    # lifestyle means (if present)
    life_cols = [c for c in LIFESTYLE_COLS_CANDIDATES if c in df_cgm.columns]
    if life_cols:
        life_hourly = (df_cgm.groupby(["patientID","hour"], as_index=False)[life_cols].mean().fillna(0.0))
        hourly = hourly.merge(life_hourly, on=["patientID","hour"], how="left")
    else:
        for c in ["steps","distance","calories","heart_rate","spo2","deep","light","rem","nap","awake"]:
            if c not in hourly.columns:
                hourly[c] = 0.0

    hourly["date"] = pd.to_datetime(pd.to_datetime(hourly["hour"]).dt.date)
    return hourly

# --------------------------
# Fit leak-safe PCA (train phase only) and add PCA cols
# --------------------------
def apply_leak_safe_pca(hourly, lifestyle_cols=None):
    hourly = hourly.copy()
    assert "Split" in hourly.columns, "hourly must have Split"
    cgm_cols = ["cgm_min","cgm_max","cgm_mean","cgm_std"]
    tr_mask  = hourly["Split"].astype(str).str.lower().eq("train")

    # CGM PCA
    scal_cgm = StandardScaler().fit(hourly.loc[tr_mask, cgm_cols].fillna(0.0))
    pca_cgm  = PCA(n_components=3, random_state=RANDOM_STATE).fit(
        scal_cgm.transform(hourly.loc[tr_mask, cgm_cols].fillna(0.0))
    )
    X_all = scal_cgm.transform(hourly[cgm_cols].fillna(0.0))
    Z_all = pca_cgm.transform(X_all)
    hourly["pca_cgm1"], hourly["pca_cgm2"], hourly["pca_cgm3"] = Z_all[:,0], Z_all[:,1], Z_all[:,2]

    # Lifestyle PCA (if present)
    if lifestyle_cols:
        scal_life = StandardScaler().fit(hourly.loc[tr_mask, lifestyle_cols].fillna(0.0))
        ZL = PCA(n_components=3, random_state=RANDOM_STATE).fit_transform(
            scal_life.transform(hourly[lifestyle_cols].fillna(0.0))
        )
        hourly["pc1_activity_energy"], hourly["pc2_physiology"], hourly["pc3_sleep_rest"] = ZL[:,0], ZL[:,1], ZL[:,2]
    else:
        hourly["pc1_activity_energy"] = 0.0

    return hourly

# --------------------------
# Build sequences by split
# --------------------------
def build_sequences(hourly, seq_len=SEQ_LEN, seq_feature_cols=DEFAULT_SEQ_FEATURE_COLS):
    for c in ["patientID","hour","hypo_label","Split"]:
        if c not in hourly.columns:
            raise KeyError(f"hourly missing {c}")

    hourly = hourly.copy()
    hourly["hour"] = pd.to_datetime(hourly["hour"], errors="coerce")

    missing = [c for c in seq_feature_cols if c not in hourly.columns]
    if missing:
        raise KeyError(f"Missing seq features: {missing}")

    train_p = sorted(hourly.loc[hourly["Split"].eq("train"), "patientID"].unique())
    val_p   = sorted(hourly.loc[hourly["Split"].eq("val"),   "patientID"].unique())
    test_p  = sorted(hourly.loc[hourly["Split"].eq("test"),  "patientID"].unique())

    def _build_for(pid_list):
        sub = hourly[hourly["patientID"].isin(pid_list)].sort_values(["patientID","hour"]).reset_index(drop=True)
        X, y = [], []
        for pid, grp in sub.groupby("patientID", sort=True):
            grp = grp.sort_values("hour").reset_index(drop=True)
            if len(grp) <= seq_len: continue
            feats  = grp[list(seq_feature_cols)].astype(float).values
            labels = grp["hypo_label"].astype(int).values
            for i in range(len(grp)-seq_len):
                X.append(feats[i:i+seq_len]); y.append(labels[i+seq_len])
        return (np.array(X), np.array(y).astype(int))

    Xtr, ytr = _build_for(train_p)
    Xva, yva = _build_for(val_p)
    Xte, yte = _build_for(test_p)

    # scaler on TRAIN only (sequence features)
    seq_scaler = None
    if Xtr.size > 0:
        F = Xtr.shape[2]
        seq_scaler = StandardScaler().fit(Xtr.reshape(-1, F))
        def _scale(X):
            if X is None or X.size==0: return X
            n, T, F = X.shape
            return seq_scaler.transform(X.reshape(-1, F)).reshape(n, T, F)
        Xtr = _scale(Xtr); Xva = _scale(Xva); Xte = _scale(Xte)

    return {
        "train": {"X_seq": Xtr, "y": ytr},
        "val":   {"X_seq": Xva, "y": yva},
        "test":  {"X_seq": Xte, "y": yte},
        "seq_features_used": list(seq_feature_cols),
        "scalers": {"seq": seq_scaler}
    }

# --------------------------
# Models
# --------------------------
def make_model(seq_len, n_seq_f, arch="LSTM_100", lr=1e-3):
    seq_in = Input(shape=(seq_len, n_seq_f), name="seq_in")
    x = seq_in
    if arch == "BiLSTM":
        x = Bidirectional(LSTM(64, return_sequences=True))(x); x = Dropout(0.2)(x)
        x = Bidirectional(LSTM(32))(x); x = Dropout(0.2)(x)
        x = Dense(16, activation="relu")(x)
    elif arch == "LSTM_50":
        x = LSTM(50, return_sequences=True)(x); x = Dropout(0.2)(x)
        x = LSTM(25)(x); x = Dropout(0.2)(x)
        x = Dense(10, activation="relu")(x)
    elif arch == "LSTM_25_L2":
        x = LSTM(50, return_sequences=True, kernel_regularizer=l2(1e-5))(x); x = Dropout(0.2)(x)
        x = LSTM(25, kernel_regularizer=l2(1e-5))(x); x = Dropout(0.2)(x)
        x = Dense(10, activation="relu", kernel_regularizer=l2(1e-5))(x)
    else:  # LSTM_100
        x = LSTM(100, return_sequences=True)(x); x = Dropout(0.2)(x)
        x = LSTM(50)(x); x = Dropout(0.2)(x)
        x = Dense(25, activation="relu")(x)

    out = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=seq_in, outputs=out)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  loss=focal_loss(), metrics=["accuracy"])
    return model

# --------------------------
# Threshold selection & evaluation
# --------------------------
def _best_threshold_in_range(thresholds, scores, thr_min=THR_MIN, thr_max=THR_MAX):
    thresholds = np.asarray(thresholds, dtype=float)
    scores     = np.asarray(scores, dtype=float)
    mask = np.isfinite(thresholds) & (thresholds >= thr_min) & (thresholds <= thr_max)
    if mask.any():
        idx = np.where(mask)[0][int(np.nanargmax(scores[mask]))]
        return float(thresholds[idx])
    # fallback: best overall, then clamp
    idx = int(np.nanargmax(scores))
    return float(np.clip(thresholds[idx], thr_min, thr_max))

def pick_val_thresholds(y_val, p_val, thr_min=THR_MIN, thr_max=THR_MAX):
    # Youden's J
    try:
        fpr, tpr, thr_roc = roc_curve(y_val, p_val)
        youden = tpr - fpr
        t_roc = _best_threshold_in_range(thr_roc, youden, thr_min, thr_max)
    except Exception:
        t_roc = 0.50
    # PR-F1
    try:
        prec, rec, thr_pr = precision_recall_curve(y_val, p_val)
        f1s = 2*prec[:-1]*rec[:-1] / (prec[:-1]+rec[:-1]+1e-9)
        t_pr = _best_threshold_in_range(thr_pr, f1s, thr_min, thr_max)
    except Exception:
        t_pr = 0.50
    return t_roc, t_pr

def eval_probs(y_true, y_prob, thresholds=(0.40, 0.50, 0.60)):
    """
    Evaluate thresholded metrics at specified operating points, plus AUC/PR-AUC/Brier (threshold-free).
    """
    y_true = np.asarray(y_true).astype(int).ravel()
    y_prob = np.asarray(y_prob).astype(float).ravel()

    try:
        roc = roc_auc_score(y_true, y_prob)
    except Exception:
        roc = np.nan
    try:
        pr  = average_precision_score(y_true, y_prob)
    except Exception:
        pr  = np.nan
    brier = mean_squared_error(y_true, y_prob)

    rows = []
    for t in thresholds:
        yhat = (y_prob >= float(t)).astype(int)
        rows.append({
            "Threshold": round(float(t), 2),
            "Accuracy": accuracy_score(y_true, yhat),
            "F1_weighted": f1_score(y_true, yhat, average="weighted", zero_division=0),
            "Prec_1": precision_score(y_true, yhat, pos_label=1, zero_division=0),
            "Recall_1": recall_score(y_true, yhat, pos_label=1, zero_division=0),
            "ROC_AUC": roc, "PR_AUC": pr, "Brier": brier
        })
    return pd.DataFrame(rows)

# --------------------------
# Core runner (one direction)
# --------------------------
def run_cross_phase(train_window, test_window, out_prefix="ram_to_sha"):
    """
    train_window/test_window: (start_date, end_date)
    """
    # A) Build hourly tables for both phases
    hourly_train = build_hourly_for_window(CSV_INTRADAY_WITH_VISITS, *train_window)
    hourly_test  = build_hourly_for_window(CSV_INTRADAY_WITH_VISITS, *test_window)

    # B) Create patient splits: TRAIN/VAL within train phase; TEST from test phase (no overlap)
    train_patients_all = sorted(hourly_train["patientID"].unique().tolist())
    test_patients_all  = sorted(hourly_test["patientID"].unique().tolist())

    # hold-out VAL from train phase (patient-wise)
    rng = np.random.default_rng(RANDOM_STATE)
    rng.shuffle(train_patients_all)
    n_val = max(1, int(len(train_patients_all) * VAL_FRACTION))
    val_patients = sorted(train_patients_all[:n_val])
    tr_patients  = sorted(train_patients_all[n_val:])

    # ensure cross-phase TEST patients don't overlap training phase patients
    test_only = sorted(list(set(test_patients_all) - set(tr_patients) - set(val_patients)))
    if len(test_only) == 0:
        # fallback: allow all test phase patients (still leak-free wrt transforms, but same subjects across phases)
        test_only = test_patients_all

    hourly_train["Split"] = np.where(hourly_train["patientID"].isin(tr_patients), "train",
                              np.where(hourly_train["patientID"].isin(val_patients), "val", "drop"))
    hourly_train = hourly_train[hourly_train["Split"] != "drop"].copy()

    hourly_test["Split"]  = np.where(hourly_test["patientID"].isin(test_only), "test", "drop")
    hourly_test = hourly_test[hourly_test["Split"] != "drop"].copy()

    # C) Merge to single hourly dataframe with Split: train/val/test
    hourly = pd.concat([hourly_train, hourly_test], ignore_index=True)

    # D) Leak-safe PCA (fit on TRAIN only)
    life_cols_present = [c for c in LIFESTYLE_COLS_CANDIDATES if c in hourly.columns]
    hourly = apply_leak_safe_pca(hourly, lifestyle_cols=life_cols_present)

    # E) Build sequences
    data = build_sequences(hourly, seq_len=SEQ_LEN, seq_feature_cols=DEFAULT_SEQ_FEATURE_COLS)
    Xtr, ytr = data["train"]["X_seq"], data["train"]["y"]
    Xva, yva = data["val"]["X_seq"],   data["val"]["y"]
    Xte, yte = data["test"]["X_seq"],  data["test"]["y"]

    if any(arr is None or arr.size == 0 for arr in [Xtr, ytr, Xva, yva, Xte, yte]):
        raise RuntimeError("Insufficient sequences in one of the splits. Check coverage or windows.")

    results_rows = []

    # F) Train/evaluate per architecture
    for arch in ARCH_LIST:
        model = make_model(seq_len=Xtr.shape[1], n_seq_f=Xtr.shape[2], arch=arch, lr=1e-3)
        es = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True, verbose=0)

        hist = model.fit(Xtr, ytr,
                         validation_data=(Xva, yva),
                         epochs=12, batch_size=64,
                         callbacks=[es],
                         class_weight=make_class_weight(ytr),
                         verbose=0)

        # Predict
        p_tr = model.predict(Xtr, verbose=0).ravel()
        p_va = model.predict(Xva, verbose=0).ravel()
        p_te = model.predict(Xte, verbose=0).ravel()

        # Pick thresholds on VAL
        t_roc, t_pr = pick_val_thresholds(yva, p_va, thr_min=THR_MIN, thr_max=THR_MAX)
        eval_df = eval_probs(yte, p_te, thresholds=(THR_MIN, 0.50, THR_MAX, t_roc, t_pr))
        eval_df.insert(0, "Model", arch)
        eval_df.insert(1, "Direction", out_prefix)
        results_rows.append(eval_df)

    results = pd.concat(results_rows, ignore_index=True)
    os.makedirs("/kaggle/working", exist_ok=True)
    out_csv = f"/kaggle/working/crossphase_{out_prefix}.csv"
    results.to_csv(out_csv, index=False)
    print(f"✅ Saved → {out_csv}")
    return results

# --------------------------
# RUN BOTH DIRECTIONS
# --------------------------
ramadan = (RAMADAN_START, RAMADAN_END)
shawwal  = (SHAWWAL_START, SHAWWAL_END)

res_ram_to_sha = run_cross_phase(ramadan, shawwal, out_prefix="Ramadan_to_Shawwal")
res_sha_to_ram = run_cross_phase(shawwal, ramadan, out_prefix="Shawwal_to_Ramadan")

# Combine + pivot for quick view
combined = pd.concat([res_ram_to_sha, res_sha_to_ram], ignore_index=True)
combined.to_csv("/kaggle/working/crossphase_combined.csv", index=False)

# Produce a compact comparison at VAL-optimized PR-F1 threshold (we pick the row with Threshold == t_pr ~= within [0.40,0.60])
def _pick_best_rows(df):
    # We approximated two VAL-optimal thresholds (t_roc, t_pr) and included both in eval table.
    # Here we pick the row with the highest F1_weighted per Model/Direction.
    key_cols = ["Direction","Model"]
    best = (df.sort_values(["Direction","Model","F1_weighted"], ascending=[True, True, False])
              .groupby(key_cols, as_index=False)
              .head(1)
              .reset_index(drop=True))
    return best[["Direction","Model","Threshold","ROC_AUC","PR_AUC","F1_weighted","Recall_1","Prec_1","Brier"]]

summary = _pick_best_rows(combined)
summary.to_csv("/kaggle/working/crossphase_summary_best.csv", index=False)
print("🧾 Summary (best per Model/Direction):")
print(summary.to_string(index=False))


2025-10-29 08:12:25.218950: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761725545.472809      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761725545.544014      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1761725576.557517      37 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1761725582.008243     110 cuda_dnn.cc:529] Loaded cuDNN version 90300
